# Exploring COVID-19 and dengue numbers: the data acquisition process

### Environment log:

(1) Using the "analytics3" anaconda environment; installing python packages directly through *pip* command.  
(2) Installed *pyvirtualenv*, *selenium*, and *[bs4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#)* packages.  
(3) Installed *chromedriver* through "pip install chromedriver".  

(4) Got this error when initializing the Display function:
> display = Display(visible=0, size=(800,600))
> display.start()  
>> FileNotFoundError: [Errno 2] No such file or directory: 'Xvfb': 'Xvfb'
>> EasyProcessError: start error EasyProcess cmd_param=['Xvfb', '-help'] cmd=['Xvfb', '-help'] oserror=[Errno 2] No such file or directory: 'Xvfb': 'Xvfb' return_code=None stdout="None" stderr="None" timeout_happened=False>

**Solution found in: https://bugs.debian.org/cgi-bin/bugreport.cgi?bug=860501**   
 
(5) Installed the system package xvfb through the shell line: *sudo apt-get install xvfb*.

- When running the "driver = webdriver.Chrome()" python command, I received the following error:  
> driver = webdriver.Chrome()
>> FileNotFoundError: [Errno 2] No such file or directory: 'chromedriver': 'chromedriver'  

(6) Even though, since I'm using conda environment I have set the chromedriver path before:  
> chromeDriverPath = '~/anaconda3/envs/anaytics3/'

The solution was to create a symbolic link in the path shown above, through the system bash, which made it work properly:  
> $ ln -s ~/anaconda3/envs/analytics3/chromedriver-Linux64 chromedriver  

(7) There are some broken links in the BMH oficial page: it misses the "https://portalarquivos.saude.gov.br/" portion of the link. We defined a function to address this issue.


## Epidemiological bulletins from World Health Organization - WHO
https://www.who.int/emergencies/diseases/novel-coronavirus-2019/situation-reports/  
https://github.com/danielsteman/COVID-19_WHO  
https://github.com/danielsteman/COVID-19_WHO/blob/master/WHO_webscrape.ipynb (the author uses PyPDF2)

## Epidemiological bulletins from Brazilian Ministry of Health
https://www.saude.gov.br/boletins-epidemiologicos
  - Coronavírus/COVID-19  
  - Dengue

### Initial statements

In [155]:
import time
import pdb   # Python debugger
from pyvirtualdisplay import Display
from selenium import webdriver
from bs4 import BeautifulSoup

In [156]:
# Definição de parâmetros
url = 'https://www.saude.gov.br/boletins-epidemiologicos'
chromeDriverPath = '~/anaconda3/envs/anaytics3/'

### Web browsing

In this step of our data crawling we want to access the webpage through the Chromium driver. It is then appropriate to create some functions both for the page access as for its content analysis. However, it would be only possible if we know the page structure. In order to reach it, at this time we will try to read and explore it, trying to find the patterns and links we are looking for.

In [157]:
# Display starting:
display = Display(visible=0, size=(800,600))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '800x600x24', ':1009'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '800x600x24', ':1009'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [158]:
# Open the Chromium.driver for the intended page:
driver = webdriver.Chrome()

In [159]:
# Reading the page content and saving it specifying its encoding scheme:
driver.get(url)
page = driver.page_source.encode('utf-8')

### Initial exploration

In [160]:
# How large is the loaded page?
print(len(page))
# Data streaming type is expected:
print(type(page))

485548
<class 'bytes'>


In [166]:
## Showing only the first 2k positions of the bytes stream:
print(page[:2000])

b'<!DOCTYPE html><!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="pt-br" dir="ltr"> <![endif]--><!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8" lang="pt-br" dir="ltr"> <![endif]--><!--[if IE 8]>         <html class="no-js lt-ie9" lang="pt-br" dir="ltr"> <![endif]--><!--[if gt IE 8]><!--><html xmlns="http://www.w3.org/1999/xhtml" class="no-js" lang="pt-br" dir="ltr"><!--<![endif]--><head>\n<!-- Google Tag Manager -->\n<script type="text/javascript" async="" src="https://www.google-analytics.com/gtm/js?id=GTM-W26S6PL&amp;t=gtm2&amp;cid=867200467.1587435801"></script><script type="text/javascript" async="" src="https://www.google-analytics.com/analytics.js"></script><script type="text/javascript" async="" src="https://www.google-analytics.com/analytics.js"></script><script src="https://connect.facebook.net/pt_BR/sdk.js?hash=6c4c5c725897ef3dcf2d8140087cfdd9&amp;ua=modern_es6" async="" crossorigin="anonymous"></script><script id="facebook-jssdk" src="//connect.

**Since the loaded page has too many links, as well as other contents we are not interested in, we will use BeautifulSoap to parse it in order to obtain the desired links.**

In [33]:
# Aplicando o *parsing* na página:
soup = BeautifulSoup(page)

/home/tbnsilveira/anaconda3/envs/analytics3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/tbnsilveira/anaconda3/envs/analytics3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [21]:
# Os links da página estão dispostos no seguinte formato: 
#</p><ul>
#<li><a href="nlpintro_co3_01.01_DR_Edit.pdf">01.01.pdf</a>
#</li><li><a href="nlpintro_co3_01.02_DR_Edit.pdf">01.02.pdf</a>*

In [23]:
#links = soup.findAll("div", {"class": "complaint-item ng-scope"}, recursive=True)

In [44]:
tags = soup('a')

In [45]:
print(type(tags))
print("Number of links on the page analyzed:", len(tags))

<class 'bs4.element.ResultSet'>
Number of links on the page analyzed: 1571


In [46]:
# Para obter apenas o nome do arquivo, i.e. o link que estava no campo href do 3o link:
tags[1000].get('href')

'/images/pdf/2016/marco/23/2016-008---DengueSE8-publica----o.pdf'

In [41]:
tags

[<a class="hide" href="#accessibility" id="topo">Ir direto para menu de acessibilidade.</a>,
 <a class="link-barra" href="https://gov.br">Brasil</a>,
 <a href="#" id="menu-icon"></a>,
 <a href="http://www.saude.gov.br/coronavirus" style="color:red;">CORONAVÍRUS (COVID-19)</a>,
 <a class="link-barra" href="http://www.simplifique.gov.br">Simplifique!</a>,
 <a class="link-barra" href="https://www.gov.br/pt-br/participacao-social/">Participe</a>,
 <a class="link-barra" href="http://www.acessoainformacao.gov.br">Acesso à informação</a>,
 <a class="link-barra" href="http://www.planalto.gov.br/legislacao">Legislação</a>,
 <a class="link-barra" href="https://gov.br/pt-br/canais-do-executivo-federal">Canais</a>,
 <a class="logo-vlibras" href="#" id="logovlibras"></a>,
 <a href="http://www.vlibras.gov.br">VLibras</a>,
 <a accesskey="1" href="#content" id="link-conteudo">
                                     Ir para o conteúdo
                                     <span>1</span>
 </a>,
 <a accessk

In [48]:
#soup = BeautifulSoup(''.join(page), 'lxml')
for link in soup.find_all('a', href=True):
    if link['href'].lower().endswith(".pdf"):
        if link['href'].lower().startswith("http"):
            print(link['href'])

https://portalarquivos.saude.gov.br/images/pdf/2020/April/19/BE12-Boletim-do-COE.pdf
https://portalarquivos.saude.gov.br/images/pdf/2020/April/17/2020-04-16---BE10---Boletim-do-COE-21h.pdf
https://portalarquivos.saude.gov.br/images/pdf/2020/April/12/2020-04-11-BE9-Boletim-do-COE.pdf
https://portalarquivos.saude.gov.br/images/pdf/2020/April/03/BE6-Boletim-Especial-do-COE.pdf
https://portalarquivos2.saude.gov.br/images/pdf/2020/fevereiro/13/Boletim-epidemiologico-COEcorona-SVS-13fev20.pdf
https://portalarquivos.saude.gov.br/images/pdf/2020/April/19/BE12-Boletim-do-COE.pdf
https://portalarquivos.saude.gov.br/images/pdf/2020/April/17/2020-04-16---BE10---Boletim-do-COE-21h.pdf
https://portalarquivos.saude.gov.br/images/pdf/2020/April/12/2020-04-11-BE9-Boletim-do-COE.pdf
https://portalarquivos2.saude.gov.br/images/pdf/2020/fevereiro/13/Boletim-epidemiologico-COEcorona-SVS-13fev20.pdf
http://saude.gov.br/images/pdf/2019/outubro/24/Boletim-epidemiologico-SVS-31.pdf
http://portalarquivos2.saude

### Getting experience with the BeautifulSoup:
https://stackoverflow.com/questions/11716380/beautifulsoup-extract-text-from-anchor-tag

In [65]:
data2 = '''
<div class="su-tabs-panes"><div class="su-tabs-pane su-clearfix" style="display: none;">
<p><a href="https://portalarquivos.saude.gov.br/images/pdf/2020/April/19/BE12-Boletim-do-COE.pdf">BE 12 - Boletim COE Coronavírus</a></p>
<p><a href="/images/pdf/2020/April/18/2020-04-17---BE11---Boletim-do-COE-21h.pdf" target="_blank">BE 11 - Boletim COE Coronavírus</a></p>
<p><a href="https://portalarquivos.saude.gov.br/images/pdf/2020/April/17/2020-04-16---BE10---Boletim-do-COE-21h.pdf" target="_blank" rel="noopener">BE 10 - Boletim COE Coronavirus</a></p>
<p><a href="/images/pdf/2020/April/16/Boletim-epidemiologico-SVS-16.pdf" target="_blank">BE Vol. 51 Nº 16 -&nbsp;Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes Aegypti (dengue, chikungunya e zika), Semanas Epidemiológicas 1 a 15, 2020</a></p>
<p><a href="https://portalarquivos.saude.gov.br/images/pdf/2020/April/12/2020-04-11-BE9-Boletim-do-COE.pdf" target="_blank" rel="noopener">BE9 - Boletim Especial do COE Coronavírus Avaliação de Risco</a></p>
<p><a href="/images/pdf/2020/April/13/Boletim-epidemiologico-SVS-15.pdf" target="_blank">BE Vol. 51 Nº 15 - Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes Aegypti (dengue, chikungunya e zika), Semanas Epidemiológicas 1 a 14 de 2020, e da febre amarela (2019/2020)</a></p>
<p><a href="/images/pdf/2020/April/09/be-covid-08-final-2.pdf" target="_blank">BE8 - Boletim Especial do COE Coronavírus Avaliação de Risco</a></p>
<p><a href="/images/pdf/2020/April/06/2020-04-06-BE7-Boletim-Especial-do-COE-Atualizacao-da-Avaliacao-de-Risco.pdf" target="_blank">BE7 - Boletim Especial do COE Coronavírus Avaliação de Risco</a></p>
<p><a href="https://portalarquivos.saude.gov.br/images/pdf/2020/April/03/BE6-Boletim-Especial-do-COE.pdf">BE6 - Boletim Especial do COE Coronavírus Avaliação de Risco</a></p>
<p><a href="/images/pdf/2020/April/03/Boletim-epidemiologico-SVS-14.pdf">BE Vol. 51 Nº 14 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 13, 2020; febre amarela; diferenças regionais no consumo de alimentos entre adolescentes e adultos e leishmaniose visceral: Experiência do IEC</a></p>
<p><a href="/images/pdf/2020/marco/27/Boletim-epidemiologico-SVS-13.pdf" target="_blank">BE Vol.51 Nº 13 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 12, 2020; febre amarela; análise da anomalias congênitas no Brasil; abastecimento de água para consumo humano em 2019 e medicamentos antileishmania no âmbito do SUS</a><a href="/images/pdf/2020/marco/24/Boletim-tuberculose-2020-marcas--1-.pdf" target="_blank"></a></p>
<p><a href="/images/pdf/2020/marco/24/Boletim-tuberculose-2020-marcas--1-.pdf" target="_blank">BE Número Especial – Tuberculose 2020</a></p>
<p><a href="/images/pdf/2020/marco/30/Boletim-epidemiologico-SVS-12.pdf">BE Vol. 51 Nº 12 - Monitoramento dos casos de arboviroses (dengue, chikungunya e zika), SE 1 a 11, 2020; febre amarela 2019/2020; sarampo: SE 1 a 10, 2020; ações de enfrentamento da tuberculose em imigrantes e Síndrome congênita associada à infecção pelo vírus Zika: SE 10, 2020</a></p>
<p><a href="/images/pdf/2020/marco/30/Boletim-epidemiologico-SVS11.pdf" target="_blank">BE Vol. 51 Nº 11 - Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e zika), SE 1 a 10, 2020; Vigilância da infecção latente pelo Mycobacterium tuberculosis; Doença de Chagas - Protocolos clínicos de seguimento pós-tratamento em grupos especiais</a></p>
<p><a href="/images/pdf/2020/marco/24/03--ERRATA---Boletim-Epidemiologico-05.pdf" target="_blank">BE Coronavírus - 13/03/2020</a></p>
<p><a href="/images/pdf/2020/marco/06/Boletim-epidemiologico-SVS-10.pdf" target="_blank">BE Vol. 51 Nº10 Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika), Semanas Epidemiológicas 1 a 7, 2020.</a></p>
<p><a href="/images/pdf/2020/marco/05/Boletim-epidemiologico-SVS-09--.pdf" target="_blank">BE Vol. 51 Nº 9 - Vigilância epidemiológica do sarampo no Brasil 2020: Semana Epidemiológica 01; situação epidemiológica da febre amarela 2019/2020; importância dos códigos garbage para a magnitude da mortalidade por causas externas no Brasil; tracoma e acidentes ofídicos no Brasil, 2018</a></p>
<p><a href="/images/pdf/2020/marco/04/2020-03-02-Boletim-Epidemiol--gico-04-corrigido.pdf">BE Coronavírus - 04/03/2020</a></p>
<p><a href="/images/pdf/2020/fevereiro/21/2020-02-21-Boletim-Epidemiologico03.pdf" target="_blank">BE Coronavírus - 21/02/2020</a></p>
<p><a href="/images/pdf/2020/fevereiro/21/Boletim-epidemiologico-SVS-08.pdf" target="_blank">BE Vol. 51 Nº 8 - Vacinação para o viajante e população em geral; orientações para prevenção de doenças transmissíveis e acidentes por animais peçonhentos; vigilância das IST; uso de álcool: alerta para o período do carnaval</a></p>
<p><a href="/images/pdf/2020/fevereiro/19/Boletim-epidemiologico-SVS-07.pdf" target="_blank">BE Vol. 51 Nº 7 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 05; síndrome congênita associada à infecção pelo vírus Zika até a SE 05 e mobilização nacional para enfrentamento da hanseníase</a></p>
<p><a href="/images/pdf/2020/April/08/Boletim-epidemiologico-SVS-06-v2.pdf" target="_blank" rel="noopener">BE Vol. 51 Nº 6 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 04; sarampo no Brasil 2019: janeiro a dezembro e situação da febre amarela: monitoramento 2019/2020</a></p>
<p><a href="https://portalarquivos2.saude.gov.br/images/pdf/2020/fevereiro/13/Boletim-epidemiologico-COEcorona-SVS-13fev20.pdf" target="_blank" rel="noopener">BE Coronavírus - 10/02/2020</a></p>
<p><a href="/images/pdf/2020/fevereiro/18/Boletim-epidemiologico-SVS-01-COE-inundacao.pdf">BE COE Inundação nº 01 - Gestão de Emergências em Saúde Pública por desastres: evento hidrometeorológico na região Sudeste do Brasil, 2020</a></p>
<p><a href="/images/pdf/2020/fevereiro/05/Boletim-epidemiologico-SVS-05.pdf">BE Vol. 51 Nº 5 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 03; Síndrome Congênita associada à infecção pelo vírus Zika: monitoramento do ano de 2019 e a qualidade da água para consumo humano no Rio de Janeiro.</a></p>
<p><a href="/images/pdf/2020/janeiro/31/Boletim-hanseniase-2020-web.pdf" target="_blank">BE Número Especial - Hanseníase</a></p>
<p><a href="/images/pdf/2020/janeiro/28/boletim-especial-27jan20-final.pdf" target="_blank">BE Número Especial - Um ano do desastre da Vale: Organização e resposta do Ministério da saúde para o desastre em Brumadinho (MG)</a></p>
<p><a href="/images/pdf/2020/fevereiro/04/Boletim-epidemiologico-SVS-04fev20.pdf">BE Coronavírus - 03/02/2020</a></p>
<p><a href="/images/pdf/2020/janeiro/23/Boletim_epidemiologico_SVS_04.pdf" target="_blank">BE Vol. 51 Nº 4 - Novo coronavírus (2019-nCoV); e vigilância de violências e acidentes no Brasil: análise da cobertura da notificação compulsória de violência interpessoal/autoprovocada nos municípios brasileiros</a></p>
<p><a href="/images/pdf/2020/janeiro/22/Boletim-epidemiologico-SVS-03-v3.pdf" target="_blank">BE Vol. 51 Nº 3 -&nbsp;Identificação de um caso de febre&nbsp;hemorrágica brasileira no estado de&nbsp;São Paulo, janeiro de 2020</a></p>
<p><a href="/images/pdf/2020/janeiro/20/Boletim-epidemiologico-SVS-02-1-.pdf" target="_blank">BE Vol. 51 Nº 2 – Monitoramento de dengue, chikungunya e Zika, até a SE 52 de 2019; Campanha de vacinação contra o sarampo</a></p>
<p><a href="/images/pdf/2020/janeiro/03/Boletim-epidemiologico-SVS-40.pdf">BE Vol. 51 Nº 1 - </a><a href="/images/pdf/2020/janeiro/15/Boletim-epidemiologico-SVS-01.pdf" target="_blank">Situação epidemiológica, fluxograma classificação de risco e manejo clínico da febre amarela; Evento de monitoramento internacional: China – Pneumonia de etiologia desconhecida; Violência contra mulheres e meninas do campo, da floresta e das águas</a></p>
</div>
<div class="su-tabs-pane su-clearfix" style="display: none;">
<p><span style="color: #000000;">
</span></p><ul class="jp-accordion ui-accordion ui-widget ui-helper-reset ui-accordion-icons" id="accordion5e9d0b39d6fa4" style="width:100%">
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">2020</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="https://portalarquivos.saude.gov.br/images/pdf/2020/April/19/BE12-Boletim-do-COE.pdf">BE 12 - Boletim COE Coronavírus</a></p>
<p><a href="/images/pdf/2020/April/18/2020-04-17---BE11---Boletim-do-COE-21h.pdf" target="_blank">BE 11 - Boletim COE Coronavírus</a></p>
<p><a href="https://portalarquivos.saude.gov.br/images/pdf/2020/April/17/2020-04-16---BE10---Boletim-do-COE-21h.pdf" target="_blank" rel="noopener">BE 10 - Boletim COE Coronavirus</a></p>
<p><a href="/images/pdf/2020/April/13/Boletim-epidemiologico-SVS-15.pdf" target="_blank">BE Vol. 51 Nº 15 - Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes Aegypti (dengue, chikungunya e zika), Semanas Epidemiológicas 1 a 14 de 2020, e da febre amarela (2019/2020)</a></p>
<p><a href="https://portalarquivos.saude.gov.br/images/pdf/2020/April/12/2020-04-11-BE9-Boletim-do-COE.pdf" target="_blank" rel="noopener">BE9 - Boletim Especial do COE Coronavírus Avaliação de Risco</a></p>
<p><a href="/images/pdf/2020/April/09/be-covid-08-final-2.pdf" target="_blank">BE8 - Boletim Especial do COE Coronavírus Avaliação de Risco</a></p>
<p><a href="/images/pdf/2020/April/09/be-covid-08-final.pdf" target="_blank"></a><a href="/images/pdf/2020/April/03/Boletim-epidemiologico-SVS-14.pdf">BE Vol. 51 Nº 14 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 13, 2020; febre amarela; diferenças regionais no consumo de alimentos entre adolescentes e adultos e leishmaniose visceral: Experiência do IEC</a></p>
<p><a href="/images/pdf/2020/marco/27/Boletim-epidemiologico-SVS-13.pdf" target="_blank">BE Vol.51 Nº 13 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 12, 2020; febre amarela; análise da anomalias congênitas no Brasil; abastecimento de água para consumo humano em 2019 e medicamentos antileishmania no âmbito do SUS</a></p>
<p><a href="/images/pdf/2020/marco/24/Boletim-tuberculose-2020-marcas--1-.pdf" target="_blank">BE Número Especial – Tuberculose 2020</a></p>
<p><a href="/images/pdf/2020/marco/30/Boletim-epidemiologico-SVS-12.pdf">BE Vol. 51 Nº 12 - Monitoramento dos casos de arboviroses (dengue, chikungunya e zika), SE 1 a 11, 2020; febre amarela 2019/2020; sarampo: SE 1 a 10, 2020; ações de enfrentamento da tuberculose em imigrantes e Síndrome congênita associada à infecção pelo vírus Zika: SE 10, 2020</a></p>
<p><a href="/images/pdf/2020/marco/30/Boletim-epidemiologico-SVS11.pdf" target="_blank">BE Vol. 51 Nº 11 - Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e zika), SE 1 a 10, 2020; Vigilância da infecção latente pelo Mycobacterium tuberculosis; Doença de Chagas - Protocolos clínicos de seguimento pós-tratamento em grupos especiais</a><a href="/images/pdf/2020/marco/21/Boletim_epidemiologico_SVS_11.pdf" target="_blank"><br></a></p>
<p><a href="/images/pdf/2020/marco/06/Boletim-epidemiologico-SVS-10.pdf" target="_blank">BE Vol. 51 Nº10 Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika), Semanas Epidemiológicas 1 a 7, 2020.</a></p>
<p><a href="/images/pdf/2020/marco/05/Boletim-epidemiologico-SVS-09--.pdf" target="_blank">BE Vol. 51 Nº 9 - Vigilância epidemiológica do sarampo no Brasil 2020: Semana Epidemiológica 01; situação epidemiológica da febre amarela 2019/2020; importância dos códigos garbage para a magnitude da mortalidade por causas externas no Brasil; tracoma e acidentes ofídicos no Brasil, 2018</a></p>
<p><a href="/images/pdf/2020/marco/04/2020-03-02-Boletim-Epidemiol--gico-04---COE-COVID-19.pdf">BE Coronavírus - 04/03/2020</a></p>
<p><a href="/images/pdf/2020/fevereiro/21/2020-02-21-Boletim-Epidemiologico03.pdf" target="_blank">BE Coronavírus - 21/02/2020</a></p>
<p><a href="/images/pdf/2020/fevereiro/21/Boletim-epidemiologico-SVS-08.pdf" target="_blank">BE Vol. 51 Nº 8 - Vacinação para o viajante e população em geral; orientações para prevenção de doenças transmissíveis e acidentes por animais peçonhentos; vigilância das IST; uso de álcool: alerta para o período do carnaval</a></p>
<p><a href="/images/pdf/2020/fevereiro/19/Boletim-epidemiologico-SVS-07.pdf" target="_blank">BE Vol. 51 Nº 7 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 05; síndrome congênita associada à infecção pelo vírus Zika até a SE 05 e mobilização nacional para enfrentamento da hanseníase</a></p>
<p><a href="/images/pdf/2020/April/08/Boletim-epidemiologico-SVS-06-v2.pdf" target="_blank" rel="noopener">BE Vol. 51 Nº 6 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 04; sarampo no Brasil 2019: janeiro a dezembro e situação da febre amarela: monitoramento 2019/2020</a></p>
<p><a href="https://portalarquivos2.saude.gov.br/images/pdf/2020/fevereiro/13/Boletim-epidemiologico-COEcorona-SVS-13fev20.pdf" target="_blank" rel="noopener">BE Coronavírus - 10/02/2020</a></p>
<p><a href="/images/pdf/2020/fevereiro/18/Boletim-epidemiologico-SVS-01-COE-inundacao.pdf">BE COE Inundação nº 01 - Gestão de Emergências em Saúde Pública por desastres: evento hidrometeorológico na região Sudeste do Brasil, 2020</a></p>
<p><a href="/images/pdf/2020/fevereiro/05/Boletim-epidemiologico-SVS-05.pdf">BE Vol. 51 Nº 5 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 03; Síndrome Congênita associada à infecção pelo vírus Zika: monitoramento do ano de 2019 e a qualidade da água para consumo humano no Rio de Janeiro.</a></p>
<p><a href="/images/pdf/2020/janeiro/31/Boletim-hanseniase-2020-web.pdf" target="_blank">BE Número Especial - Hanseníase</a></p>
<p><a href="/images/pdf/2020/janeiro/28/boletim-especial-27jan20-final.pdf" target="_blank">BE Número Especial - Um ano do desastre da Vale: Organização e resposta do Ministério da saúde para o desastre em Brumadinho (MG)</a></p>
<p><a href="/images/pdf/2020/fevereiro/04/Boletim-epidemiologico-SVS-04fev20.pdf">BE Coronavírus - 03/02/2020</a></p>
<p><a href="/images/pdf/2020/janeiro/28/Boletim-epidemiologico-SVS-final2.pdf" target="_blank"></a><a href="/images/pdf/2020/janeiro/23/Boletim_epidemiologico_SVS_04.pdf" target="_blank">BE Vol. 51 Nº 4 - Novo coronavírus (2019-nCoV); e vigilância de violências e acidentes no Brasil: análise da cobertura da notificação compulsória de violência interpessoal/autoprovocada nos municípios brasileiros</a></p>
<p><a href="/images/pdf/2020/janeiro/21/Boletim-epidemiologico-SVS-03.pdf" target="_blank">BE Vol. 51 Nº 3 -&nbsp;Identificação de um caso de febre&nbsp;hemorrágica brasileira no estado de&nbsp;São Paulo, janeiro de 2020</a></p>
<p><a href="/images/pdf/2020/janeiro/20/Boletim-epidemiologico-SVS-03.pdf" target="_blank"></a><a href="/images/pdf/2020/janeiro/20/Boletim-epidemiologico-SVS-02-1-.pdf" target="_blank">BE Vol. 51 Nº 2 – Monitoramento de dengue, chikungunya e Zika, até a SE 52 de 2019; Campanha de vacinação contra o sarampo</a></p>
<p><a href="/images/pdf/2020/janeiro/03/Boletim-epidemiologico-SVS-40.pdf">BE Vol. 51 Nº 1 - </a><a href="/images/pdf/2020/janeiro/15/Boletim-epidemiologico-SVS-01.pdf" target="_blank">Situação epidemiológica, fluxograma classificação de risco e manejo clínico da febre amarela; Evento de monitoramento internacional: China – Pneumonia de etiologia desconhecida; Violência contra mulheres e meninas do campo, da floresta e das águas</a></p>
<p><span style="color: #000000;"></span></p></div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">2019</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/fevereiro/27/boletim-hepatites-2019-c.pdf" target="_blank">Boletim Epidemiológico Hepatites Virais 2019 - Secretaria de Vigilância em Saúde | Ministério da Saúde Volume 50 | jul. 2019</a></p>
<p><a href="/images/pdf/2020/janeiro/16/Boletim-epidemiologico-SVS-40-v2.pdf">BE Vol. 50 Nº 40 - Panorama da vigilância de doenças crônicas não transmissíveis no Brasil e Fortalecimento das ações de prevenção, controle e eliminação da malária e ações de vigilância da leishmaniose visceral e doença de Chagas</a></p>
<p><a href="/images/PDF/2019/dezembro/27/Boletim-epidemiologico-SVS-39-FINAL.PDF" target="_blank">BE Vol. 50 Nº 39 – Sarampo (SE 39 a 50 de 2019); manejo de capivaras em áreas com casos de febre maculosa brasileira; Verão seguro; Situação Epidemiológica da Febre Amarela no monitoramento 2018/2019 e perspectivas para o período sazonal 2019/2020 no Brasil</a></p>
<p><a href="/images/pdf/2019/dezembro/23/Boletim-epidemiologico-SVS-38-2-interativo.pdf" target="_blank">BE Vol. 50 Nº 38 – Influenza (SE 1 a 49 de 2019); ampliação do diagnóstico da toxoplasmose congênita, monitoramento de dengue, chikungunya e Zika, até a SE 49; atualizações das indicações da vacina Febre Amarela e Campanha de Vacinação contra o HPV</a></p>
<p><a href="/images/pdf/2019/dezembro/16/Boletim-epidemiologico-SVS-37-interativo-final.pdf">BE Vol. 50 Nº 37 – Sarampo (SE 36 a 47) vigilância epidemiológica, apuração final da campanha, análise da mortalidade de pedestres no Brasil (2010 a 2017), monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (SE 1 a 47)</a></p>
<p><a href="/images/pdf/2019/dezembro/16/Boletim-epidemiologico-SVS-37-interativo-final.pdf"></a><a href="/images/pdf/2019/dezembro/05/be-sindrome-congenita-vfinal.pdf">Número Especial -&nbsp;Síndrome congênita associada à infecção pelo vírus Zika - Situação Epidemiológica, ações desenvolvidas e desafios de 2015 a 2019</a></p>
<p><a href="/images/pdf/2019/novembro/29/Boletim-epidemiologico-SVS-36-interativo.pdf" target="_blank">BE Vol. 50 Nº 36 - Panorama da doença de Chagas no Brasil e monitoramento das manchas de óleo no litoral brasileiro</a></p>
<p><a href="/images/pdf/2019/novembro/29/Boletim-Ist-Aids-2019-especial-web.pdf">Boletim Epidemiológico - Número Especial, Dez. 2019, HIV/Aids 2019</a></p>
<p><a href="/images/pdf/2019/novembro/20/Boletim-epidemiologico-SVS-35.pdf" target="_blank">BE Vol. 50 Nº 35 - Sarampo (SE 34 a 45), vigilância laboratorial do sarampo, vacinação contra o sarampo, orientações para a 2ª etapa da vacinação contra o sarampo, raiva, recomendações para uso dos imunobiológicos, malária (2017 a 2019) e monitoramento das manchas de óleo</a></p>
<p><a href="/images/pdf/2019/novembro/13/Boletim-epidemiologico-SVS-34-V1.pdf" target="_blank">BE Vol. 50 Nº 34 - Monitoramento das manchas de óleo no Nordeste; sarampo (Orientações para 2ª etapa da vacinação) e arboviroses (SE 1 a 44)</a></p>
<p><a href="/images/pdf/2019/novembro/07/Boletim-epidemiologico-SVS-33-7nov19.pdf" target="_blank">BE Vol. 50 Nº 33, Sarampo (SE 32 a 43); vigilância laboratorial do sarampo; informações sobre vacinação contra o sarampo; situação dos imunobiológicos; Programa Vida no Trânsito e monitoramento das manchas de óleo no Nordeste</a></p>
<p><a href="/images/pdf/2019/novembro/07/Boletim-epidemiologico-SVS-33b.pdf"></a><a href="/images/pdf/2019/outubro/31/Boletim-epidemiologico-SVS-32.pdf" target="_blank">BE Vol. 50 Nº 32, 2019: Vigilância em Saúde: monitoramento das manchas de óleo no litoral do Nordeste ; arboviroses (SE 1 a 41 de 2019) e consumo de bebidas alcoólicas entre escolares no Brasil: uma análise da Pesquisa Nacional de Saúde do Escolar, 2015</a></p>
<p><a href="/images/pdf/2019/outubro/30/Boletim-S--filis-2019-internet.pdf" target="_blank">BE Número Especial: Sífilis | 2019</a></p>
<p><a href="http://saude.gov.br/images/pdf/2019/outubro/24/Boletim-epidemiologico-SVS-31.pdf">BE Vol. 50 Nº 31, 2019: Sarampo (SE 30 a 42 de 2019);&nbsp; arboviroses (SE 1 a 40 de 2019) e óbitos por lesões de trânsito (2000 a 2017)</a></p>
<p><a href="/images/pdf/2019/novembro/07/Boletim-epidemiologico-SVS-30.pdf">BE Vol. 50 Nº 30, 2019: &nbsp;Sarampo (SE 29 a 40 de 2019); violências contra mulheres (2011-2018)&nbsp; e CIOCS - Copa do Mundo FIFA Sub-17</a></p>
<p><a href="/images/pdf/2019/outubro/10/Boletim-epidemiologico-SVS-29.pdf" target="_blank">BE Vol. 50 Nº 29, 2019: Cobertura nacional da vacina tríplice viral: primeira dose para crianças com até 1 ano, de 2015 a 2019</a><a href="/images/pdf/2019/outubro/04/BE-multitematico-n28.pdf">&nbsp;arboviroses urbanas transmitidas pelo Aedes Aegypti (SE 1 a 38 de 2019)</a></p>
<p><a href="/images/pdf/2019/outubro/04/BE-multitematico-n28.pdf" target="_blank">BE Vol. 50 Nº 28, 2019: sarampo (SE 28 a 39 de 2019); febre do Mayaro e febre do Oropouche; acidentes escorpiônicos; arboviroses urbanas transmitidas pelo Aedes Aegypti (SE 1 a 36 de 2019)</a></p>
<p><a href="/images/pdf/2019/outubro/01/Boletim-tuberculose-2019.pdf" target="_blank">BE Vol. 50 Nº 26, 2019: Panorama epidemiológico da coinfecção TB-HIV no Brasil 2019: Setembro 2019</a></p>
<p><a href="/images/pdf/2019/dezembro/05/Boletim-Epidemiologico-Especial-SVS-16-anos-web.pdf" target="_blank">Vigilância em Saúde no Brasil 2003 | 2019 - Da criação da Secretaria de Vigilância em Saúde aos dias atuais</a></p>
<p><a href="/images/pdf/2019/setembro/25/BE-sarampo-27-25set19.pdf">BE Vol. 50 Nº 27, 2019: &nbsp;Vigilância Epidemiológica do Sarampo no Brasil 2019: Semanas Epidemiológicas 27 a 38 de 2019</a></p>
<p><a href="/images/pdf/2019/setembro/19/BE-sarampo-25-18set19.pdf">BE Vol. 50 Nº 25, 2019: Vigilância Epidemiológica do Sarampo no Brasil 2019: Semanas Epidemiológicas 26 a 37 de 2019</a></p>
<p><a href="/images/pdf/2019/setembro/13/BE-suic--dio-24-final.pdf" target="_blank">BE Vol. 50 Nº 24, 2019: Perfil epidemiológico dos casos notificados de violência autoprovocada e óbitos</a></p>
<p><a href="/images/pdf/2019/setembro/13/BE-sarampo-23-final.pdf" target="_blank" rel="noopener">BE Vol. 50 Nº 23, 2019: Vigilância Epidemiológica do Sarampo&nbsp;no Brasil, 2019, Semanas Epidemiológicas 25 a 36 de 2019</a></p>
<p><a href="/images/pdf/2019/setembro/11/BE-arbovirose-22.pdf">BE Vol. 50&nbsp; Nº&nbsp; 22, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 34 de 2019</a></p>
<p><a href="/images/pdf/2019/setembro/06/BE-21-influenza-04set19.pdf" target="_blank">BE Vol. 50 Nº 21, 2019: Monitoramento dos casos de influenza no Brasil, Semanas Epidemiológicas 1 a 32 de 2019</a></p>
<p><a href="/images/pdf/2019/setembro/06/BE-sarampo-20-.pdf">BE Vol. 50 Nº 20, 2019: Vigilância Epidemiológica do sarampo no Brasil, Semanas Epidemiológicas 24 a 35 de 2019</a></p>
<p><a href="/images/pdf/2019/setembro/09/BE-sarampo-19.pdf">BE Vol. 50 Nº 19, 2019: Vigilância Epidemiológica do sarampo no Brasil, Semanas Epidemiológicas 23 a 34 de 2019</a></p>
<p><a href="/images/pdf/2019/setembro/09/BE-sarampo-18.pdf">BE Vol. 50&nbsp; Nº&nbsp; 18, 2019: Vigilância Epidemiológica do sarampo no Brasil, Semanas Epidemiológicas 21 a 33 de 2019</a></p>
<p><a href="/images/pdf/2019/setembro/09/BE-sarampo-17.pdf">BE Vol. 50&nbsp; Nº&nbsp; 17, 2019: Monitoramento dos casos de sarampo no Brasil, Semanas Epidemiológicas 20 a 31 de 2019. Lista de municípios com surto de sarampo até 13/08/2019.. Lista de municípios com surto de sarampo até 13/08/2019.</a></p>
<p><a href="/images/pdf/2019/julho/08/2019-007-Uso-de-servicos-de-saude-e-fatores-associados-a-procura-pela-UBS-entre-adolescentes-publicacao-04-07.pdf">BE Vol. 50&nbsp; Nº&nbsp; 16, 2019: Uso de serviços de saúde e fatores associados à procura pela Unidade Básica de Saúde entre adolescentes brasileiros: resultados da Pesquisa Nacional de Saúde do Escolar (PeNSE), 2015</a></p>
<p><a href="/images/pdf/2019/julho/17/2019-014-Publicacao-02-07.pdf">BE Vol. 50&nbsp; Nº&nbsp; 15, 2019: Suicídio: tentativas e óbitos por intoxicação exógena no Brasil, 2007 a 2016</a></p>
<p><a href="/images/pdf/2019/junho/13/2019-010-publicacao.pdf" target="_blank">BE Vol. 50&nbsp; Nº&nbsp; 14, 2019: População em situação de rua e violência - uma análise das notificações no Brasil de 2015 a 2017</a></p>
<p><a href="/images/pdf/2019/abril/30/informe-arboviroses-15.pdf"></a><a href="/images/pdf/2019/abril/30/2019-013-Monitoramento-dos-casos-de-arboviroses-urbanas-transmitidas-pelo-Aedes-publicacao.pdf">BE Vol. 50&nbsp; Nº&nbsp; 13, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 12 de 2019</a><a href="http://portalarquivos2.saude.gov.br/images/pdf/2019/abril/01/2018-066.pdf" target="_blank" rel="noopener noreferrer"></a></p>
<p><a href="http://portalarquivos2.saude.gov.br/images/pdf/2019/abril/01/2018-066.pdf" target="_blank" rel="noopener noreferrer">BE Vol. 50&nbsp; Nº&nbsp; 12, 2019: Descrição do Processo de Monitoramento dos Testes de Diagnóstico Rápido de Malária, Brasil, 2014 a 2016</a></p>
<p><a href="http://portalarquivos2.saude.gov.br/images/pdf/2019/marco/29/2018-059.pdf" target="_blank" rel="noopener noreferrer">BE Vol. 50&nbsp; Nº&nbsp; 11, 2019: Acidentes de Trabalho por animais peçonhentos entre trabalhadores do campo, floresta e águas, Brasil 2007 a 2017</a></p>
<p><a href="/images/pdf/2019/marco/25/2019-013-Monitoramento-dos-casos-de-arboviroses-publicacao-25-03-2019.pdf">BE Vol. 50&nbsp; Nº&nbsp; 10, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 11 de 2019</a></p>
<p><a href="/images/pdf/2019/marco/22/2019-009.pdf">BE Vol. 50&nbsp; Nº&nbsp; 09, 2019: Brasil Livre da Tuberculose: evolução dos cenários epidemiológicos e operacionais da doença</a></p>
<p><a href="/images/pdf/2019/marco/22/2019-001.pdf">BE Vol. 50&nbsp; Nº&nbsp; 08, 2019: Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 52 de 2018</a></p>
<p><a href="/images/pdf/2019/marco/21/2019-006.pdf">BE Vol. 50&nbsp; Nº&nbsp; 07, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 7 de 2019</a></p>
<p><a href="/images/pdf/2019/marco/21/2019-005.pdf">BE Vol. 50&nbsp; Nº&nbsp; 06, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 6 de 2019</a></p>
<p><a href="/images/pdf/2019/fevereiro/26/2019-004-Dengue-SE-5-publica----o-18-02-2019.pdf">Vol. 50 Nº 05, 2019: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 5 de 2019</a></p>
<p><a href="/images/pdf/2019/janeiro/28/2019-002.pdf" target="_blank" rel="noopener noreferrer">Vol. 50&nbsp; Nº 04, 2019: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica SE 52&nbsp; 2018 </a></p>
<p><a href="/images/pdf/2019/fevereiro/01/2018-038.pdf">Vol. 50&nbsp; Nº 03, 2019: Meningites bacterianas não especificadas, Brasil, 2007 a 2016</a></p>
<p><a href="/images/pdf/2019/janeiro/23/2018-025.pdf">Vol. 50&nbsp; Nº 02, 2019: Doença de Chagas Aguda e distribuição espacial dos triatomíneos de importância epidemiológica, Brasil 2012 a 2016</a></p>
<p><a href="/images/pdf/2019/janeiro/28/2018-068.pdf" target="_blank" rel="noopener noreferrer">Vol. 50&nbsp; Nº 01, 2019: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica SE 50 2018</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">2018</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2019/janeiro/02/2018-067.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 Nº 59, 2018: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 49 de 2018</a></p>
<p><a href="/images/pdf/2018/dezembro/26/2018-027.pdf">Vol. 49 Nº 58, 2018: Intoxicações exógenas relacionadas ao trabalho no Brasil de 2007 a 2016</a></p>
<p><a href="/images/pdf/2018/dezembro/18/2018-064.pdf">Vol. 49 Nº 57, 2018: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 48 de 2018</a></p>
<p><a href="/images/pdf/2019/janeiro/14/BE-Vol. 49 – Nº 56, 2018- dengue chikungunya zika_aed_10jan19.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 Nº 56, 2018: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 47 de 2018</a></p>
<p><a href="/images/pdf/2019/janeiro/14/BE-Vol. 49 - Nº 55, 2018- dengue chikungunya zika_aed_10jan19.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 Nº 55, 2018: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 46 de 2018</a></p>
<p><a href="/images/pdf/2018/dezembro/14/2018-061.pdf">Vol. 49&nbsp; Nº 54, 2018: Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 45 de 2018</a></p>
<p><a href="/images/pdf/2019/janeiro/14/BE-Vol. 49 - Nº 53, 2018 - dengue chikungunya zika_aed_10jan19.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 53, 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 45&nbsp;de 2018</a></p>
<p><a href="/images/pdf/2018/novembro/21/2018-028.pdf">Vol. 49 - Nº 52, 2018 - Perfil epidemiológico dos casos notificados dos erros de imunização, Brasil, 2016</a></p>
<p><a href="/images/pdf/2018/novembro/23/Publicacao-2018-57-SE44.pdf">Vol. 49 - Nº 51, 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 44&nbsp;de 2018</a></p>
<p><a href="/images/pdf/2018/dezembro/18/2018-031.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 50, 2018 - Casos notificados de intoxicações exógenas relacionados ao glifosato no Brasil, no período de 2007 a 2016</a></p>
<p><a href="/images/pdf/2018/novembro/19/2018-032.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 49, 2018 - Situação Epidemiológica e estratégias de prevenção, controle e eliminação das Doenças Tropicais Negligenciadas no Brasil, 1995 a 2016</a></p>
<p><a href="/images/pdf/2018/novembro/13/2018-056.pdf">Vol. 49 - Nº 48, 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 43&nbsp;de 2018</a></p>
<p><a href="/images/pdf/2018/novembro/12/2018-034.pdf">Vol. 49 - Nº&nbsp; 47, 2018&nbsp;-&nbsp;Situação epidemiológica da infecção pelo vírus Zika no Brasil, de 2015 a 2017</a></p>
<p><a href="/images/pdf/2018/novembro/07/2018-053-Monitoramento-integrado-Semana-Epidemiologica-40-publicacao.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 46 - 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 40 de 2018</a></p>
<p><a href="/images/pdf/2018/novembro/07/2018-052-Monitoramento-integrado-Semana-Epidemiologica-35-publicacao.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 45 - 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 35 de 2018</a></p>
<p><a href="/images/pdf/2018/novembro/05/2018-051-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-doen--a-aguda-pelo-v--rus-Zika-at---a-Semana-Epidemiol--gica-42-de-2018-publica--ao.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 44 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 42 de 2018 </a></p>
<p><a href="/images/pdf/2018/outubro/31/BE-2018-050-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-doen--a-aguda-pelo-v--rus-Zika-at---a-Semana-Epidemiol--gica-41-de-2018-ajustado.pdf" target="_blank" rel="noopener noreferrer">Vol. 49&nbsp; Nº&nbsp; 43, 2018: Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 41 de 2018</a></p>
<p><a href="/images/pdf/2018/outubro/26/2018-049.pdf">Vol. 49&nbsp; Nº&nbsp; 42, 2018: Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 38&nbsp;de 2018</a></p>
<p><a href="/images/pdf/2018/outubro/25/2018-033-Leptospirose-situa----o-epidemiol--gica-do-Brasil-no-per--odo-de-2007-a-2016-publica--ao.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº&nbsp; 41- 2018 - Leptospirose: Situação epidemiológica do Brasil no período de 2007 a 2016</a></p>
<p><a href="/images/pdf/2018/outubro/31/Vol. 49 Nº 40.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 40 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 36 de 2018</a></p>
<p><a href="/images/pdf/2018/outubro/31/Vol. 49 Nº 39.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 39 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica - 2018</a></p>
<p><a href="/images/pdf/2018/outubro/05/Vol. 49 Nº 38.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº&nbsp;38 - 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 30</a></p>
<p><a href="/images/pdf/2018/outubro/05/Vol. 49 Nº 37.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº&nbsp;37 - 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 25</a></p>
<p><a href="/images/pdf/2018/outubro/05/Vol.49 Nº 36.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº&nbsp;36 - 2018 - Experiências de Programas de Controle da Tuberculose: Porque juntos iremos detectar, tratar e acabar com a tuberculose como problema de saúde pública no Brasil</a></p>
<p><a href="/images/pdf/2018/outubro/31/Vol. 49 Nº 35.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 35 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 32 -2018</a></p>
<p><a href="/images/pdf/2018/agosto/21/Publicacao-BE-2018-SE-30.pdf">Vol. 49 - Nº 34 - 2018 - &nbsp;Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 30 - 2018</a></p>
<p><a href="http://portalarquivos2.saude.gov.br/images/pdf/2018/agosto/07/2018_040.pdf">Vol. 49 - Nº 33 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 28 - 2018</a></p>
<p><a href="/images/pdf/2018/julho/12/Monitoramento-dos-casos-ate-a-Semana-Epidemiologica-25-de-2018.pdf">Vol. 49 - Nº 32 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 25 de 2018</a></p>
<p><a href="/images/pdf/2018/julho/06/Monitoramento-dos-casos-de-dengue-23-018.pdf">Vol. 49 - Nº 31 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 23 de 2018</a></p>
<p><a href="/images/pdf/2018/julho/06/Monitoramento-dos-casos-de-dengue-21-de-2018.pdf">Vol. 49 - Nº 30 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 21 de 2018</a></p>
<p><a href="http://portalarquivos2.saude.gov.br/images/pdf/2018/junho/29/Monitoramento-integrado-de-alteracoes-no-crescimento-e-desenvolvimento-relacionadas-a-infeccao-pelo-virus-Zika.pdf">Vol. 49 - Nº 29 - 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 20 de 2018</a></p>
<p><a href="/images/pdf/2018/junho/25/2018-023.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 28- 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 19 2018</a></p>
<p><a href="/images/pdf/2018/junho/25/2018-024.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 27- 2018 - Análise epidemiológica da violência sexual contra crianças e adolescentes no Brasil, 2011 a 2017 </a></p>
<p><a href="/images/pdf/2018/junho/20/2017-033-publicacao.pdf">Vol. 49 - Nº 26 - 2018 - Acidentes de transporte relacionados ao trabalho no Brasil, 2007-2016</a></p>
<p><a href="/images/pdf/2018/junho/11/2017-041-Tetano-publicacao.pdf">Vol. 49 - Nº 25 - 2018 - Situação epidemiológica do tétano acidental, Brasil, 2007 a 2016 </a></p>
<p><a href="/images/pdf/2018/junho/11/2018-022.pdf">Vol. 49 - Nº 24 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 17 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/28/2018-023.pdf">Vol. 49 - Nº 23 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 16 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/28/2018-022.pdf">Vol. 49 - Nº 22 - 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 15 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/28/BE-2018-021.pdf">Vol. 49 - Nº 21 - 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 09 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/07/2018-018.pdf">Vol. 49 - Nº 20 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 15 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/04/2018-016.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 19 - 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 05 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/02/2018-017-Publicacao.pdf">Vol. 49 - Nº 18 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 14 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/02/2018-015-Publicacao.pdf">Vol. 49 - Nº 17 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 13 de 2018</a></p>
<p><a href="/images/pdf/2018/abril/27/BE-2018-14-SE-12.pdf">Vol. 49 - Nº 16 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 12 de 2018</a></p>
<p><a href="/images/pdf/2018/abril/19/BE-2018-13-SE-11-Publicacao.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 15 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 11, de 2018</a></p>
<p><a href="/images/pdf/2018/abril/06/2018-012.pdf">Vol. 49 - Nº 14 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 10, de 2018</a></p>
<p><a href="/images/pdf/2018/abril/06/2018-011.pdf">Vol. 49 - Nº 13 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 09, de 2018</a></p>
<p><a href="http://portalarquivos2.saude.gov.br/images/pdf/2018/marco/29/2018-010.pdf">Vol. 49 -&nbsp; Nº 12- 2018 -&nbsp;Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 8 de 2018</a></p>
<p><a href="/images/pdf/2018/marco/26/2018-009.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 -&nbsp; Nº&nbsp; 11- 2018 - Implantação do Plano Nacional pelo Fim da Tuberculose como Problema de Saúde Pública no Brasil: primeiros passos rumo ao alcance das metas.</a></p>
<p><a href="/images/pdf/2018/marco/22/2017-032-Publicacao.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 10 - 2018 - Desastres naturais e saúde: análise do cenário de eventos hidrológicos no Brasil e seus potenciais impactos sobre o Sistema Único de Saúde</a></p>
<p><a href="/images/pdf/2018/marco/06/2018-008-Publicacao.pdf">Vol. 49 - Nº 9 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 07 de 2018</a></p>
<p><a href="/images/pdf/2018/marco/06/2018-007-Publicacao.pdf">Vol. 49 - Nº 8 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 06 de 2018 </a></p>
<p><a href="/images/pdf/2018/fevereiro/20/2018-007.pdf">Vol. 49 - Nº 7 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 5 de 2018</a></p>
<p><a href="/images/pdf/2018/fevereiro/20/2018-003-Final.pdf">Vol. 49 - Nº 6 - 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 52 de 2017</a></p>
<p><a href="/images/pdf/2018/fevereiro/19/2018-005.pdf">Vol. 49 - Nº 5 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 3 de 2018</a></p>
<p><a href="/images/pdf/2018/janeiro/31/2018-004-Hanseniase-publicacao.pdf">Vol. 49 - Nº 4 - 2018 - Caracterização da situação epidemiológica da hanseníase e diferenças por sexo, Brasil, 2012-2016</a></p>
<p><a href="/images/pdf/2018/janeiro/30/2018-002.pdf"> Vol. 49 -&nbsp; Nº&nbsp; 3- 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 48 de 2017</a></p>
<p><a href="/images/pdf/2018/janeiro/23/Boletim-2018-001-Dengue.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 2 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 52 de 2017</a></p>
<p><a href="/images/pdf/2018/janeiro/23/Boletim-2017-051-Dengue.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 1 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 51 de 2017</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">2017</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2018/janeiro/10/2017-046-Publicacao.pdf">Vol. 48 - Nº 45 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 50, 2017</a></p>
<p><a href="/images/pdf/2017/dezembro/26/2017-044.pdf">Vol. 48 - Nº 44 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 49 de 2017</a></p>
<p><a href="/images/pdf/2017/dezembro/26/BE-2017-043-Boletim.pdf">Vol. 48 - Nº 43 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 48 de 2017</a></p>
<p><a href="/images/pdf/2017/dezembro/21/BE-2017-042-Boletim.pdf">Vol. 48 - Nº 42 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 47 de 2017</a></p>
<p><a href="/images/pdf/2017/dezembro/12/Boletim-2017-043-final.pdf">Vol. 48&nbsp;- Nº 41 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 44</a></p>
<p><a href="/images/pdf/2017/dezembro/05/2017-030.pdf">Vol. 48 - Nº 40 - 2017 - Coinfecção TB-HIV no Brasil: panorama epidemiológico e atividades colaborativas</a></p>
<p><a href="/images/pdf/2017/dezembro/08/2017-040.pdf">Vol. 48 - Nº 39 -2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 46 </a></p>
<p><a href="/images/PDF/2017/novembro/28/2017_039.pdf">Vol. 48&nbsp;- Nº 38 -2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 45, 2017</a></p>
<p><a href="/images/pdf/2017/novembro/16/2017-036.pdf">Vol.48 - Nº 37 -2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 38/2017</a></p>
<p><a href="http://portalarquivos.saude.gov.br/images/pdf/2017/novembro/13/BE-2017-038-Boletim-Sifilis-11-2017-publicacao-.pdf">Vol. 48 - Nº 36 - 2017 - Sífilis 2017</a><br> <a href="/images/pdf/2017/novembro/13/2017-037.pdf"><br> Vol. 48 - Nº 35 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 43 de 2017</a><br> <a href="/images/pdf/2017/novembro/07/2017-035-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica-41.pdf"><br> Vol. 48 - Nº 34 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 41 de 2017</a></p>
<p><a href="/images/pdf/2017/outubro/23/2017-034-Monitoramento-integrado-de-alteracoes-no-crescimento-e-desenvolvimento-relacionadas-a-infeccao-pelo-virus-Zika-e-outras-etiologias-infecciosas--ate-a-Semana-Epidemiologica-34.pdf">Vol.48&nbsp;&nbsp; Nº 33 -2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 34/2017</a></p>
<p><a href="/images/pdf/2017/outubro/18/boletim-n32.pdf">Vol. 48 - Nº 32 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 39, 2017</a></p>
<p><a href="/images/pdf/2017/outubro/18/BE-Vol-48-n-31-2017-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-ate-a-Semana-Epidemiologica-37.pdf">Vol. 48 - Nº 31 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 37, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/setembro/21/2017-025-Perfil-epidemiologico-das-tentativas-e-obitos-por-suicidio-no-Brasil-e-a-rede-de-atencao-a-saude.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 30 - 2017 - Perfil epidemiológico das tentativas e óbitos por suicídio no Brasil e a rede de atenção à saúde</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/setembro/15/2017-028-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica-35.pdf">Vol. 48 - Nº 29 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 35, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/setembro/06/2017_027.pdf">Vol. 48 - Nº 28 - 2017 - Emergência epidemiológica de febre amarela no Brasil, no período de dezembro de 2016 a julho de 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/agosto/29/2017-026-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica-33-de-2017.pdf">Vol. 48 - Nº 27 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 33, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/agosto/23/2017_024-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 26 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 31, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/agosto/21/2017_023-Situacao-Epidemiologica-da-Influenza-no-Brasil-ate-a-Semana-Epidemiologica-32-de-2017.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 25 - 2017 - Situação Epidemiológica da Influenza no Brasil, até a Semana Epidemiológica 32 de 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/agosto/16/BE-2017_024-Monitoramento-integrado-de-alteracoes-no-crescimento-e-desenvolvimento-relacionadas-a-infeccao-pelo-virus-Zika.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 24 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 28/2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/agosto/04/BE_Integrado_2017_023_Final.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 23 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 26/2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/agosto/04/BE_Integrado_2017_022_Final.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 22 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 24/2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/agosto/04/BE_Integrado_2017_021_Final.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 21 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 22/2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/julho/25/Boletim-2017_020-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-Zika.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 20 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 25, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/julho/04/2017-017-Monitoramento-integrado-de-alteracoes-no-crescimento-e-desenvolvimento-relacionadas-a-infeccao-pelo-virus-Zika.pdf">Vol. 48 - Nº 19 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 20/2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/junho/23/2017-005-Vigilancia-em-Saude-do-Trabalhador.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 18 - 2017 - Vigilância em Saúde do Trabalhador: um breve panorama</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/junho/01/BE-2017-015-Monitoramento-integrado-de-alteracoes-no-crescimento-e-desenvolvimento.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 17 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 18/2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/maio/25/Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 16 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 19, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/maio/15/BE-2017-014-Monitoramento-integrado-de-alteracoes-no-crescimento-e-desenvolvimento-relacionadas-a-infeccao-pelo-virus-Zika-e-outras-etiologias-infecciosas--ate-a-Semana-Epidemiologica-16.pdf">Vol. 48 - Nº 15 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 16/2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/maio/05/Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica.pdf">Vol. 48 - Nº 14 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 15, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/abril/27/Monitoramento-integrado-de-alteracoes-no-crescimento-e-desenvolvimento-relacionadas-a-infeccao-pelo-virus-Zika.pdf">Vol. 48 - Nº 13 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 14/2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/abril/10/2017_011%20Monitoramento%20integrado%20de%20alteracoes%20no%20crescimento%20e%20desenvolvimento%20relacionadas%20a%20infeccao%20pelo%20virus%20Zika%20e%20outras%20etiologias%20infecciosas%20ate%20a%20Semana%20Epidemiologica%2012_rev.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 12 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 12/2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/abril/06/2017-010-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiol--gica-12--2017.pdf">Vol. 48 - Nº 11 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 12, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/abril/03/Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-10-2017.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 10 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 10, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/marco/31/Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-10-%202017.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 9 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 10, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/marco/23/2017-V-48-N-8-Indicadores-priorit--rios-para-o-monitoramento-do-Plano-Nacional-pelo-Fim-da-Tuberculose-como-Problema-de-Sa--de-P--blica-no-Brasil.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 8 - 2017 - Indicadores prioritários para o monitoramento do Plano Nacional pelo Fim da Tuberculose como Problema de Saúde Pública no Brasil</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/marco/28/2017_006-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica-7.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 7 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 7, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/fevereiro/27/2017_003.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 6 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, da Semana Epidemiológica 45/2015 até a Semana Epidemiológica 02/2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/fevereiro/17/Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-febre-pelo-v--rus-Semana-Epidemiologica-4-2017.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 5 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 4, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/fevereiro/17/Indicadores-de-Vigilancia-em-Saude-descritos-segundo-ra--a-cor.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 4 - 2017 - Indicadores de Vigilância em Saúde segundo a variável raça/cor, Brasil</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/abril/06/2017-002-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-febre-pelo-v--rus-Zika-ate-a-Semana-Epidemiologica-52--2016.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 3 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 52, 2016</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/janeiro/12/2017_001%20-%20Dengue%20SE51_publicacao.pdf">Vol. 48 - Nº 2 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 51, 2016</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/janeiro/05/2016_034-Aids_publicacao.pdf">Vol. 48 - Nº 1 - 2017 - Boletim Epidemiológico HIV/Aids, 2016</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">2016</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2016/dezembro/20/2016-033---Dengue-SE49-publicacao.pdf" target="_blank" rel="noopener noreferrer">Nº 38 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 49, 2016</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2016/novembro/15/2016_031-Mulheres_publicacao.pdf">Nº 37 - 2016 - Zika Vírus: perfil epidemiológico em mulheres 47, 2016</a></p>
<p><a href="http://portalsaude.saude.gov.br/pdf/2017/julho/07/2015_042-Situacao-epidemiologica-da-vigilanciade-eventos-adversos-pos-vacinacao.pdf" target="_blank" rel="noopener noreferrer">Nº 36 - 2016 - Situação epidemiológica da vigilância de eventos adversos pós vacinação nas Secretarias Estaduais de Saúde, Brasil, 2013</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2016/outubro/31/2016_030_Sifilis-publicao2.pdf" target="_blank" rel="noopener noreferrer">Nº 35 - 2016 - Sífilis V, 2016</a>
 <a href="http://portalsaude.saude.gov.br/images/pdf/2016/outubro/18/2016-029-Dengue-publicacao-n-34.pdf" target="_blank" rel="noopener noreferrer">Nº 34 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 37, 2016</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2016/setembro/16/2016-028---Dengue-SE32.pdf" target="_blank" rel="noopener noreferrer">Nº 33 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 32, 2016</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2016/setembro/15/2016-025---Coqueluche-publica----o.pdf" target="_blank" rel="noopener noreferrer">Nº 32 - 2016 - Situação epidemiológica da coqueluche, Brasil, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2016/agosto/10/2016-026--2-..pdf" target="_blank" rel="noopener noreferrer">Nº 31 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 27, 2016</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2016/julho/29/2016-010.pdf" target="_blank" rel="noopener noreferrer">Nº 30 - Perfil dos atendimentos antirrábicos humanos, Brasil, 2009-2013</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2016/julho/29/2016-015---DM.pdf" target="_blank" rel="noopener noreferrer">Nº 29 - Situação epidemiológica da doença meningocócica, Brasil, 2007-2013</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2016/julho/15/2016-boletim-epi-n28-dengue-chik-zika-se23.pdf" target="_blank" rel="noopener noreferrer">Nº 28 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 23, 2016</a></p>
<p>&nbsp;<a href="http://portalsaude.saude.gov.br/images/pdf/2016/junho/30/2016-021.pdf" target="_blank" rel="noopener noreferrer">Nº 27 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 21, 2016</a></p>
<p>&nbsp;<a href="http://portalsaude.saude.gov.br/images/pdf/2016/junho/30/2016-020--1-.pdf" target="_blank" rel="noopener noreferrer">Nº 26 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 20, 2016</a></p>
<p>&nbsp;<a href="http://portalsaude.saude.gov.br/images/pdf/2016/junho/17/2016-019.pdf" target="_blank" rel="noopener noreferrer">Nº 25 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 19, 2016</a></p>
<p>&nbsp;<a href="http://portalsaude.saude.gov.br/images/pdf/2016/junho/15/2016-018---Dengue-SE18-substitui----o.pdf" target="_blank" rel="noopener noreferrer">Nº 24 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 18, 2016</a></p>
<p>&nbsp;<a href="http://portalsaude.saude.gov.br/images/pdf/2016/junho/14/boletim-epid-2016-n23-dcz-se17.pdf" target="_blank" rel="noopener noreferrer">Nº 23 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 17, 2016</a></p>
<p>&nbsp;<a href="http://portalsaude.saude.gov.br/images/pdf/2016/maio/12/2015-036---DCJ-publica----o.pdf" target="_blank" rel="noopener noreferrer">Nº 22 -&nbsp;</a><a href="http://portalsaude.saude.gov.br/images/pdf/2016/maio/12/2015-036---DCJ-publica----o.pdf" target="_blank" rel="noopener noreferrer">Diagnóstico Situacional dos Casos da Doença de Creutzfeldt-Jakob notificados e confirmados no Sinan-NET por UF, Brasil - 2005 a 2013</a></p>
<p>&nbsp;<a href="http://portalsaude.saude.gov.br/images/pdf/2016/maio/12/2015-038---Campanha-publica----o.pdf" target="_blank" rel="noopener noreferrer">Nº 21 -&nbsp;</a><a href="http://portalsaude.saude.gov.br/images/pdf/2016/maio/12/2015-038---Campanha-publica----o.pdf" target="_blank" rel="noopener noreferrer">Hanseníase, Verminoses e Tracoma tem cura: a experiência de uma campanha integrada</a></p>
<p>&nbsp;<a href="http://portalsaude.saude.gov.br/images/pdf/2016/maio/17/2016-016---Dengue-SE16-publica----o.pdf" target="_blank" rel="noopener noreferrer">Nº 20 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 16, 2016</a></p>
<p>&nbsp;<a href="http://portalsaude.saude.gov.br/images/pdf/2016/maio/06/2015-026-doencas-respiratorias-cronicas.pdf" target="_blank" rel="noopener noreferrer">Nº 19 - Perfil da morbimortalidade por doenças respiratórias crônicas no Brasil, 2003 a 2013</a></p>
<p>&nbsp;<a href="http://portalsaude.saude.gov.br/images/pdf/2016/abril/27/2016-014---Dengue-SE13-substitui----o.pdf" target="_blank" rel="noopener noreferrer">Nº 18 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 13, 2016</a></p>
<p>&nbsp;<a href="http://portalsaude.saude.gov.br/images/pdf/2016/abril/07/2015-041---Ebola-publica----o.pdf" target="_blank" rel="noopener noreferrer">Nº 17 - Investigação de caso suspeito de doença pelo vírus Ebola no município de Cascavel/Paraná, outubro de 2014</a></p>
<p><strong>&nbsp;</strong><a href="http://portalsaude.saude.gov.br/images/pdf/2016/abril/11/2015-013---Dengue-SE9.pdf" target="_blank" rel="noopener noreferrer">Nº 16 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 9, 2016</a></p>
<p>&nbsp;<a href="http://portalsaude.saude.gov.br/images/pdf/2016/abril/05/2016-012---Relatorio-reuniao-especialistas-Aedes-publica----o.pdf" target="_blank" rel="noopener noreferrer">Nº 15 - Relatório da Reunião Internacional para Implementação de Alternativas para o Controle do Aedes aegypti no Brasil</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2016/marco/29/2016-011-boletim-epi-dcz-se8.pdf" target="_blank" rel="noopener noreferrer">&nbsp;Nº 14 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 8, 2016</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2016/marco/24/2016-009-Tuberculose-001.pdf" target="_blank" rel="noopener noreferrer">&nbsp;Nº 13 - Perspectivas brasileiras para o fim da tuberculose como problema de saúde pública</a></p>
<p>&nbsp;<a href="http://portalsaude.saude.gov.br/images/pdf/2016/marco/23/2015-052---Agrot--xicos-publica----o.pdf" target="_blank" rel="noopener noreferrer">Nº 12 - Monitoramento de agrotóxicos em água para consumo humano no Brasil, 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2016/marco/23/2016-008---DengueSE8-publica----o.pdf" target="_blank" rel="noopener noreferrer">&nbsp;Nº 11 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 7, 2016</a></p>
<p>&nbsp;<a href="http://portalsaude.saude.gov.br/images/pdf/2016/marco/23/2016-007---Dengue-SE-6-publica----o.pdf" target="_blank" rel="noopener noreferrer">Nº 10 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 6, 2016</a></p>
<p><strong>&nbsp;</strong><a href="http://portalsaude.saude.gov.br/images/pdf/2016/marco/11/2015-039---Filariose-vers--ofinal.pdf" target="_blank" rel="noopener noreferrer">Nº 9 - Situação epidemiológica da filariose linfática no Brasil</a></p>
<p><strong>&nbsp;</strong><a href="http://portalsaude.saude.gov.br/images/pdf/2016/fevereiro/29/2016-006---Dengue-SE5-publica----o.pdf" target="_blank" rel="noopener noreferrer">Nº 8 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 5, 2016</a></p>
<p><strong>&nbsp;</strong><a href="http://portalsaude.saude.gov.br/images/pdf/2016/fevereiro/29/2016-005---Dengue-SE4-publica----o.pdf" target="_blank" rel="noopener noreferrer">Nº 7 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 4, 2016</a></p>
<p>&nbsp;<a href="http://portalsaude.saude.gov.br/images/pdf/2016/fevereiro/04/2016-004---Dengue-SE3.pdf" target="_blank" rel="noopener noreferrer">Nº 6 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 3, 2016</a></p>
<p>&nbsp;<a href="http://portalsaude.saude.gov.br/images/pdf/2016/fevereiro/04/2016-003-Dengue-SE2.pdf" target="_blank" rel="noopener noreferrer">Nº 5 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 2, 2016</a></p>
<p><strong>&nbsp;</strong><a href="http://portalsaude.saude.gov.br/images/pdf/2016/fevereiro/02/2015-011---Asbestose.pdf" target="_blank" rel="noopener noreferrer">Nº 4 - Asbestose na saúde do trabalhador notificados no SINAN, Brasil, 2007-2013</a></p>
<p><strong>&nbsp;</strong><a href="http://portalsaude.saude.gov.br/images/pdf/2016/janeiro/15/svs2016-be003-dengue-se52.pdf" target="_blank" rel="noopener noreferrer">Nº 3 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 52, 2015</a></p>
<p><strong>&nbsp;</strong><a href="http://portalsaude.saude.gov.br/images/pdf/2016/janeiro/15/svs2016-be002-dengue-se51.pdf" target="_blank" rel="noopener noreferrer">Nº 2 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 51, 2015</a></p>
<p>&nbsp;<a href="http://portalsaude.saude.gov.br/images/pdf/2016/janeiro/11/2015-053-para-substituir-na-p--gina.pdf" target="_blank" rel="noopener noreferrer">Nº 1 - Monitoramento dos casos de microcefalias no Brasil até a Semana Epidemiológica 51</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">2015</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/dezembro/28/2015-boletim-microcefalia-se50-vol46-n46.pdf" target="_blank" rel="noopener noreferrer">Nº 46 - Monitoramento dos casos de microcefalias no Brasil até a Semana Epidemiológica 50</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/dezembro/23/2015-050---microcefalia-SE-49---publica----o-23.12.15.pdf" target="_blank" rel="noopener noreferrer">Nº 45 - Monitoramento dos casos de microcefalias no Brasil até a Semana Epidemiológica 49</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2016/janeiro/07/2015-svs-be-pncd-se48.pdf" target="_blank" rel="noopener noreferrer">Nº 44 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 48, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/dezembro/16/2015-003---Mal--ria.pdf" target="_blank" rel="noopener noreferrer">Nº 43 - Situação epidemiológica da malária no Brasil, 2012 e 2013</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/dezembro/11/svs-be-2015-047-dengue-se47-final.pdf">Nº 42 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 47, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/dezembro/11/svs-be-2015-048-microcefalia-se48-final2.pdf">Nº 41 - Monitoramento dos casos de microcefalia no Brasil até a Semana Epidemiológica 48</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/dezembro/08/2014-015---Fl--or---prelo---01.12.15.pdf" target="_blank" rel="noopener noreferrer">Nº 40 - Monitoramento do parâmetro fluoreto na água para consumo humano e a situação da fluorose e cárie nas capitais brasileiras no ano de 2010</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/dezembro/08/2015-012---Coqueluche-08.12.15.pdf" target="_blank" rel="noopener noreferrer">Nº 39 - Coqueluche no Brasil: análise da situação epidemiológica de 2010 a 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/dezembro/02/2015-046---Microcefalia----02.12.15.pdf" target="_blank" rel="noopener noreferrer">Nº 38 - Monitoramento dos casos de microcefalias no Brasil, até a semana epidemiológica 47, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/novembro/30/Microcefalia-2-boletim.pdf" target="_blank" rel="noopener noreferrer">Nº 37 - Monitoramento dos casos de microcefalias no Brasil, até a semana epidemiológica 46, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/novembro/26/2015-dengue-SE45.pdf" target="_blank" rel="noopener noreferrer">Nº 36 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 45, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/novembro/24/2015-040.pdf" target="_blank" rel="noopener noreferrer">Nº 35 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 43, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/novembro/19/Microcefalia-bol-final.pdf" target="_blank" rel="noopener noreferrer">Nº 34 - Situação epidemiológica de ocorrência de microcefalias no Brasil, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/outubro/29/2015-035---Dengue-SE-40---29.10.15.pdf" target="_blank" rel="noopener noreferrer">Nº 33 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 40, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/outubro/22/2015-034---Dengue-SE-38.pdf" target="_blank" rel="noopener noreferrer">Nº 32 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 38, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/outubro/15/svsbe-denchikzik-v46-n31.pdf" target="_blank" rel="noopener noreferrer">Nº 31 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 36, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/outubro/14/besvs-pni-v46-n30.pdf">Nº 30 - Programa Nacional de Imunizações: aspectos históricos dos calendários de vacinação e avanços dos indicadores de coberturas vacinais, no período de 1980 a 2013</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/outubro/19/2015-032---FA-ok.pdf" target="_blank" rel="noopener noreferrer">Nº 29 - Reemergência da Febre Amarela Silvestre no Brasil, 2014/2015: situação epidemiológica e a importância da vacinação preventiva e da vigilância intensificada no período sazonal</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/outubro/01/2015-030-bol--2-.pdf" target="_blank" rel="noopener noreferrer">Nº 28 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 34, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/outubro/01/2015-030-bol--1-.pdf" target="_blank" rel="noopener noreferrer">Nº 27 -Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 32, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/agosto/26/2015-020-publica----o.pdf" target="_blank" rel="noopener noreferrer">Nº 26 - Febre pelo vírus Zika: uma revisão narrativa sobre a doença</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/agosto/18/2015-009---Mal--ria-para-publica----o.pdf" target="_blank" rel="noopener noreferrer">Nº 25 - Malária: Monitoramento dos casos no Brasil em 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/setembro/03/2015-029---SE-30.pdf" target="_blank" rel="noopener noreferrer">Nº 24 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 30, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/agosto/07/2015-008---FA.pdf" target="_blank" rel="noopener noreferrer">Nº 23 - Vigilância Epidemiológica da Febre Amarela e a importância da preparação para o monitoramento do período sazonal, Brasil, 2014-2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/agosto/07/2015-028---Dengue-SE-28.pdf" target="_blank" rel="noopener noreferrer">Nº 22 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 28, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/agosto/03/2014-020..pdf" target="_blank" rel="noopener noreferrer">Nº 21 - Doença de Chagas aguda no Brasil: série histórica de 2000 a 2013</a><br> <strong><br> </strong><a href="http://portalsaude.saude.gov.br/images/pdf/2015/julho/27/2015-027---Dengue-SE-26--.pdf" target="_blank" rel="noopener noreferrer">Nº 20 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 26, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/julho/20/20150716-Boletim-dengue-SE24-2.pdf" target="_blank" rel="noopener noreferrer">Nº 19 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 24, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/julho/06/2015-024---Dengue-SE-23.pdf" target="_blank" rel="noopener noreferrer">Nº 18 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 23, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/julho/02/Monitoramento-dos-casos-de-dengue-e-febre-de-chikungunya-23-NUCOM.pdf" target="_blank" rel="noopener noreferrer">Nº 17 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 22, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/julho/02/Monitoramento-dos-casos-de-dengue-e-febre-de-chikungunya-22-NUCOM.pdf" target="_blank" rel="noopener noreferrer">Nº 16 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 21, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/junho/30/Monitoramento-dos-casos-de-dengue-e-febre-de-chikungunya-20.pdf" target="_blank" rel="noopener noreferrer">Nº 15 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 20, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/maio/04/2015-016---Boletim-Dengue-SE15-2015.pdf" target="_blank" rel="noopener noreferrer">Nº 14 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 15, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/maio/04/2015-015---Boletim-Dengue-SE14-2015.pdf" target="_blank" rel="noopener noreferrer">Nº 13 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 14, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/abril/29/Boletim-Dengue-SE12-2015.pdf" target="_blank" rel="noopener noreferrer">Nº 12 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 12, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/abril/29/2015-013---Boletim-Dengue-SE11-2015.pdf" target="_blank" rel="noopener noreferrer">Nº 11 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 11, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/abril/22/Boletim-raca-cor-09-04-15-v2.pdf" target="_blank" rel="noopener noreferrer">Nº 10 - Indicadores de Vigilância em Saúde, analisados segundo a variável raça/cor</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/marco/27/2015-007---BE-Tuberculose---para-substitui----o-no-site.pdf" target="_blank" rel="noopener noreferrer">Nº 9 - Tuberculose: Detectar, tratar e curar: desafios e estratégias brasileiras frente à tuberculose</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/marco/13/Boletim-Dengue-SE09-2015.final.pdf" target="_blank" rel="noopener noreferrer">Nº 8 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 9, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/fevereiro/25/2015-006---Boletim-Dengue-SE06-2015.pdf" target="_blank" rel="noopener noreferrer">Nº 7 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 6, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/fevereiro/25/2015-005---Boletim-Dengue-SE05-2015.pdf" target="_blank" rel="noopener noreferrer">Nº 6 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 5, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/fevereiro/11/Boletim-Dengue-SE04-2015..pdf" target="_blank" rel="noopener noreferrer">Nº 5 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 4, 2015</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/agosto/20/2014-028---Boletim-Agrotoxico-.pdf" target="_blank" rel="noopener noreferrer">Nº 4 - Monitoramento de Agrotóxicos na Água para Consumo Humano no Brasil, 2013</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/janeiro/19/2015-002---BE-at---SE-53.pdf" target="_blank" rel="noopener noreferrer">Nº 3 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 53 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/janeiro/22/BE-dengue-SE-52-20-01-15.pdf" target="_blank" rel="noopener noreferrer">Nº 2 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 52 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/janeiro/19/2014-042-ok-50.pdf" target="_blank" rel="noopener noreferrer">Nº 1 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 50 de 2014</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">2014</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/janeiro/02/2014-039---Dengue-SE-47.pdf" target="_blank" rel="noopener noreferrer">Nº 30 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 47 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/dezembro/11/2014-038---BE-Dengue-at---SE-46.pdf" target="_blank" rel="noopener noreferrer">Nº 29 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 46 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/dezembro/11/2014-037---Dengue-SE-45.pdf" target="_blank" rel="noopener noreferrer">Nº 28 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 45 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/dezembro/10/2014-036---BE-SE-44.pdf" target="_blank" rel="noopener noreferrer">Nº 27 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 44 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/outubro/27/BE-2014-45--26----Dengue-SE41-e-CHIKV-SE42.pdf" target="_blank" rel="noopener noreferrer">Nº 26 - Monitoramento dos casos de dengue até a semana epidemiológica (SE) 41 e febre de chikungunya até a SE 42 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/outubro/22/BE-2014-45--25----Dengue--SE40--e-CHIKV--SE41-.pdf" target="_blank" rel="noopener noreferrer">Nº 25 - Monitoramento dos casos de dengue até a semana epidemiológica (SE) 40 e febre de chikungunya até a SE 41 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/outubro/15/Boletim-Dengue-SE39-2.pdf" target="_blank" rel="noopener noreferrer">Nº 24 - Monitoramento dos casos de dengue até a semana epidemiológica (SE) 39 e febre de chikungunya até a SE 40 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/outubro/03/BE-2014-45--23----Dengue--SE38--e-CHIKV--SE39-.pdf" target="_blank" rel="noopener noreferrer">Nº 23 - Monitoramento dos casos de dengue até a Semana Epidemiológica (SE) 38 e febre de chikungunya até a SE 39 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/setembro/30/BE-2014-45--22----Dengue--SE37--e-CHIKV--SE38-.pdf" target="_blank" rel="noopener noreferrer">Nº 22 - Monitoramento dos casos de dengue até a Semana Epidemiológica (SE) 37 e febre de chikungunya até a SE 38 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/setembro/30/BE-2014-45--21----Dengue--SE36--e-CHIKV--SE37-.pdf" target="_blank" rel="noopener noreferrer">Nº 21 - Monitoramento dos casos de dengue até a Semana Epidemiológica (SE) 36 e febre de chikungunya até a SE 37 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/setembro/30/BE-2014-45--20----Dengue--SE35--e-CHIKV--SE36-.pdf" target="_blank" rel="noopener noreferrer">Nº 20 - Monitoramento dos casos de dengue Semana Epidemiológica (SE) 35 e febre de chikungunya SE 36 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/setembro/01/Boletim-Dengue-SE32.pdf" target="_blank" rel="noopener noreferrer">Nº 19 - Dengue: monitoramento até a Semana Epidemiológica (SE) 32 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/agosto/13/BE-2014-45--18----Tuberculose.pdf" target="_blank" rel="noopener noreferrer">Nº 18 - Tuberculose, população indígena e determinantes sociais</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/agosto/04/BE-2014-45--17----Dengue-SE29.pdf" target="_blank" rel="noopener noreferrer">Nº 17 - Dengue: monitoramento até a Semana Epidemiológica (SE) 29 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/17/BE-2014-45--16----Dengue--SE27-.pdf" target="_blank" rel="noopener noreferrer">Nº 16 - Dengue: monitoramento até a Semana Epidemiológica (SE) 27 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/14/Be-2014-45--15----Dengue--SE26-.pdf" target="_blank" rel="noopener noreferrer">Nº 15 - Dengue: monitoramento até a Semana Epidemiológica (SE) 26 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/02/BE-2014-45--14-.pdf" target="_blank" rel="noopener noreferrer">Nº 14 - Dengue: monitoramento até a Semana Epidemiológica (SE) 25 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/27/BE--2014-Dengue-SE24.pdf" target="_blank" rel="noopener noreferrer">Nº 13 - Dengue: monitoramento até a Semana Epidemiológica (SE) 24 de 2014</a><strong>
 </strong><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/20/be2014-45-12-dengue-se23.pdf" target="_blank" rel="noopener noreferrer">Nº 12 - Dengue: monitoramento até a Semana Epidemiológica (SE) 23 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/13/BE-2014-45--11--final.pdf" target="_blank" rel="noopener noreferrer">Nº 11 - Dengue: monitoramento até a Semana Epidemiológica (SE) 22 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/04/BE-2014-45--10-.pdf" target="_blank" rel="noopener noreferrer">Nº 10 - Dengue: monitoramento até a Semana Epidemiológica (SE) 21 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/maio/29/BE-2014-45--9-.pdf" target="_blank" rel="noopener noreferrer">Nº 9 - Dengue: monitoramento até a Semana Epidemiológica (SE) 20 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/maio/29/BE-2014-45--8----Copa-do-Brasil.pdf" target="_blank" rel="noopener noreferrer">Nº 8 - Preparação e resposta da vigilância em saúde para a Copa do Mundo da FIFA Brasil 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/maio/27/BE-V45-n---07-FebreAmarela.pdf" target="_blank" rel="noopener noreferrer">Nº 7 - Febre Amarela: Aspectos epidemiológicos da febre amarela silvestre e a vigilância intensificada durante período de monitoramento, Brasil, 2012/2013</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/maio/20/Boletim-Dengue3-abril-2014.pdf" target="_blank" rel="noopener noreferrer">Nº 6 - Dengue: monitoramento até a Semana Epidemiológica (SE) 18 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/maio/29/BE-2014-45--5--semana-16.pdf" target="_blank" rel="noopener noreferrer">Nº 5 - Dengue: monitoramento até a Semana Epidemiológica (SE) 16 de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/maio/29/BE-2014-45--4--semana-15.pdf" target="_blank" rel="noopener noreferrer">Nº 4 - Dengue: monitoramento até a Semana Epidemiológica (SE) 15 de 2014, Brasil</a><strong><a href="http://portalsaude.saude.gov.br/images/pdf/2014/maio/29/BE-2014-45--4--semana-15.pdf" target="_blank" rel="noopener noreferrer"><br> </a><a href="http://portalsaude.saude.gov.br/images/pdf/2014/abril/23/Boletim-dengue-abril.pdf" target="_blank" rel="noopener noreferrer"><br> </a></strong><a href="http://portalsaude.saude.gov.br/images/pdf/2014/maio/29/BE-2014-45--3--primeiras-semanas.pdf" target="_blank" rel="noopener noreferrer">Nº 3&nbsp;- Dengue: monitoramento das primeiras Semanas Epidemiológicas de 2014</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/maio/29/BE-2014-45--2--tb.pdf" target="_blank" rel="noopener noreferrer">Nº 2 - O controle da tuberculose no Brasil: avanços, inovações e desafios</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/11/BE-2014-45--1--Cianobact--rias.pdf" target="_blank" rel="noopener noreferrer">Nº 1 - Monitoramento de cianobactérias nos mananciais de abastecimento de água para consumo humano no Brasil, 2012</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">2013</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/11/BE-2013-44--17----Agrot--xicos.pdf" target="_blank" rel="noopener noreferrer">Nº 17 - Monitoramento de agrotóxicos na água para consumo humano no Brasil, 2012</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/03/Boletim-Sarampo-02-12-13.pdf" target="_blank" rel="noopener noreferrer">Nº 16 - Surto de sarampo em Pernambuco
 </a><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/11/BE-2013-44--15----Influenza--SE29-.pdf" target="_blank" rel="noopener noreferrer">Nº 15 - Influenza: monitoramento até a semana epidemiológica 29 de 2013</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/11/BE-2013-44--14----Influenza.pdf" target="_blank" rel="noopener noreferrer">Nº 14 - Análise da situação epidemiológica da influenza. Brasil, 2012</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/11/BE-2013-44--13----TB.pdf" target="_blank" rel="noopener noreferrer">Nº 13 - O controle da tuberculose na população indígena</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/11/BE-2013-44--12----Vigitel.pdf" target="_blank" rel="noopener noreferrer">Nº 12 - Prevalência de fatores de risco e de proteção: população adulta brasileira - Vigitel 2011</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/11/BE-2013-44--11----Hanseniase.pdf" target="_blank" rel="noopener noreferrer">Nº 11 - Situação epidemiológica da hanseníase no Brasil - análise de indicadores selecionados na última década e desafios para eliminação</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/11/BE-2013-44--10----Agua--agrotoxicos-.pdf" target="_blank" rel="noopener noreferrer">Nº 10 - Monitoramento de agrotóxicos na água para consumo humano no Brasil, 2011</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/11/BE-2013-44--9----VIVA-SINAN.pdf" target="_blank" rel="noopener noreferrer">Nº 9 - Vigilância de violência doméstica, sexual e/ou outras violências: Viva/Sina - Brasil, 2011</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/11/BE-2013-44--8----VIVA-Inquerito.pdf" target="_blank" rel="noopener noreferrer">Nº 8 - Vigilância de violências e acidentes em serviços de urgência e emergência. Viva Inquérito - Capitais e Distrito Federal. Brasil, 2011</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/11/BE-2013-44--7----Influenza--SE19-.pdf" target="_blank" rel="noopener noreferrer">Nº 7 - Influenza: monitoramento da semana epidemiológica 19</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/11/BE-2013-44--6----Influenza--SE18-.pdf" target="_blank" rel="noopener noreferrer">Nº 6 - Influenza: monitoramento da semana epidemiológica 18</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/11/BE-2013-44--5----Influenza--SE17--1.pdf" target="_blank" rel="noopener noreferrer">Nº 5 - Influenza: monitoramento da semana epidemiológica 17</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/11/BE-2013-44--4----Influenza--SE16-.pdf" target="_blank" rel="noopener noreferrer">Nº 4 - Influenza: monitoramento da semana epidemiológica 16</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/11/BE-2013-44--3----Influenza--SE15-.pdf" target="_blank" rel="noopener noreferrer">Nº 3 - Influenza: monitoramento das primeiras 15 semanas epidemiológicas de 2013</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/maio/06/boletim2-2013-tb-web.pdf" target="_blank" rel="noopener noreferrer">Nº 2 - Tuberculose: alinhada com o social, afinada com a tecnologia</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/11/BE-2013-44--1----Mal--ria.pdf" target="_blank" rel="noopener noreferrer">Nº 1 - Situação epidemiológica da malária no Brasil, 2000 a 2011</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">2012</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/23/BE-2012-43-Mar--o---Protocolo-Influenza.pdf" target="_blank" rel="noopener noreferrer">Protocolo de tratamento de influenza - 2011</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/23/BE-2012-43-Mar--o---Especial-Tuberculose.pdf" target="_blank" rel="noopener noreferrer">Especial Tuberculose</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/23/BE-2012-43--1--pag-1-a-7---Mortalidade-Materna.pdf" target="_blank" rel="noopener noreferrer">Mortalidade materna no Brasil</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/23/BE-2012-43--1--pag-8-a-10-Aids-no-Brasil.pdf" target="_blank" rel="noopener noreferrer">Aids no Brasil: epidemia concentrada e estabilizada em populações de maior vulnerabilidade</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/23/BE-2012-43--1--pag-11-a-15-Dengue.pdf" target="_blank" rel="noopener noreferrer">Dengue: situação epidemiológica (de janeiro a abril de 2012)</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/23/BE-2012-43--1--pag-15-19-Tuberculose-no-Brasil.pdf" target="_blank" rel="noopener noreferrer">Tuberculose no Brasil: realidade e perspectivas</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/23/BE-2012-43--2--pag-1-a-5-Chagas.pdf" target="_blank" rel="noopener noreferrer">Investigação de pseudo-surto da doença de Chagas aguda em Altamira/Pará, julho de 2010</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/23/BE-2012-43--2--pag-6-a-9-Hantavirose.pdf" target="_blank" rel="noopener noreferrer">Investigação de casos e aumento da letalidade por hantavirose, Distrito Federal, 2010</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/23/BE-2012-43--2---pag-10-a-11-Malaria.pdf" target="_blank" rel="noopener noreferrer">Surto de malária no assentamento Três Pontões, município de Nova Venécia, estado do Espírito Santo,2012</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/23/BE-2012-43--2---pag-12-a-16-Toxoplasmose.pdf" target="_blank" rel="noopener noreferrer">Investigação de casos de toxoplasmose aguda em Paraty e Angra dos Reis – Rio de Janeiro, Agosto de 2010</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/23/BE-2012-43--3--pag-1-a-5-Coqueluche.pdf" target="_blank" rel="noopener noreferrer">Surto de Coqueluche em Santana do Mundaú, Alagoas, julho de 2010</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/23/BE-2012-43--3--pag-6-a-10-SRAG.pdf" target="_blank" rel="noopener noreferrer">Investigação de casos de Síndrome Respiratória Aguda Grave no Município de Coruripe-Alagoas,outubro de 2010</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/23/BE-2012-43--3--pag-11-a-16-Sindrome-gripal.pdf" target="_blank" rel="noopener noreferrer">Investigação de Surto de Sindrome Gripal em Populações Indígenas, Altamira – Pará, abril – setembro de 2010</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/23/BE-2012-43--3--pag-17-a-20-Sarampo.pdf" target="_blank" rel="noopener noreferrer">Investigação de surto de sarampo no Rio Grande do Sul, agosto de 2010</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/23/BE-2012-43--4--pag-1-a-6-Cobertura-vacinal.pdf" target="_blank" rel="noopener noreferrer">Inquérito de cobertura vacinal em menores de 5 anos, fronteira Brasil-Colômbia, julho de 2011</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/23/BE-2012-43--4---pag-7-a-10-Agente-ambiental.pdf" target="_blank" rel="noopener noreferrer">Avaliação de Conhecimentos, Atitudes e Práticas dos Agentes de Saúde Ambiental e Controle de Endemias (Asace) do município do Recife em relação ao Diflubenzuron, Pernambuco, 2011</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/23/BE-2012-43--4---pag-11-a-15-micobacteria.pdf" target="_blank" rel="noopener noreferrer">Investigação de surto de infecção por micobactéria de crescimento rápido em pacientes submetidos a procedimentos de videocirurgia no município de Manaus, agosto de 2010</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">2010</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/fevereiro/23/boletim-epi-n9-dez2010-.pdf" target="_blank" rel="noopener noreferrer">Investigação do surto de leptospirose no município de Pacoti, Ceará, em 2009</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/fevereiro/23/boletim-epi-n8-dez2010-.pdf" target="_blank" rel="noopener noreferrer">Surto de hepatite viral tipo A no município de Águas Lindas de Goiás, em 2009</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/fevereiro/23/boletim-epi-n7-dez2010-.pdf" target="_blank" rel="noopener noreferrer">Investigação de surto de botulismo alimentar em Coruripe/AL, em abril de 2009</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/fevereiro/23/boletim-epi-n6-dez2010-2-.pdf" target="_blank" rel="noopener noreferrer">Surto de varicela em Várzea, na Paraíba, setembro, 2008</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/fevereiro/23/boletim-epi-n5-17dez2010-.pdf" target="_blank" rel="noopener noreferrer">Surto de malária no Distrito de Jaci-Paraná, municípios de Porto Velho - Ro, em 2009</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/fevereiro/23/boletim-epi-n4-18dez2010-.pdf" target="_blank" rel="noopener noreferrer">Investigação do surto de malária em Anajás– Pará</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2015/fevereiro/23/boletim-epi-n3-dez2010-.pdf" target="_blank" rel="noopener noreferrer">Surto de doença meningocócica no município de Itaguaí-RJ, 2009</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">2008</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n01-obitos-criancas-aldeia-ro-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação epidemiológica de óbitos de crianças indígenas da Aldeia Lage Velho, Etnia Pacaás Novos, Guajará-Mirim-RO, Feveiro de 2007</a>
 <a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n02-surto-deteccao-rotavirus-to-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação de surto de doença diarreica aguda com detecção de rotavírus do Tocatins-TO, Julho a agosto de 2006</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n03-intox-alim-tapioca-ce-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação de surto de intoxicação alimentar por ingestão de tapioca em Sobral-CE, 2007</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n04-surtos-dchagas-aguda-pa-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação de dois surtos de doença de chagas aguda na região do arquipélago do Marajó, Estado do Pará, Brasil, em 2007</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n05-surto-febre-hem-dengue-pa-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação de surto de febre hemorrágica da dengue, Redenção-PA, dezembro de 2007</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n06-hep-a-caract-casos-sinan-br-completo.pdf" target="_blank" rel="noopener noreferrer">Hepatite viral tipo A - caracterização dos casos notificados pelo Sistema Nacional de Vigilância no Brasil, 2006</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n07-surto-rubeola-ce-completo.pdf" target="_blank" rel="noopener noreferrer">Surto de rubéola em Fortaleza-CE, janeiro a junho de 2007</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n08-surto-diarreia-aguda-df-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação de surto de doença diarreica aguda em evento científico, Brasília-DF, 2007</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n09-cob-vacinal-surto-rubeola-ce-completo.pdf" target="_blank" rel="noopener noreferrer">Levantamento rápido de cobertura vacinal para intervenção em um surto de rubéola em Horizonte-CE, agosto de 2007</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n10-infeccao-hosp-micob-pr-completo.pdf" target="_blank" rel="noopener noreferrer">Surto de infecção hospitalar por microbactérias de crescimento rápido, Curitiba-PR, 2007</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n11-surto-rotavirus-cob-vacinal-ms-completo.pdf" target="_blank" rel="noopener noreferrer">Surto de doença diarreica aguda por rotavírus e levantamento de cobertura de vacina oral de rotavírus humano, Corumbá-MS, 2007</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n12-surto-brucelose-humana-to-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação de casos de brucelose humana em Araguaína no Estado do Tocantins, Brasil, junho de 2008</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n13-surto-intox-piretroides-go-completo.pdf" target="_blank" rel="noopener noreferrer">Surto de intoxicação por piretróides no Município de Porangatu-GO</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n14-surto-diarreia-aguda-msdf-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação de surto de doença diarreica aguda no Edifício-Sede do Ministério da Saúde, Distrito Federal</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n15-surto-rubeola-pa-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação de surto de rubéola em Ananindeua-PA, julho de 2008</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n16-surto-diarreia-aguda-go-completo.pdf" target="_blank" rel="noopener noreferrer">Surto de doença diarreica aguda em Anápolis, Goiás</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n17-surto-dmeningococica-go-completo.pdf" target="_blank" rel="noopener noreferrer">Surto de doença meningocócica no Município de Rio Verde-GO. Brasil</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n18-meningite-eosinofilica-es-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação de casos de meningite eosinofilica causada pela infecção por Angiostrongylus cantonensis no Espírito Santo. Brasil</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n19-surto-micob-cirurg-pr-completo.pdf" target="_blank" rel="noopener noreferrer">Surto de infecção hospitalar por microbactérias de crescimento rápido em procedimento de videocirurgia, Curitiba-PR</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/junho/25/Ano08-n20-surto-leptospirose-ce-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação de surto de leptospirose em Várzea Alegre-CE</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">2007</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/16/Ano07-n01-surto-dengue-conh-atit-prat-to-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação de surto de dengue em municpipio do Estado do Tocatins e avaliação de conhecimentos, atitudes e práticas da população em relaçãoà dengueduranteo surto em 2006</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/16/Ano07-n02-surto-rubeola-mg-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação de surto de rubéola em Belo Horizonte, Estado de Minas Gerais</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/16/Ano07-n03-eventos-imunogl-humana-mg-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação de eventos adversos à imunoglobulina humana intravenosa total no Estado de Minas Gerais, Brasil</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/16/Ano07-n04-surto-gastroenterite-viral-ba-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação de surto de gastroenterite em Salvador-BA</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/16/Ano07-n05-endoftalmite-cirurg-oftam-rr-completo.pdf" target="_blank" rel="noopener noreferrer">Surto de endofiltamite aguda após cirurgias oftalmológicas em Boa Vista-RR</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/16/Ano07-n06-scp-hantavirus-br-completo.pdf" target="_blank" rel="noopener noreferrer">Síndrome cardiopulmonar por hantavírus no Brasil em 2006: estudo clínico epidemiológico</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/16/Ano07-n07-aval-notif-sifilis-cong-br-completo--1-.pdf" target="_blank" rel="noopener noreferrer">Avaliação da notificação da sífilis congênita no Brasil, 2000 a 2003</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/16/Ano07-n08-toxoplasmose-adquirida-go-completo.pdf" target="_blank" rel="noopener noreferrer">Surto de toxoplasmose adquirida, Anápolis-GO, Fevereiro de 2006</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">2006</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/18/Ano06-n01-surto-rotavirus-creche-df-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação de surto de gastroenterite associada a rotavírus em creche de Brasília-DF, no ano de 2005</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/18/Ano06-n02-raiva-hum-morcegos-ma-completo.pdf" target="_blank" rel="noopener noreferrer">Raiva humana transmitida por morcegos no Alto Turi, MA, Julho-Setembro de 2005</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/18/Ano06-n03-surto-familiar-toxoplasmose-rs-completo.pdf" target="_blank" rel="noopener noreferrer">Surto intrafamiliar de toxoplasmose, Santa Vitória do Palmar-RS, Julho de 2005</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/18/Ano06-n04-surto-rickettsiose-rj-completo.pdf" target="_blank" rel="noopener noreferrer">Surto de rickettsiose em Petrópolis-RJ, 2005</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/18/Ano06-n05-epidemiol-hantavirose-br-completo.pdf" target="_blank" rel="noopener noreferrer">Epidemiologia da hantavírus e da síndrome cardiopulmonar por hantavírus no Brasil no ano de 2005</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/18/Ano06-n06-conj-aguda-adiaspiromicose-to-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação epidemiológica do surto de doença ocular, Araguatins/TO, 2005</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/18/Ano06-n07-susp-intox-pi-completo--1-.pdf" target="_blank" rel="noopener noreferrer">Investigação de surto suspeito de intoxicação exógena, Riveiro Gonçalves-PI, 2005</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/18/Ano06-n08-surto-rotavirus-ac-completo.pdf" target="_blank" rel="noopener noreferrer">Surto de gastroenterite por rotavírus no Município de Rio Branco-AC</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/18/Ano06-n09-surto-malaria-df-completo.pdf" target="_blank" rel="noopener noreferrer">Surto de malária autóctone por Plasmodium vivax no Distrito Federal - Maio de 2005</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/18/Ano06-n10-surto-susp-histoplasmose-df-completo.pdf" target="_blank" rel="noopener noreferrer">Surto de doença febril de etiologia desconhecida em adolescentes que visitaram a gruta do Taboril em Unaí/MG - Stembro de 2004</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">2005</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/15/Ano05-n01-surto-scp-hantavirus-df-completo.pdf" target="_blank" rel="noopener noreferrer">Surto de síndrome cardiopulmonar por hantavírus no Distrito Federal e Goiás - Maio a Setembro de 2004</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/15/Ano05-n02-novo-rsi-completo.pdf" target="_blank" rel="noopener noreferrer">O novo Regulamento Sanitário Internacional</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/15/Ano05-n03-asp-epid-patog-hant-br-completo.pdf" target="_blank" rel="noopener noreferrer">Aspectos epidemiológicos da infecção e da patogenicidade por hantavírus no Brasil (2004)</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/15/Ano05-n04-evento-adv-vac-tv-pr-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação de eventos adversos após a administração de vacina tríplice viral durante a campanha de seguimento contra sarampo em 2004</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/15/Ano05-n05-fasciola-hepatica-am-completo.pdf" target="_blank" rel="noopener noreferrer">Detecção de casos humanos de fascíola hepática no Estado do Amazonas</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/15/Ano05-n06-ve-dta-brasil-completo.pdf" target="_blank" rel="noopener noreferrer">Vigilância epidemiológica das doenças transmitidas por alimentos no Brasil, 1999 - 2004</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/15/Ano05-n07-surto-hep-a-mg-completo.pdf" target="_blank" rel="noopener noreferrer">Surto de hepatite viral tipo A (HVA), Minas Gerais, 2004</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">2004</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/16/Ano04-n01-desafio-influenza-br-completo.pdf" target="_blank" rel="noopener noreferrer">O desafio da influenza: epidemiologia e organização da vigilância no Brasil</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/16/Ano04-n02-inq-sorol-aves-virus-fno-rn-completo.pdf" target="_blank" rel="noopener noreferrer">Inquérito sorológico em aves migratórias e residentes de Galinhos/RN para detecção do vírus da febre do Nilo ocidental e outros vírus</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/16/Ano-04-n03-obitos-hemodialise-pe.pdf.pdf" target="_blank" rel="noopener noreferrer">Investigação de óbitos ocorridos em serviço de homodiálise de Caruarú, Pernambuco - Julho 2003</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/16/Ano04-n03-surto-gast-astro-noro-virus-rj.pdf.pdf" target="_blank" rel="noopener noreferrer">Investigação de surto de gastroenterite associado com astrovírus sorotipo 1, Itatiaia, RJ - Agosto de 2002</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/16/Ano04-n03-endoftalmites-pos-cirug-rj.pdf.pdf" target="_blank" rel="noopener noreferrer">Investigação sobre a ocorrência de endofitalmites pós-cirúrgica de catarata, RJ, Junho de 2003</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/16/Ano04-n04-sve-sindr-feb-ict-hem-br-completo.pdf" target="_blank" rel="noopener noreferrer">Sistema de Vigilância da Síndrome Febril Ictérica Aguda e/ou Síndrome Febril Hemorrágica Aguda. Brasil, 2004</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/16/Ano04-n05-2---inq-sorol-aves-virus-fno-rs-completo.pdf" target="_blank" rel="noopener noreferrer">Segundo inquérito sorológico em aves migratórias e residentes do Parque Nacional Lagoa do Peixe/RS para detecção do vírus do Nilo Ocidental e outros vírus</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/16/Ano04-n06-surto-raiva-hum-morcegos-pa-completo.pdf" target="_blank" rel="noopener noreferrer">Surto de raiva humana transmitida por morcegos no Município de Portel-Pará, Março/Abril de 2004</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/16/Ano04-n07-obitos-humanos-epizootia-rn-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação de óbitos humanos de causa desconhecida ocorridos no Rio Grande do Norte e de Epizootia em saguis (Callitherix jaccus) no parque das Dunas, Rio Grande do Norte, 2004</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/16/Ano04-n08-sdiarreia-vcholerae-pe-completo.pdf" target="_blank" rel="noopener noreferrer">Investigação de surto de diarreia aguda com isolamento de&nbsp;<em>V. cholerae&nbsp;</em>o1 Ogawa toxigênico no município de São Bento do Una - PE</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">2003</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/17/Ano03-n01-obito-mening-criptocococica-ro.pdf" target="_blank" rel="noopener noreferrer">Investigação de óbitos por meningite criptocócica com provável transmissão por fezes de quirópteros. Rondônia, 2002</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/17/Ano03-n01-1---inq-sorol-aves-virus-fno-rs.pdf" target="_blank" rel="noopener noreferrer">Primeiro inquérito sorológico em aves migratórias e nativas do Parque Nacional Lagoa do Peixe/RS para detecção do vírus do Nilo Ocidental</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/17/Ano03-n02-surto-hep-a-indigenas-mt.pdf%20(1).pdf" target="_blank" rel="noopener noreferrer">Surto de hepatite viral do tipo A em população indígena. Aldeia Kukuruzinho. Mato Grosso, 2001</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/17/Ano03-n02-surto-rotavirus-es.pdf" target="_blank" rel="noopener noreferrer">Surto de rotavírus em Ecoporanga-ES, agosto de 2002</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/17/Ano03-n03-intoxicacao-radiol-bario-go.pdf.pdf" target="_blank" rel="noopener noreferrer">Investigação de surto de intoxicação pelo bário. Goiás, Junho de 2003</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/17/Ano03-n03-reacoes-sacarato-hidroxido-ce.pdf.pdf" target="_blank" rel="noopener noreferrer">Reações adversas depois da administração do sacarato de hidróxido de ferro III endovenoso em Unidades de Hemodiálise, Fortaleza-CE, 2002 - Resultados preliminares</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/17/Bol-Eletr-Ano03-05-2003-completo.pdf" target="_blank" rel="noopener noreferrer">Reunião Nacional das Leishmanioses - Uberaba - MG, 22 a 24 de outubro de 2003</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/17/Ano03-n06-epidemia-fa-mg-completo.pdf" target="_blank" rel="noopener noreferrer">Epidemia de febre amarela silvestre na região da Bacia do Rio Doce - Minas Gerais. Dezembro de 2002 a Março de 2003</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2014/julho/17/Ano03-n04-reuniao-aval-dchagas-mg-completo.pdf" target="_blank" rel="noopener noreferrer">II Reunião anual de avaliação do programa de controle da doença de chagas. Uberaba, MG, 22 a 24 de outubro de 2003</a></p>
</div>
	</li>
	</ul>
</div>
<div class="su-tabs-pane su-clearfix" style="display: block;">
<ul class="jp-accordion ui-accordion ui-widget ui-helper-reset ui-accordion-icons" id="accordion5e9d0b39d75b3" style="width:100%">
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Acidentes e violência </a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom" style="display: none;">
<p><a href="/images/pdf/2020/fevereiro/21/Boletim-epidemiologico-SVS-08.pdf" target="_blank">BE Vol. 51 Nº 8 - Vacinação para o viajante e população em geral; orientações para prevenção de doenças transmissíveis e acidentes por animais peçonhentos; vigilância das IST; uso de álcool: alerta para o período do carnaval</a></p>
<p><a href="/images/pdf/2020/janeiro/23/Boletim_epidemiologico_SVS_04.pdf" target="_blank">BE Vol. 51 Nº 4 - Novo coronavírus (2019-nCoV); e vigilância de violências e acidentes no Brasil: análise da cobertura da notificação compulsória de violência interpessoal/autoprovocada nos municípios brasileiros</a></p>
<p><a href="/images/pdf/2020/janeiro/03/Boletim-epidemiologico-SVS-40.pdf">BE Vol. 51 Nº 1 - </a><a href="/images/pdf/2020/janeiro/15/Boletim-epidemiologico-SVS-01.pdf" target="_blank">Situação epidemiológica, fluxograma classificação de risco e manejo clínico da febre amarela; Evento de monitoramento internacional: China – Pneumonia de etiologia desconhecida; Violência contra mulheres e meninas do campo, da floresta e das águas</a></p>
<p><a href="/images/pdf/2019/dezembro/16/Boletim-epidemiologico-SVS-37-interativo-final.pdf">BE Vol. 50 Nº 37 – Sarampo (SE 36 a 47) vigilância epidemiológica, apuração final da campanha, análise da mortalidade de pedestres no Brasil (2010 a 2017), monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (SE 1 a 47)</a><span style="color: #ffffff; font-family: open_sansregular, 'Open Sans', Arial, Helvetica, sans-serif; font-size: 16.17px;"></span></p>
<p><span style="color: #ffffff; font-family: open_sansregular, 'Open Sans', Arial, Helvetica, sans-serif; font-size: 16.17px;"></span><a href="/images/pdf/2019/novembro/07/Boletim-epidemiologico-SVS-33-7nov19.pdf" target="_blank" rel="noopener">BE Vol. 50 Nº 33, Sarampo (SE 32 a 43); vigilância laboratorial do sarampo; informações sobre vacinação contra o sarampo; situação dos imunobiológicos; Programa Vida no Trânsito e monitoramento das manchas de óleo no Nordeste</a></p>
<p><a href="http://saude.gov.br/images/pdf/2019/outubro/24/Boletim-epidemiologico-SVS-31.pdf">BE Vol. 50 Nº 31, 2019: Sarampo (SE 30 a 42 de 2019);&nbsp; arboviroses (SE 1 a 40 de 2019) e óbitos por lesões de trânsito (2000 a 2017)</a><a href="/images/pdf/2019/junho/13/2019-010-publicacao.pdf" target="_blank"></a></p>
<p><a href="/images/pdf/2019/novembro/07/Boletim-epidemiologico-SVS-30.pdf">BE Vol. 50 Nº 30, 2019: &nbsp;Sarampo (SE 29 a 40 de 2019); violências contra mulheres (2011-2018)&nbsp; e CIOCS - Copa do Mundo FIFA Sub-17</a></p>
<p><a href="/images/pdf/2019/setembro/13/BE-suic--dio-24-final.pdf" target="_blank">BE Vol. 50 Nº 24, 2019: Perfil epidemiológico dos casos notificados de violência autoprovocada e óbitos</a></p>
<p><a href="/images/pdf/2019/junho/13/2019-010-publicacao.pdf" target="_blank">Vol. 50&nbsp; Nº&nbsp; 14, 2019: População em situação de rua e violência - uma análise das notificações no Brasil de 2015 a 2017</a></p>
<p><a href="/images/pdf/2018/junho/25/2018-024.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 27- 2018 - Análise epidemiológica da violência sexual contra crianças e adolescentes no Brasil, 2011 a 2017 </a></p>
<p><a href="/images/pdf/2018/junho/20/2017-033-publicacao.pdf">Vol. 49 - Nº 26- 2018 - Acidentes de transporte relacionados ao trabalho no Brasil, 2007-2016</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--9----VIVA-SINAN.pdf" target="_blank" rel="noopener noreferrer">Vol. 44 - Nº 9, 2013 - Vigilância de violência doméstica, sexual e/ou outras violências: Viva/Sinan - Brasil, 2011</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--8----VIVA-Inquerito.pdf" target="_blank" rel="noopener noreferrer">Vol. 44 - Nº 8, 2013 - Vigilância de violências e acidentes em serviços de urgência e emergência. Viva Inquérito - Capitais e Distrito Federal. Brasil, 2011</a>
 </p></div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Agentes químicos</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2018/dezembro/26/2018-027.pdf">Vol. 49 Nº 58, 2018: Intoxicações exógenas relacionadas ao trabalho no Brasil de 2007 a 2016</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Agrotóxicos</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2018/dezembro/26/2018-027.pdf">Vol. 49 Nº 58, 2018: Intoxicações exógenas relacionadas ao trabalho no Brasil de 2007 a 2016</a></p>
<p><a href="/images/pdf/2018/dezembro/18/2018-031.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 50, 2018 - Casos notificados de intoxicações exógenas relacionados ao glifosato no Brasil, no período de 2007 a 2016</a></p>
<p><a href="/images/pdf/2016/marco/23/2015-052---Agrot--xicos-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 12, 2016 - Monitoramento de agrotóxicos em água para consumo humano no Brasil, 2014 </a></p>
<p><a href="/images/pdf/2015/agosto/20/2014-028---Boletim-Agrotoxico-.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 4, 2015 - Monitoramento de Agrotóxicos na Água para Consumo Humano no Brasil, 2013</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--17----Agrot--xicos.pdf" target="_blank" rel="noopener noreferrer">Vol. 44 - Nº 17, 2013 - Monitoramento de agrotóxicos na água para consumo humano no Brasil, 2012</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--10----Agua--agrotoxicos-.pdf" target="_blank" rel="noopener noreferrer">Vol. 44 - Nº 10, 2013 - Monitoramento de agrotóxicos na água para consumo humano no Brasil, 2011</a>
 </p></div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Aids</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2019/novembro/29/Boletim-Ist-Aids-2019-especial-web.pdf">Boletim Epidemiológico - Número Especial, Dez. 2019, HIV/Aids 2019</a></p>
<p><a href="/images/pdf/2019/outubro/01/Boletim-tuberculose-2019.pdf" target="_blank">BE Vol. 50 Nº 26, 2019: Panorama epidemiológico da coinfecção TB-HIV no Brasil 2019: Setembro 2019</a></p>
<p><a href="/images/pdf/2017/dezembro/05/2017-030.pdf">Vol. 48 - Nº 40 - 2017 - Coinfecção TB-HIV no Brasil: panorama epidemiológico e atividades colaborativas</a><br> <a href="/images/pdf/2017/janeiro/05/2016_034-Aids_publicacao.pdf"><br> Vol. 48 - Nº 1 - 2017 - Boletim Epidemiológico HIV/Aids, 2016</a></p>
<p><a href="/images/pdf/2017/janeiro/05/2016_034-Aids_publicacao.pdf"></a><a href="/images/pdf/2014/julho/23/BE-2012-43--1--pag-8-a-10-Aids-no-Brasil.pdf" target="_blank" rel="noopener noreferrer">Vol. 43 - Nº 1 - 2012 - Aids no Brasil: epidemia concentrada e estabilizada em populações de maior vulnerabilidade</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Análise de situação de saúde</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/April/03/Boletim-epidemiologico-SVS-14.pdf">BE Vol. 51 Nº 14 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 13, 2020; febre amarela; diferenças regionais no consumo de alimentos entre adolescentes e adultos e leishmaniose visceral: Experiência do IEC</a></p>
<p><a href="/images/pdf/2020/marco/27/Boletim-epidemiologico-SVS-13.pdf" target="_blank">BE Vol.51 Nº 13 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 12, 2020; febre amarela; análise da anomalias congênitas no Brasil; abastecimento de água para consumo humano em 2019 e medicamentos antileishmania no âmbito do SUS</a></p>
<p><a href="/images/pdf/2019/outubro/31/Boletim-epidemiologico-SVS-32.pdf" target="_blank">BE Vol. 50 Nº 32, 2019: Vigilância em Saúde: monitoramento das manchas de óleo no litoral do Nordeste ; arboviroses (SE 1 a 41 de 2019) e consumo de bebidas alcoólicas entre escolares no Brasil: uma análise da Pesquisa Nacional de Saúde do Escolar, 2015</a></p>
<p><a href="/images/pdf/2019/julho/08/2019-007-Uso-de-servicos-de-saude-e-fatores-associados-a-procura-pela-UBS-entre-adolescentes-publicacao-04-07.pdf">BE Vol. 50&nbsp; Nº&nbsp; 16, 2019: Uso de serviços de saúde e fatores associados à procura pela Unidade Básica de Saúde entre adolescentes brasileiros: resultados da Pesquisa Nacional de Saúde do Escolar (PeNSE), 2015</a></p>
<p><a href="/images/pdf/2019/julho/17/2019-014-Publicacao-02-07.pdf">BE Vol. 50&nbsp; Nº&nbsp; 15, 2019: Suicídio: tentativas e óbitos por intoxicação exógena no Brasil, 2007 a 2016</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/setembro/21/2017-025-Perfil-epidemiologico-das-tentativas-e-obitos-por-suicidio-no-Brasil-e-a-rede-de-atencao-a-saude.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 30 - 2017 - Perfil epidemiológico das tentativas e óbitos por suicídio no Brasil e a rede de atenção à saúde</a></p>
<p><a href="/images/pdf/2017/fevereiro/17/Indicadores-de-Vigilancia-em-Saude-descritos-segundo-ra--a-cor.pdf">Vol. 48 - Nº 4 - 2017 - Indicadores de Vigilância em Saúde segundo a variável raça/cor, Brasil</a></p>
<p><a href="/images/pdf/2015/abril/22/Boletim-raca-cor-09-04-15-v2.pdf">Vol. 46 - Nº 10 - 2015 - Indicadores de Vigilância em Saúde, analisados segundo a variável raça/cor</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--13----TB.pdf">Vol. 44 - Nº 13 - 2013 - O controle da tuberculose na população indígena</a></p>
<p><a href="/images/pdf/2014/julho/23/BE-2012-43--3--pag-11-a-16-Sindrome-gripal.pdf">Vol. 43 - Nº 3 - 2012 - Investigação de Surto de Síndrome Gripal em Populações Indígenas, Altamira – Pará, abril – setembro de 2010</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Animais peçonhentos</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/marco/05/Boletim-epidemiologico-SVS-09--.pdf" target="_blank">BE Vol. 51 Nº 9 - Vigilância epidemiológica do sarampo&nbsp;no Brasil 2020: Semana Epidemiológica&nbsp;01 (29/12/2019) a 06 (08/02/2020)</a></p>
<p><a href="/images/pdf/2020/fevereiro/21/Boletim-epidemiologico-SVS-08.pdf" target="_blank">BE Vol. 51 Nº 8 - Vacinação para o viajante e população em geral; orientações para prevenção de doenças transmissíveis e acidentes por animais peçonhentos; vigilância das IST; uso de álcool: alerta para o período do carnaval</a></p>
<p><a href="/images/pdf/2019/outubro/04/BE-multitematico-n28.pdf" target="_blank">BE Vol. 50 Nº 28, 2019: sarampo (SE 28 a 39 de 2019); febre do Mayaro e febre do Oropouche; acidentes escorpiônicos; arboviroses urbanas transmitidas pelo Aedes Aegypti (SE 1 a 36 de 2019)</a></p>
<p><a href="http://portalarquivos2.saude.gov.br/images/pdf/2019/marco/29/2018-059.pdf" target="_blank" rel="noopener noreferrer">BE Vol. 50&nbsp; Nº&nbsp; 11, 2019: Acidentes de Trabalho por animais peçonhentos entre trabalhadores do campo, floresta e águas, Brasil 2007 a 2017</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Asbestose</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2016/fevereiro/02/2015-011---Asbestose.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 4 - 2016 - Asbestose na saúde do trabalhador notificados no Sinan, Brasil, 2007-2013</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Boletim especial</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2019/dezembro/05/Boletim-Epidemiologico-Especial-SVS-16-anos-web.pdf" target="_blank">Vigilância em Saúde no Brasil 2003 | 2019 - Da criação da Secretaria de Vigilância em Saúde aos dias atuais</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Botulismo</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom" style="display: none;"><p></p>
<p><a href="/images/pdf/2015/fevereiro/23/boletim-epi-n7-dez2010-.pdf">Vol. 10 - Nº 7 - 2010 - Investigação de surto de botulismo alimentar em Coruripe/AL, em abril de 2009</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Brucelose</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2014/junho/25/Ano08-n12-surto-brucelose-humana-to-completo.pdf">Vol. 8 - Nº 12 - 2008 - Investigação de casos de brucelose humana em Araguaína no Estado do Tocantins, Brasil, junho de 2008</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-active ui-corner-top"><a href="#">Coronavírus/COVID-19</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom ui-accordion-content-active" style="display: block;">
<p><a href="https://portalarquivos.saude.gov.br/images/pdf/2020/April/19/BE12-Boletim-do-COE.pdf">BE 12 - Boletim COE Coronavírus</a></p>
<p><a href="/images/pdf/2020/April/18/2020-04-17---BE11---Boletim-do-COE-21h.pdf" target="_blank">BE 11 - Boletim COE Coronavírus</a></p>
<p><a href="https://portalarquivos.saude.gov.br/images/pdf/2020/April/17/2020-04-16---BE10---Boletim-do-COE-21h.pdf" target="_blank" rel="noopener">BE 10 - Boletim COE Coronavirus</a></p>
<p><a href="https://portalarquivos.saude.gov.br/images/pdf/2020/April/12/2020-04-11-BE9-Boletim-do-COE.pdf" target="_blank" rel="noopener">BE9 - Boletim Especial do COE Coronavírus Avaliação de Risco</a></p>
<p><a href="/images/pdf/2020/April/09/be-covid-08-final-2.pdf" target="_blank">BE8 - Boletim Especial do COE Coronavírus Avaliação de Risco</a></p>
<p><a href="/images/pdf/2020/April/09/be-covid-08-final.pdf" target="_blank"></a><a href="/images/pdf/2020/April/06/2020-04-06-BE7-Boletim-Especial-do-COE-Atualizacao-da-Avaliacao-de-Risco.pdf" target="_blank">BE7 - Boletim Especial do COE Coronavírus Avaliação de Risco</a></p>
<p><a href="https://portalarquivos.saude.gov.br/images/pdf/2020/April/03/BE6-Boletim-Especial-do-COE.pdf">BE6 - Boletim Especial do COE Coronavírus Avaliação de Risc 03/04/2020</a></p>
<p><a href="/images/pdf/2020/marco/21/2020-03-13-Boletim-Epidemiologico-05.pdf" target="_blank">BE Coronavírus - 13/03/2020</a></p>
<p><a href="/images/pdf/2020/marco/04/2020-03-02-Boletim-Epidemiol--gico-04-corrigido.pdf">BE Coronavírus - 04/03/2020</a></p>
<p><a href="/images/pdf/2020/fevereiro/21/2020-02-21-Boletim-Epidemiologico03.pdf" target="_blank">BE Coronavírus - 21/02/2020</a></p>
<p><a href="https://portalarquivos2.saude.gov.br/images/pdf/2020/fevereiro/13/Boletim-epidemiologico-COEcorona-SVS-13fev20.pdf" target="_blank" rel="noopener">BE Coronavírus - 10/02/2020</a></p>
<p><a href="/images/pdf/2020/fevereiro/18/Boletim-epidemiologico-SVS-01-COE-inundacao.pdf">BE COE Inundação nº 01 - Gestão de Emergências em Saúde Pública por desastres: evento hidrometeorológico na região Sudeste do Brasil, 2020</a></p>
<p><a href="/images/pdf/2020/fevereiro/04/Boletim-epidemiologico-SVS-04fev20.pdf">BE Coronavírus - 03/02/2020</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Cobertura vacinal</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2019/outubro/10/Boletim-epidemiologico-SVS-29.pdf">BE Vol. 50 Nº 29, 2019: Cobertura nacional da vacina tríplice viral: primeira dose para crianças com até 1 ano, de 2015 a 2019 e arboviroses urbanas transmitidas pelo Aedes Aegypti (SE 1 a 38 de 2019)</a></p>
<p><a href="/images/pdf/2018/novembro/21/2018-028.pdf">Vol. 49 - Nº 52, 2018 - Perfil epidemiológico dos casos notificados dos erros de imunização, Brasil, 2016</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/agosto/21/2017_023-Situacao-Epidemiologica-da-Influenza-no-Brasil-ate-a-Semana-Epidemiologica-32-de-2017.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 25 - 2017 - Situação Epidemiológica da Influenza no Brasil, até a Semana Epidemiológica 32 de 2017</a></p>
<p><a href="/images/pdf/2015/outubro/14/besvs-pni-v46-n30.pdf"> Vol. 46 - Nº 30 - 2015 - Programa Nacional de Imunizações: aspectos históricos dos calendários de vacinação e avanços dos indicadores de coberturas vacinais, no período de 1980 a 2013</a></p>
<p><a href="/images/pdf/2014/julho/23/BE-2012-43--4--pag-1-a-6-Cobertura-vacinal.pdf">Vol. 43 - Nº 4 - 2012 - Inquérito de cobertura vacinal em menores de 5 anos, fronteira Brasil-Colômbia, julho de 2011</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n11-surto-rotavirus-cob-vacinal-ms-completo.pdf">Vol. 8 - Nº 11 - 2008 - Surto de doença diarreica aguda por rotavírus e levantamento de cobertura de vacina oral de rotavírus humano, Corumbá-MS, 2007</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n09-cob-vacinal-surto-rubeola-ce-completo.pdf">Vol. 8 - Nº 9 - 2008 - Levantamento rápido de cobertura vacinal para intervenção em um surto de rubéola em Horizonte-CE, agosto de 2007</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Cólera</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2014/julho/16/Ano04-n08-sdiarreia-vcholerae-pe-completo.pdf">Vol. 4 - Nº 8 - 2004 - Investigação de surto de diarreia aguda com isolamento de <em>V. cholerae </em>o1 Ogawa toxigênico no município de São Bento do Una - PE</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Coqueluche</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2015/dezembro/08/2015-012---Coqueluche-08.12.15.pdf">Vol. 46 - Nº 39 - 2015 - Coqueluche no Brasil: análise da situação epidemiológica de 2010 a 2014 Vol. 43 – Nº 3 - 2012 - Surto de Coqueluche em Santana do Mundaú, Alagoas, julho de 2010</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-active ui-corner-top"><a href="#">Dengue</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom ui-accordion-content-active" style="display: block;">
<p><a href="/images/pdf/2020/April/16/Boletim-epidemiologico-SVS-16.pdf" target="_blank">BE Vol. 51 Nº 16 -&nbsp;Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes Aegypti (dengue, chikungunya e zika), Semanas Epidemiológicas 1 a 15, 2020</a></p>
<p><a href="/images/pdf/2020/April/13/Boletim-epidemiologico-SVS-15.pdf" target="_blank">BE Vol. 51 Nº 15 - Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes Aegypti (dengue, chikungunya e zika), Semanas Epidemiológicas 1 a 14 de 2020, e da febre amarela (2019/2020)</a></p>
<p><a href="/images/pdf/2020/April/03/Boletim-epidemiologico-SVS-14.pdf">BE Vol. 51 Nº 14 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 13, 2020; febre amarela; diferenças regionais no consumo de alimentos entre adolescentes e adultos e leishmaniose visceral: Experiência do IEC</a></p>
<p><a href="/images/pdf/2020/marco/27/Boletim-epidemiologico-SVS-13.pdf" target="_blank">BE Vol.51 Nº 13 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 12, 2020; febre amarela; análise da anomalias congênitas no Brasil; abastecimento de água para consumo humano em 2019 e medicamentos antileishmania no âmbito do SUS</a><a href="/images/pdf/2020/marco/30/Boletim-epidemiologico-SVS-12.pdf"></a></p>
<p><a href="/images/pdf/2020/marco/30/Boletim-epidemiologico-SVS-12.pdf">BE Vol. 51 Nº 12 - Monitoramento dos casos de arboviroses (dengue, chikungunya e zika), SE 1 a 11, 2020; febre amarela 2019/2020; sarampo: SE 1 a 10, 2020; ações de enfrentamento da tuberculose em imigrantes e Síndrome congênita associada à infecção pelo vírus Zika: SE 10, 2020</a></p>
<p><a href="/images/pdf/2020/marco/23/Boletim-epidemiologico-SVS11.pdf" target="_blank">BE Vol. 51 Nº 11 - Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e zika), SE 1 a 10, 2020; Vigilância da infecção latente pelo Mycobacterium tuberculosis; Doença de Chagas - Protocolos clínicos de seguimento pós-tratamento em grupos especiais</a><a href="/images/pdf/2020/marco/30/Boletim-epidemiologico-SVS11.pdf" target="_blank"><br></a></p>
<p><a href="/images/pdf/2020/marco/06/Boletim-epidemiologico-SVS-10.pdf" target="_blank">BE Vol. 51 Nº10 Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika), Semanas Epidemiológicas 1 a 7, 2020.</a></p>
<p><a href="/images/pdf/2020/fevereiro/19/Boletim-epidemiologico-SVS-07.pdf" target="_blank">BE Vol. 51 Nº 7 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 05; síndrome congênita associada à infecção pelo vírus Zika até a SE 05 e mobilização nacional para enfrentamento da hanseníase</a></p>
<p><a href="/images/pdf/2020/April/08/Boletim-epidemiologico-SVS-06-v2.pdf" target="_blank" rel="noopener">BE Vol. 51 Nº 6 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 04; sarampo no Brasil 2019: janeiro a dezembro e situação da febre amarela: monitoramento 2019/2020</a></p>
<p><a href="/images/pdf/2020/fevereiro/05/Boletim-epidemiologico-SVS-05.pdf">BE Vol. 51 Nº 5 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 03; Síndrome Congênita associada à infecção pelo vírus Zika: monitoramento do ano de 2019 e a qualidade da água para consumo humano no Rio de Janeiro.</a></p>
<p><a href="/images/pdf/2020/janeiro/20/Boletim-epidemiologico-SVS-02-1-.pdf" target="_blank">BE Vol. 51 Nº 2 – Monitoramento de dengue, chikungunya e Zika, até a SE 52 de 2019; Campanha de vacinação contra o sarampo</a></p>
<p><a href="/images/pdf/2019/dezembro/23/Boletim-epidemiologico-SVS-38-2-interativo.pdf" target="_blank">BE Vol. 50 Nº 38 – Influenza (SE 1 a 49 de 2019); ampliação do diagnóstico da toxoplasmose congênita, monitoramento de dengue, chikungunya e Zika, até a SE 49; atualizações das indicações da vacina Febre Amarela e Campanha de Vacinação contra o HPV</a></p>
<p><a href="/images/pdf/2019/dezembro/16/Boletim-epidemiologico-SVS-37-interativo-final.pdf">BE Vol. 50 Nº 37 – Sarampo (SE 36 a 47) vigilância epidemiológica, apuração final da campanha, análise da mortalidade de pedestres no Brasil (2010 a 2017), monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (SE 1 a 47)</a></p>
<p><a href="https://portalarquivos2.saude.gov.br/images/pdf/2019/novembro/13/Boletim-epidemiologico-SVS-34-V1.pdf" target="_blank" rel="noopener">BE Vol. 50 Nº 34 - Monitoramento das manchas de óleo no Nordeste; sarampo (Orientações para 2ª etapa da vacinação) e arboviroses (SE 1 a 44)</a></p>
<p><a href="/images/pdf/2019/novembro/07/Boletim-epidemiologico-SVS-33-7nov19.pdf" target="_blank">BE Vol. 50 Nº 33, Sarampo (SE 32 a 43); vigilância laboratorial do sarampo; informações sobre vacinação contra o sarampo; situação dos imunobiológicos; Programa Vida no Trânsito e monitoramento das manchas de óleo no Nordeste</a></p>
<p><a href="/images/pdf/2019/outubro/31/Boletim-epidemiologico-SVS-32.pdf" target="_blank">BE Vol. 50 Nº 32, 2019: Vigilância em Saúde: monitoramento das manchas de óleo no litoral do Nordeste ; arboviroses (SE 1 a 41 de 2019) e consumo de bebidas alcoólicas entre escolares no Brasil: uma análise da Pesquisa Nacional de Saúde do Escolar, 2015</a></p>
<p><a href="http://saude.gov.br/images/pdf/2019/outubro/24/Boletim-epidemiologico-SVS-31.pdf">BE Vol. 50 Nº 31, 2019: Sarampo (SE 30 a 42 de 2019);&nbsp; arboviroses (SE 1 a 40 de 2019) e óbitos por lesões de trânsito (2000 a 2017)</a></p>
<p><a href="/images/pdf/2019/outubro/10/Boletim-epidemiologico-SVS-29.pdf" target="_blank">BE Vol. 50 Nº 29, 2019: Cobertura nacional da vacina tríplice viral: primeira dose para crianças com até 1 ano, de 2015 a 2019</a><a href="/images/pdf/2019/outubro/04/BE-multitematico-n28.pdf">&nbsp;arboviroses urbanas transmitidas pelo Aedes Aegypti (SE 1 a 38 de 2019)</a></p>
<p><a href="/images/pdf/2019/outubro/04/BE-multitematico-n28.pdf" target="_blank">BE Vol. 50 Nº 28, 2019: sarampo (SE 28 a 39 de 2019); febre do Mayaro e febre do Oropouche; acidentes escorpiônicos; arboviroses urbanas transmitidas pelo Aedes Aegypti (SE 1 a 36 de 2019)</a></p>
<p><a href="/images/pdf/2019/setembro/11/BE-arbovirose-22.pdf">BE Vol. 50&nbsp; Nº&nbsp; 22, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 34 de 2019</a></p>
<p><a href="/images/pdf/2019/abril/30/2019-013-Monitoramento-dos-casos-de-arboviroses-urbanas-transmitidas-pelo-Aedes-publicacao.pdf">BE Vol. 50&nbsp; Nº&nbsp; 13, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 12 de 2019</a><a href="http://portalarquivos2.saude.gov.br/images/pdf/2019/abril/01/2018-066.pdf" target="_blank" rel="noopener noreferrer"></a></p>
<p><a href="/images/pdf/2019/marco/25/2019-013-Monitoramento-dos-casos-de-arboviroses-publicacao-25-03-2019.pdf">BE Vol. 50&nbsp; Nº&nbsp; 10, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 11 de 2019</a></p>
<p><a href="/images/pdf/2019/marco/21/2019-006.pdf">BE Vol. 50&nbsp; Nº&nbsp; 07, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 7 de 2019</a></p>
<p><a href="/images/pdf/2019/marco/21/2019-005.pdf">BE Vol. 50&nbsp; Nº&nbsp; 06, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 6 de 2019</a></p>
<p><a href="/images/pdf/2019/fevereiro/26/2019-004-Dengue-SE-5-publica----o-18-02-2019.pdf">Vol. 50 Nº 05, 2019: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 5 de 2019</a></p>
<p><a href="/images/pdf/2019/janeiro/28/2019-002.pdf" target="_blank" rel="noopener noreferrer">Vol. 50&nbsp; Nº 04, 2019: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica SE 52&nbsp; 2018 </a></p>
<p><a href="/images/pdf/2019/janeiro/28/2018-068.pdf" target="_blank" rel="noopener noreferrer">Vol. 50&nbsp; Nº 01, 2019: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica SE 50 2018</a></p>
<p><a href="/images/pdf/2019/janeiro/02/2018-067.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 Nº 59, 2018: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 49 de 2018</a></p>
<p><a href="/images/pdf/2018/dezembro/18/2018-064.pdf">Vol. 49 Nº 57, 2018: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 48 de 2018</a></p>
<p><a href="/images/pdf/2019/janeiro/14/BE-Vol. 49 – Nº 56, 2018- dengue chikungunya zika_aed_10jan19.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 Nº 56, 2018: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 47 de 2018</a></p>
<p><a href="/images/pdf/2019/janeiro/14/BE-Vol. 49 - Nº 55, 2018- dengue chikungunya zika_aed_10jan19.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 Nº 55, 2018: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 46 de 2018</a></p>
<p><a href="/images/pdf/2019/janeiro/14/BE-Vol. 49 - Nº 53, 2018 - dengue chikungunya zika_aed_10jan19.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 53, 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 45&nbsp;de 2018</a></p>
<p><a href="/images/pdf/2018/novembro/23/Publicacao-2018-57-SE44.pdf">Vol. 49 - Nº 51, 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica&nbsp; 44&nbsp;de 2018</a></p>
<p><a href="/images/pdf/2018/novembro/13/2018-056.pdf">Vol. 49 Nº 48, 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 43&nbsp;de 2018</a></p>
<p><a href="/images/pdf/2018/novembro/05/2018-051-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-doen--a-aguda-pelo-v--rus-Zika-at---a-Semana-Epidemiol--gica-42-de-2018-publica--ao.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 44 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 42 de 2018 </a></p>
<p><a href="/images/pdf/2018/outubro/31/BE-2018-050-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-doen--a-aguda-pelo-v--rus-Zika-at---a-Semana-Epidemiol--gica-41-de-2018-ajustado.pdf" target="_blank" rel="noopener noreferrer">Vol. 49&nbsp; Nº&nbsp; 43, 2018: Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 41 de 2018</a></p>
<p><a href="/images/pdf/2018/outubro/26/2018-049.pdf">Vol. 49&nbsp; Nº&nbsp; 42, 2018: Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 38&nbsp;de 2018</a></p>
<p><a href="/images/pdf/2018/outubro/31/Vol. 49 Nº 40.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 40 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 36 de 2018</a></p>
<p><a href="/images/pdf/2018/outubro/31/Vol. 49 Nº 39.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 39 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 34 - 2018</a></p>
<p><a href="/images/pdf/2018/agosto/31/2018-043.pdf">Vol. 49 - Nº 36- 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 32 -2018</a></p>
<p><a href="/images/pdf/2018/outubro/31/Vol. 49 Nº 35.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 35 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 32 -2018</a></p>
<p><a href="/images/pdf/2018/agosto/21/Publicacao-BE-2018-SE-30.pdf">Vol. 49 -&nbsp; Nº&nbsp; 34- 2018 - &nbsp;Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 30 - 2018</a></p>
<p><a href="http://portalarquivos2.saude.gov.br/images/pdf/2018/agosto/07/2018_040.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 33- 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 28 - 2018</a></p>
<p><a href="/images/pdf/2018/julho/12/Monitoramento-dos-casos-ate-a-Semana-Epidemiologica-25-de-2018.pdf">Vol. 49 -&nbsp; Nº&nbsp; 32 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 25 de 2018</a></p>
<p><a href="/images/pdf/2018/julho/06/Monitoramento-dos-casos-de-dengue-23-018.pdf">Vol. 49 -&nbsp; Nº&nbsp; 31 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 23 de 2018</a></p>
<p><a href="/images/pdf/2018/julho/06/Monitoramento-dos-casos-de-dengue-21-de-2018.pdf">Vol. 49 -&nbsp; Nº&nbsp; 30 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 21 de 2018</a></p>
<p><a href="/images/pdf/2018/junho/25/2018-023.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 -&nbsp; Nº&nbsp; 28- 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 19_2018</a></p>
<p><a href="/images/pdf/2018/junho/11/2018-022.pdf">Vol. 49 -&nbsp; Nº&nbsp; 24- 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 17 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/28/2018-023.pdf">Vol. 49 -&nbsp; Nº&nbsp; 23 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 16 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/28/2018-022.pdf">Vol. 49 -&nbsp; Nº&nbsp; 22- 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 15 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/28/BE-2018-021.pdf">Vol. 49 -&nbsp; Nº&nbsp; 21- 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 09 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/07/2018-018.pdf">Vol. 49 -&nbsp; Nº&nbsp; 20 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 15 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/02/2018-017-Publicacao.pdf">Vol. 49 -&nbsp; Nº&nbsp; 18 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 14 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/02/2018-015-Publicacao.pdf">Vol. 49 -&nbsp; Nº&nbsp; 17 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 13 de 2018</a></p>
<p><a href="/images/pdf/2018/abril/27/BE-2018-14-SE-12.pdf">Vol. 49 - Nº 16- 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 12 de 2018</a></p>
<p><a href="/images/pdf/2018/abril/19/BE-2018-13-SE-11-Publicacao.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 15 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 11, de 2018</a></p>
<p><a href="/images/pdf/2018/abril/06/2018-012.pdf">Vol. 49 - Nº 14 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 10, de 2018</a></p>
<p><a href="/images/pdf/2018/abril/06/2018-011.pdf">Vol. 49 - Nº 13 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 09, de 2018</a></p>
<p><a href="/images/pdf/2018/marco/29/2018-010.pdf">Vol. 49 -&nbsp; Nº 12&nbsp;</a><a href="/images/pdf/2018/marco/06/2018-008-Publicacao.pdf">- 2018 -</a><a href="/images/pdf/2018/marco/29/2018-010.pdf">&nbsp;Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 8 de 2018</a></p>
<p><a href="/images/pdf/2018/marco/06/2018-008-Publicacao.pdf">Vol. 49 - Nº 9 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 07 de 2018</a></p>
<p><a href="/images/pdf/2018/marco/06/2018-007-Publicacao.pdf">Vol. 49 - Nº 8 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 06 de 2018 </a></p>
<p><a href="/images/pdf/2018/fevereiro/20/2018-007.pdf">Vol. 49 - Nº 7 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 5 de 2018</a></p>
<p><a href="/images/pdf/2018/fevereiro/20/2018-003-Final.pdf">Vol. 49 - Nº 6 - 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 52 de 2017</a></p>
<p><a href="/images/pdf/2018/fevereiro/19/2018-005.pdf">Vol. 49 - Nº 5 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 3 de 2018</a></p>
<p><a href="/images/pdf/2018/janeiro/23/Boletim-2018-001-Dengue.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 2 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 52 de 2017</a></p>
<p><a href="/images/pdf/2018/janeiro/23/Boletim-2017-051-Dengue.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 1 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 51 de 2017</a></p>
<p><a href="/images/pdf/2018/janeiro/10/2017-046-Publicacao.pdf">Vol. 48 Nº 45 -2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 50, 2017</a></p>
<p><a href="/images/pdf/2017/dezembro/26/2017-044.pdf">Vol. 48 - Nº 44 - 2017 - “Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 49 de 2017</a></p>
<p><a href="/images/pdf/2017/dezembro/26/BE-2017-043-Boletim.pdf">Vol. 48 - Nº 43 - 2017 - “Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 48 de 2017</a></p>
<p><a href="/images/pdf/2017/dezembro/21/BE-2017-042-Boletim.pdf">Vol. 48 - Nº 42 - 2017 - “Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 47 de 2017</a></p>
<p><a href="/images/pdf/2017/dezembro/12/Boletim-2017-043-final.pdf">Vol. 48&nbsp;- Nº &nbsp;41 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 44</a></p>
<p><a href="/images/pdf/2017/dezembro/08/2017-040.pdf">Vol. 48&nbsp; Nº 39 -2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 46 </a></p>
<p><a href="/images/PDF/2017/novembro/28/2017_039.pdf">Vol. 48&nbsp;- Nº 38 -2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 45, 2017</a></p>
<p><a href="/images/pdf/2017/novembro/13/2017-037.pdf">Vol. 48 - Nº 35 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 43 de 2017</a></p>
<p><a href="/images/pdf/2017/novembro/07/2017-035-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica-41.pdf">Vol. 48 - Nº 34 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 41 de 2017</a></p>
<p><a href="/images/pdf/2017/outubro/18/boletim-n32.pdf">Vol. 48 - Nº 32 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 39, 2017</a></p>
<p><a href="/images/pdf/2017/outubro/18/BE-Vol-48-n-31-2017-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-ate-a-Semana-Epidemiologica-37.pdf">Vol. 48 - Nº 31 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 37, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/setembro/15/2017-028-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica-35.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 29 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 35, 2017</a>&nbsp;</p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/agosto/29/2017-026-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica-33-de-2017.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 27 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 33, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/agosto/23/2017_024-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 26 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 31, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/julho/25/Boletim-2017_020-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-Zika.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 20 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 25, 2017</a></p>
<p><a href="/images/pdf/2017/maio/25/Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 16 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 19, 2017</a></p>
<p><a href="/images/pdf/2017/maio/05/Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 14 - 2017 -Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 15, 2017</a></p>
<p><a href="/images/pdf/2017/abril/06/2017-010-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiol--gica-12--2017.pdf">Vol. 48 - Nº 11 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 12, 2017</a></p>
<p><a href="/images/pdf/2017/marco/31/Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-10-%202017.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 9 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 10, 2017</a></p>
<p><a href="/images/pdf/2017/marco/28/2017_006-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica-7.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 7 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 7, 2017</a></p>
<p><a href="/images/pdf/2017/fevereiro/17/Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-febre-pelo-v--rus-Semana-Epidemiologica-4-2017.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 5 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 4, 2017</a></p>
<p><a href="/images/pdf/2017/abril/06/2017-002-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-febre-pelo-v--rus-Zika-ate-a-Semana-Epidemiologica-52--2016.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 3 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 52, 2016</a></p>
<p><a href="/images/pdf/2017/janeiro/12/2017_001%20-%20Dengue%20SE51_publicacao.pdf">Vol. 48 - Nº 2 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 51, 2016</a></p>
<p><a href="/images/pdf/2016/dezembro/20/2016-033---Dengue-SE49-publicacao.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 38 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 49, 2016</a></p>
<p><a href="/images/pdf/2016/outubro/18/2016-029-Dengue-publicacao-n-34.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 34 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 37, 2016</a></p>
<p><a href="/images/pdf/2016/agosto/10/2016-026--2-..pdf">Vol. 47 - Nº 31 - 2016 -&nbsp;</a><a href="/images/pdf/2016/agosto/10/2016-026--2-..pdf" target="_blank" rel="noopener noreferrer">Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 27, 2016</a></p>
<p><a href="/images/pdf/2016/julho/15/2016-boletim-epi-n28-dengue-chik-zika-se23.pdf">Vol. 47 - Nº 28 - 2016 -&nbsp;</a><a href="/images/pdf/2016/julho/15/2016-boletim-epi-n28-dengue-chik-zika-se23.pdf" target="_blank" rel="noopener noreferrer">Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 23, 2016</a></p>
<p><a href="/images/pdf/2016/junho/30/2016-021.pdf">Vol. 47 - Nº 27 - 2016 -</a><a href="/images/pdf/2016/junho/30/2016-021.pdf" target="_blank" rel="noopener noreferrer">Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 21, 2016</a></p>
<p><a href="/images/pdf/2016/junho/30/2016-020--1-.pdf">Vol. 47 - Nº 26 - 2016 -&nbsp;</a><a href="/images/pdf/2016/junho/30/2016-020--1-.pdf" target="_blank" rel="noopener noreferrer">Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 20, 2016</a></p>
<p><a href="/images/pdf/2016/junho/17/2016-019.pdf">Vol. 47 - Nº 25 - 2016 -&nbsp;Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 19, 2016</a></p>
<p><a href="/images/pdf/2016/junho/15/2016-018---Dengue-SE18-substitui----o.pdf">Vol. 47 - Nº 24 - 2016 -&nbsp;Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 18, 2016</a></p>
<p><a href="/images/pdf/2016/junho/14/boletim-epid-2016-n23-dcz-se17.pdf">Vol. 47 - Nº 23 - 2016 -&nbsp;Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 17, 2016</a></p>
<p><a href="/images/pdf/2016/maio/17/2016-016---Dengue-SE16-publica----o.pdf">Vol. 47 - Nº 20 - 2016 -&nbsp;Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 16, 2016</a></p>
<p><a href="/images/pdf/2016/junho/10/2016-014---Dengue-SE13-substitui----o.pdf">Vol. 47 - Nº 18 - 2016 -&nbsp;Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 13, 2016</a></p>
<p><a href="/images/pdf/2016/abril/11/2015-013---Dengue-SE9.pdf">Vol. 47 - Nº 16 - 2016 -&nbsp;Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 9, 2016</a></p>
<p><a href="/images/pdf/2016/abril/05/2016-012---Relatorio-reuniao-especialistas-Aedes-publica----o.pdf">Vol. 47 - Nº 15 - 2016 -&nbsp;Relatório da Reunião Internacional para Implementação de Alternativas para o Controle do&nbsp;<em>Aedes aegypti&nbsp;</em>no Brasil</a></p>
<p><a href="/images/pdf/2016/marco/29/2016-011-boletim-epi-dcz-se8.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 14 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 8, 2016&nbsp;</a></p>
<p><a href="/images/pdf/2016/marco/23/2016-008---DengueSE8-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 11 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 7, 2016&nbsp;</a></p>
<p><a href="/images/pdf/2016/marco/23/2016-007---Dengue-SE-6-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 10 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 6, 2016</a></p>
<p><a href="/images/pdf/2016/fevereiro/29/2016-006---Dengue-SE5-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 8 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 5, 2016</a></p>
<p><a href="/images/pdf/2016/fevereiro/29/2016-005---Dengue-SE4-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 7 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 4, 2016</a></p>
<p><a href="/images/pdf/2016/fevereiro/04/2016-004---Dengue-SE3.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 6 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 3, 2016</a></p>
<p><a href="/images/pdf/2016/fevereiro/04/2016-003-Dengue-SE2.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 5 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 2, 2016</a></p>
<p><a href="/images/pdf/2016/janeiro/15/svs2016-be003-dengue-se52.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 3 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 52, 2015</a></p>
<p><a href="/images/pdf/2016/janeiro/15/svs2016-be002-dengue-se51.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 2 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 51, 2015</a></p>
<p><a href="/images/pdf/2016/janeiro/07/2015-svs-be-pncd-se48.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 44 - 2015 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 48, 2015</a></p>
<p><a href="/images/pdf/2015/dezembro/11/svs-be-2015-047-dengue-se47-final.pdf">Vol. 46 - Nº 42 - 2015 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 47, 2015</a></p>
<p><a href="/images/pdf/2015/novembro/26/2015-dengue-SE45.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 36 - 2015 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 45, 2015</a></p>
<p><a href="/images/pdf/2015/novembro/24/2015-040.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 35 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 43, 2015&nbsp;</a></p>
<p><a href="/images/pdf/2015/outubro/29/2015-035---Dengue-SE-40---29.10.15.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 33 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 40, 2015&nbsp;</a></p>
<p><a href="/images/pdf/2015/outubro/22/2015-034---Dengue-SE-38.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 32 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 38, 2015&nbsp;</a></p>
<p><a href="/images/pdf/2015/outubro/15/svsbe-denchikzik-v46-n31.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 31 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 36, 2015</a></p>
<p><a href="/images/pdf/2015/outubro/01/2015-030-bol--2-.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 28 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 34, 2015</a></p>
<p><a href="/images/pdf/2015/outubro/01/2015-030-bol--1-.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 27 - 2015 -Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 32, 2015</a></p>
<p><a href="/images/pdf/2015/setembro/03/2015-029---SE-30.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 24 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 30, 2015</a></p>
<p><a href="/images/pdf/2015/agosto/07/2015-028---Dengue-SE-28.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 22 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 28, 2015</a></p>
<p><a href="/images/pdf/2015/julho/27/2015-027---Dengue-SE-26--.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 20 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 26, 2015</a></p>
<p><a href="/images/pdf/2015/julho/20/20150716-Boletim-dengue-SE24-2.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 19 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 24, 2015</a></p>
<p><a href="/images/pdf/2015/julho/06/2015-024---Dengue-SE-23.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 18 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 23, 2015</a></p>
<p><a href="/images/pdf/2015/julho/02/Monitoramento-dos-casos-de-dengue-e-febre-de-chikungunya-23-NUCOM.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 17 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 22, 2015</a></p>
<p><a href="/images/pdf/2015/julho/02/Monitoramento-dos-casos-de-dengue-e-febre-de-chikungunya-22-NUCOM.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 16 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 21, 2015</a></p>
<p><a href="/images/pdf/2015/junho/30/Monitoramento-dos-casos-de-dengue-e-febre-de-chikungunya-20.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 15 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 20, 2015</a></p>
<p><a href="/images/pdf/2015/maio/04/2015-016---Boletim-Dengue-SE15-2015.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 14 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 15, 2015</a></p>
<p><a href="/images/pdf/2015/maio/04/2015-015---Boletim-Dengue-SE14-2015.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 13 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 14, 2015</a></p>
<p><a href="/images/pdf/2015/abril/29/Boletim-Dengue-SE12-2015.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 12 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 12, 2015</a></p>
<p><a href="/images/pdf/2015/abril/29/2015-013---Boletim-Dengue-SE11-2015.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 11 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 11, 2015</a></p>
<p><a href="/images/pdf/2015/marco/13/Boletim-Dengue-SE09-2015.final.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 8 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 9, 2015</a></p>
<p><a href="/images/pdf/2015/fevereiro/25/2015-006---Boletim-Dengue-SE06-2015.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 7 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 6, 2015</a></p>
<p><a href="/images/pdf/2015/fevereiro/25/2015-005---Boletim-Dengue-SE05-2015.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 6 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 5, 2015</a></p>
<p><a href="/images/pdf/2015/fevereiro/11/Boletim-Dengue-SE04-2015..pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 5 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 4, 2015</a></p>
<p><a href="/images/pdf/2015/janeiro/19/2015-002---BE-at---SE-53.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 3 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 53 de 2014</a></p>
<p><a href="/images/pdf/2015/janeiro/22/BE-dengue-SE-52-20-01-15.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 2 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 52 de 2014</a></p>
<p><a href="/images/pdf/2015/janeiro/19/2014-042-ok-50.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 1 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 50 de 2014</a></p>
<p><a href="/images/pdf/2015/janeiro/02/2014-039---Dengue-SE-47.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 30 - 2014 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 47 de 2014</a></p>
<p><a href="/images/pdf/2014/dezembro/11/2014-038---BE-Dengue-at---SE-46.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 29 - 2014 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 46 de 2014</a></p>
<p><a href="/images/pdf/2014/dezembro/11/2014-037---Dengue-SE-45.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 28 - 2014 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 45 de 2014</a></p>
<p><a href="/images/pdf/2014/dezembro/10/2014-036---BE-SE-44.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 27 - 2014 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 44 de 2014</a></p>
<p><a href="/images/pdf/2014/outubro/27/BE-2014-45--26----Dengue-SE41-e-CHIKV-SE42.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 26 - 2014 - Monitoramento dos casos de dengue até a semana epidemiológica (SE) 41 e febre de chikungunya até a 42 de 2014</a></p>
<p><a href="/images/pdf/2014/outubro/22/BE-2014-45--25----Dengue--SE40--e-CHIKV--SE41-.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 25 - 2014 - Monitoramento dos casos de dengue até a semana epidemiológica (SE) 40 e febre de chikungunya até a SE 41 de 2014</a></p>
<p><a href="/images/pdf/2014/outubro/15/Boletim-Dengue-SE39-2.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 24 - 2014 - Monitoramento dos casos de dengue até a semana epidemiológica (SE) 39 e febre de chikungunya até a SE 40 de 2014</a></p>
<p><a href="/images/pdf/2014/outubro/03/BE-2014-45--23----Dengue--SE38--e-CHIKV--SE39-.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 23 - 2014 - Monitoramento dos casos de dengue até a Semana Epidemiológica (SE) 38 e febre de chikungunya até a SE 39 de 2014</a></p>
<p><a href="/images/pdf/2014/setembro/30/BE-2014-45--22----Dengue--SE37--e-CHIKV--SE38-.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 22 - 2014 - Monitoramento dos casos de dengue até a Semana Epidemiológica (SE) 37 e febre de chikungunya até a SE 38 de 2014</a></p>
<p><a href="/images/pdf/2014/setembro/30/BE-2014-45--21----Dengue--SE36--e-CHIKV--SE37-.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 21 - 2014 - Monitoramento dos casos de dengue até a Semana Epidemiológica (SE) 36 e febre de chikungunya até a SE 37 de 2014</a></p>
<p><a href="/images/pdf/2014/setembro/30/BE-2014-45--20----Dengue--SE35--e-CHIKV--SE36-.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 20 - 2014 - Monitoramento dos casos de dengue Semana Epidemiológica (SE) 35 e febre de chikungunya SE 36 de 2014</a></p>
<p><a href="/images/pdf/2014/setembro/01/Boletim-Dengue-SE32.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 19 - 2014 - Dengue: monitoramento até a Semana Epidemiológica (SE) 32 de 2014</a></p>
<p><a href="/images/pdf/2014/agosto/04/BE-2014-45--17----Dengue-SE29.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 17 - 2014 - Dengue: monitoramento até a Semana Epidemiológica (SE) 29 de 2014</a></p>
<p><a href="/images/pdf/2014/julho/17/BE-2014-45--16----Dengue--SE27-.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 16 - 2014 - Dengue: monitoramento até a Semana Epidemiológica (SE) 27 de 2014&nbsp;</a></p>
<p><a href="/images/pdf/2014/julho/14/Be-2014-45--15----Dengue--SE26-.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 15 - 2014 - Dengue: monitoramento até a Semana Epidemiológica (SE) 26 de 2014</a></p>
<p><a href="/images/pdf/2014/julho/02/BE-2014-45--14-.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 14 - 2014 - Dengue: monitoramento até a Semana Epidemiológica (SE) 25 de 2014</a></p>
<p><a href="/images/pdf/2014/junho/27/BE--2014-Dengue-SE24.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 13 - 2014 - Dengue: monitoramento até a Semana Epidemiológica (SE) 24 de 2014</a></p>
<p><a href="/images/pdf/2014/junho/20/be2014-45-12-dengue-se23.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 12 - 2014 - Dengue: monitoramento até a Semana Epidemiológica (SE) 23 de 2014</a></p>
<p><a href="/images/pdf/2014/junho/13/BE-2014-45--11--final.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 11 - 2014 - Dengue: monitoramento até a Semana Epidemiológica (SE) 22 de 2014</a></p>
<p><a href="/images/pdf/2014/junho/04/BE-2014-45--10-.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 10 - 2014 - Dengue: monitoramento até a Semana Epidemiológica (SE) 21 de 2014</a></p>
<p><a href="/images/pdf/2014/maio/29/BE-2014-45--9-.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 9 - 2014 - Dengue: monitoramento até a Semana Epidemiológica (SE) 20 de 2014</a></p>
<p><a href="/images/pdf/2014/maio/20/Boletim-Dengue3-abril-2014.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 6 - 2014 - Dengue: monitoramento até a Semana Epidemiológica (SE) 18 de 2014</a></p>
<p><a href="/images/pdf/2014/maio/29/BE-2014-45--5--semana-16.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 5 - 2014 - Dengue: monitoramento até a Semana Epidemiológica (SE) 16 de 2014</a></p>
<p><a href="/images/pdf/2014/maio/29/BE-2014-45--4--semana-15.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 4 - 2014 - Dengue: monitoramento até a Semana Epidemiológica (SE) 15 de 2014, Brasil</a></p>
<p><a href="/images/pdf/2014/maio/29/BE-2014-45--3--primeiras-semanas.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 3 - 2014 - Dengue: monitoramento das primeiras Semanas Epidemiológicas de 2014</a></p>
<p><a href="/images/pdf/2014/julho/23/BE-2012-43--1--pag-11-a-15-Dengue.pdf">Vol. 43 - Nº 1 - 2012 -&nbsp;Dengue: situação epidemiológica (de janeiro a abril de 2012)</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n05-surto-febre-hem-dengue-pa-completo.pdf">Vol. 8 - Nº 5 - 2008 -&nbsp;Investigação de surto de febre hemorrágica da dengue, Redenção-PA, dezembro de 2007</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano07-n01-surto-dengue-conh-atit-prat-to-completo.pdf">Vol. 7 - Nº 1 - 2007 -&nbsp;Investigação de surto de dengue em município do estado do Tocantins e avaliação de conhecimentos, atitudes e práticas da população em relação à dengue durante o surto em 2006</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Desastres Ambientais</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/janeiro/28/boletim-especial-27jan20-final.pdf" target="_blank">BE Número Especial - Um ano do desastre da Vale: Organização e resposta do Ministério da saúde para o desastre em Brumadinho (MG)</a></p>
<p><a href="/images/pdf/2019/novembro/29/Boletim-epidemiologico-SVS-36-interativo.pdf" target="_blank">BE Vol. 50 Nº 36 - Panorama da doença de Chagas no Brasil e monitoramento das manchas de óleo no litoral brasileiro</a></p>
<p><a href="/images/pdf/2019/novembro/20/Boletim-epidemiologico-SVS-35.pdf" target="_blank">BE Vol. 50 Nº 35 - Sarampo (SE 34 a 45), vigilância laboratorial do sarampo, vacinação contra o sarampo, orientações para a 2ª etapa da vacinação contra o sarampo, raiva, recomendações para uso dos imunobiológicos, malária (2017 a 2019) e monitoramento das manchas de óleo</a></p>
<p><a href="https://portalarquivos2.saude.gov.br/images/pdf/2019/novembro/13/Boletim-epidemiologico-SVS-34-V1.pdf" target="_blank" rel="noopener">BE Vol. 50 Nº 34 - Monitoramento das manchas de óleo no Nordeste; sarampo (Orientações para 2ª etapa da vacinação) e arboviroses (SE 1 a 44)</a></p>
<p><a href="https://portalarquivos2.saude.gov.br/images/pdf/2019/novembro/07/Boletim-epidemiologico-SVS-33-7nov19.pdf" target="_blank" rel="noopener">BE Vol. 50 Nº 33, Sarampo (SE 32 a 43); vigilância laboratorial do sarampo; informações sobre vacinação contra o sarampo; situação dos imunobiológicos; Programa Vida no Trânsito e monitoramento das manchas de óleo no Nordeste</a></p>
<p><a href="/images/pdf/2019/outubro/31/Boletim-epidemiologico-SVS-32.pdf" target="_blank">BE Vol. 50 Nº 32, 2019: Vigilância em Saúde: monitoramento das manchas de óleo no litoral do Nordeste ; arboviroses (SE 1 a 41 de 2019) e consumo de bebidas alcoólicas entre escolares no Brasil: uma análise da Pesquisa Nacional de Saúde do Escolar, 2015</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Doença de Chagas</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/marco/30/Boletim-epidemiologico-SVS-12.pdf">BE Vol. 51 Nº 12 - Monitoramento dos casos de arboviroses (dengue, chikungunya e zika), SE 1 a 11, 2020; febre amarela 2019/2020; sarampo: SE 1 a 10, 2020; ações de enfrentamento da tuberculose em imigrantes e Síndrome congênita associada à infecção pelo vírus Zika: SE 10, 2020</a></p>
<p><a href="/images/pdf/2020/marco/30/Boletim-epidemiologico-SVS11.pdf" target="_blank">BE Vol. 51 Nº 11 - Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e zika), SE 1 a 10, 2020; Vigilância da infecção latente pelo Mycobacterium tuberculosis; Doença de Chagas - Protocolos clínicos de seguimento pós-tratamento em grupos especiais</a><a href="/images/pdf/2020/marco/21/Boletim_epidemiologico_SVS_11.pdf" target="_blank"><br></a></p>
<p><a href="/images/pdf/2020/janeiro/16/Boletim-epidemiologico-SVS-40-v2.pdf">BE Vol. 50 Nº 40 - Panorama da vigilância de doenças crônicas não transmissíveis no Brasil e Fortalecimento das ações de prevenção, controle e eliminação da malária e ações de vigilância da leishmaniose visceral e doença de Chagas</a></p>
<p><a href="/images/pdf/2019/novembro/29/Boletim-epidemiologico-SVS-36-interativo.pdf" target="_blank">BE Vol. 50 Nº 36 - Panorama da doença de Chagas no Brasil e monitoramento das manchas de óleo no litoral brasileiro</a></p>
<p><a href="/images/pdf/2019/janeiro/23/2018-025.pdf">Vol. 50&nbsp; Nº&nbsp; 02, 2019: Doença de Chagas Aguda e distribuição espacial dos triatomíneos de importância epidemiológica, Brasil 2012 a 2016</a></p>
<p><a href="/images/pdf/2015/agosto/03/2014-020..pdf">Vol. 46 - Nº 21 - 2015 - Doença de Chagas aguda no Brasil: série histórica de 2000 a 2013</a></p>
<p><a href="/images/pdf/2014/julho/23/BE-2012-43--2--pag-1-a-5-Chagas.pdf">Vol. 43 - Nº 2 - 2012 - Investigação de pseudo-surto da doença de Chagas aguda em Altamira/Pará, julho de 2010</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n04-surtos-dchagas-aguda-pa-completo.pdf">Vol. 8 - Nº 4 - 2008 - Investigação de dois surtos de doença de chagas aguda na região do arquipélago do Marajó, Estado do Pará, Brasil, em 2007</a></p>
<p><a href="/images/pdf/2014/julho/17/Ano03-n04-reuniao-aval-dchagas-mg-completo.pdf">Vol. 3 - Nº 4 - 2003 - II Reunião anual de avaliação do programa de controle da doença de chagas. Uberaba, MG, 22 a 24 de outubro de 2003</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Doença Diarreica Aguda</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/fevereiro/21/Boletim-epidemiologico-SVS-08.pdf" target="_blank">BE Vol. 51 Nº 8 - Vacinação para o viajante e população em geral; orientações para prevenção de doenças transmissíveis e acidentes por animais peçonhentos; vigilância das IST; uso de álcool: alerta para o período do carnaval</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n16-surto-diarreia-aguda-go-completo.pdf">Vol. 8 - Nº 16 - 2008 - Surto de doença diarreica aguda em Anápolis, Goiás</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n14-surto-diarreia-aguda-msdf-completo.pdf">Vol. 8 - Nº 14 - 2008 - Investigação de surto de doença diarreica aguda no Edifício-Sede do Ministério da Saúde, Distrito Federal</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n11-surto-rotavirus-cob-vacinal-ms-completo.pdf">Vol 8 - Nº 11 - 2008 - Surto de doença diarreica aguda por rotavírus e levantamento de cobertura de vacina oral de rotavírus humano, Corumbá-MS, 2007</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n08-surto-diarreia-aguda-df-completo.pdf">Vol. 8 - Nº 8 - 2008 - Investigação de surto de doença diarreica aguda em evento científico, Brasília-DF, 2007</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n02-surto-deteccao-rotavirus-to-completo.pdf">Vol. 8 - Nº 2 - 2008 - Investigação de surto de doença diarreica aguda com detecção de rotavírus do Tocantins -TO, Julho a agosto de 2006</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano07-n04-surto-gastroenterite-viral-ba-completo.pdf">Vol. 7 - Nº 4 - 2007 - Investigação de surto de gastroenterite em Salvador-BA</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n08-surto-rotavirus-ac-completo.pdf">Vol. 6 - Nº 8 - 2006 - Surto de gastroenterite por rotavírus no Município de Rio Branco-AC</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n01-surto-rotavirus-creche-df-completo.pdf">Vol. 6 - Nº 1 - 2006 - Investigação de surto de gastroenterite associada a rotavírus em creche de Brasília-DF, no ano de 2005</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano04-n08-sdiarreia-vcholerae-pe-completo.pdf">Vol. 4 - Nº 8 - 2004 - Investigação de surto de diarreia aguda com isolamento de <em>V.cholerae </em>o1 Ogawa toxigênico no município de São Bento do Una - PE</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano04-n03-surto-gast-astro-noro-virus-rj.pdf.pdf">Vol. 4 - Nº 3 - 2004 - Investigação de surto de gastroenterite associado com astrovírus sorotipo 1, Itatiaia, RJ - Agosto de 2002</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Doenças febris de etiologia desconhecida</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2014/julho/18/Ano06-n10-surto-susp-histoplasmose-df-completo.pdf">Vol. 6 - Nº 10 - 2006 - Surto de doença febril de etiologia desconhecida em adolescentes que visitaram a gruta do Taboril em Unaí/MG - Stembro de 2004</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Doença meningocócica</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2019/fevereiro/01/2018-038.pdf">Vol. 50&nbsp; Nº 03, 2019: Meningites bacterianas não especificadas, Brasil, 2007 a 2016</a></p>
<p><a href="/images/pdf/2016/julho/29/2016-015---DM.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 29 - 2016 - Situação epidemiológica da doença meningocócica, Brasil, 2007-2013</a></p>
<p><a href="/images/pdf/2015/fevereiro/23/boletim-epi-n3-dez2010-.pdf">Vol. 10 - Nº 3 - 2010 - Surto de doença meningocócica no município de Itaguaí-RJ, 2009</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n17-surto-dmeningococica-go-completo.pdf">Vol. 8 - Nº 17 - 2008 - Surto de doença meningocócica no Município de Rio Verde-GO. Brasil</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Doença negligenciadas</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2018/novembro/19/2018-032.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 49, 2018 - Situação Epidemiológica e estratégias de prevenção, controle e eliminação das Doenças Tropicais Negligenciadas no Brasil, 1995 a 2016</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Doenças crônicas não transmissíveis</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/janeiro/16/Boletim-epidemiologico-SVS-40-v2.pdf">BE Vol. 50 Nº 40 - Panorama da vigilância de doenças crônicas não transmissíveis no Brasil e Fortalecimento das ações de prevenção, controle e eliminação da malária e ações de vigilância da leishmaniose visceral e doença de Chagas</a></p>
<p><a href="/images/pdf/2020/janeiro/03/Boletim-epidemiologico-SVS-40.pdf">BE Vol. 50 Nº 40 - Panorama da vigilância de doenças crônicas não transmissíveis no Brasil</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--12----Vigitel.pdf" target="_blank" rel="noopener noreferrer">Vol. 44 - Nº 12 - 2013 - Prevalência de fatores de risco e de proteção: população adulta brasileira - Vigitel 2011</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Doenças transmitidas por água e alimentos</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/fevereiro/21/Boletim-epidemiologico-SVS-08.pdf" target="_blank">BE Vol. 51 Nº 8 - Vacinação para o viajante e população em geral; orientações para prevenção de doenças transmissíveis e acidentes por animais peçonhentos; vigilância das IST; uso de álcool: alerta para o período do carnaval</a></p>
<p><a href="/images/pdf/2020/fevereiro/05/Boletim-epidemiologico-SVS-05.pdf">BE Vol. 51 Nº 5 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 03; Síndrome Congênita associada à infecção pelo vírus Zika: monitoramento do ano de 2019 e a qualidade da água para consumo humano no Rio de Janeiro.</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n16-surto-diarreia-aguda-go-completo.pdf" target="_blank">Vol. 8 - Nº 16 - 2008 - Surto de doença diarreica aguda em Anápolis, Goiás</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n14-surto-diarreia-aguda-msdf-completo.pdf" target="_blank">Vol. 8 - Nº 14 - 2008 - Investigação de surto de doença diarreica aguda no Edifício-Sede do Ministério da Saúde, Distrito Federal</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n11-surto-rotavirus-cob-vacinal-ms-completo.pdf" target="_blank">Vol. 8 - Nº 11 - 2008 - Surto de doença diarreica aguda por rotavírus e levantamento de cobertura de vacina oral de rotavírus humano, Corumbá-MS, 2007</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n08-surto-diarreia-aguda-df-completo.pdf" target="_blank">Vol. 8 - Nº 8 - 2008 - Investigação de surto de doença diarreica aguda em evento científico, Brasília-DF, 2007</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n06-hep-a-caract-casos-sinan-br-completo.pdf" target="_blank">Vol. 8 - Nº 6 - 2008 - Hepatite viral tipo A - caracterização dos casos notificados pelo Sistema Nacional de Vigilância no Brasil, 2006</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n04-surtos-dchagas-aguda-pa-completo.pdf" target="_blank">Vol. 8 - Nº 4 - 2008 - Investigação de dois surtos de doença de Chagas aguda na região do arquipélago do Marajó, Estado do Pará, Brasil, em 2007</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n03-intox-alim-tapioca-ce-completo.pdf" target="_blank">Vol. 8 - Nº 3 - 2008 - Investigação de surto de intoxicação alimentar por ingestão de tapioca em Sobral-CE, 2007</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n02-surto-deteccao-rotavirus-to-completo.pdf" target="_blank">Vol. 8 - Nº 2 - 2008 - Investigação de surto de doença diarreica aguda com detecção de rotavírus do Tocantins-TO, julho a agosto de 2006</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano07-n08-toxoplasmose-adquirida-go-completo.pdf" target="_blank">Vol. 7 - Nº 8 - 2007 - Surto de toxoplasmose adquirida, Anápolis-GO, fevereiro de 2006</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano07-n04-surto-gastroenterite-viral-ba-completo.pdf" target="_blank">Vol. 7 - Nº 4 - 2007 - Investigação de surto de gastroenterite em Salvador-BA</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n08-surto-rotavirus-ac-completo.pdf" target="_blank">Vol. 6 - Nº 8 - 2006 - Surto de gastroenterite por rotavírus no Município de Rio Branco-AC</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n03-surto-familiar-toxoplasmose-rs-completo.pdf" target="_blank">Vol. 6 - Nº 3 - 2006 - Surto intrafamiliar de toxoplasmose, Santa Vitória do Palmar-RS, julho de 2005</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n01-surto-rotavirus-creche-df-completo.pdf" target="_blank">Vol. 6 - Nº 1 - 2006 - Investigação de surto de gastroenterite associada a rotavírus em creche de Brasília-DF, no ano de 2005</a></p>
<p><a href="/images/pdf/2014/julho/15/Ano05-n07-surto-hep-a-mg-completo.pdf" target="_blank">Vol. 5 - Nº 7 - 2005 - Surto de hepatite viral tipo A (HVA), Minas Gerais, 2004</a></p>
<p><a href="/images/pdf/2014/julho/15/Ano05-n06-ve-dta-brasil-completo.pdf" target="_blank">Vol. 5 - Nº 6 - 2005 - Vigilância epidemiológica das doenças transmitidas por alimentos no Brasil, 1999 - 2004</a></p>
<p><a href="/images/pdf/2014/julho/15/Ano05-n05-fasciola-hepatica-am-completo.pdf" target="_blank">Vol. 5 - Nº 5 - 2005 - Detecção de casos humanos de fascíola hepática no Estado do Amazonas</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano04-n08-sdiarreia-vcholerae-pe-completo.pdf" target="_blank">Vol. 4 - Nº 8 - 2004 - Investigação de surto de diarreia aguda com isolamento de <em>V.cholerae</em> o1 Ogawa toxigênico no município de São Bento do Una - PE</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano04-n03-surto-gast-astro-noro-virus-rj.pdf.pdf" target="_blank">Vol. 4 - Nº 3 - 2004 - Investigação de surto de gastroenterite associado com astrovírus sorotipo 1, Itatiaia, RJ - Agosto de 2002</a></p>
<p><a href="/images/pdf/2014/julho/17/Ano03-n02-surto-rotavirus-es.pdf" target="_blank">Vol. 3 - Nº 2 - 2003 - Surto de rotavírus em Ecoporanga-ES, agosto de 2002</a></p>
<p><a href="/images/pdf/2014/julho/17/Ano03-n02-surto-hep-a-indigenas-mt.pdf%20(1).pdf" target="_blank">Vol. 3 - Nº 2 - 2003 – Surto de hepatite viral do tipo A em população indígena. Aldeia Kukuruzinho. Mato Grosso, 2001</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Epizootias</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2014/julho/16/Ano04-n07-obitos-humanos-epizootia-rn-completo.pdf" target="_blank">Vol. 4 - Nº 7 - 2004 - Investigação de óbitos humanos de causa desconhecida ocorridos no Rio Grande do Norte e de Epizootia em saguis (Callitherix jaccus) no parque das Dunas, Rio Grande do Norte, 2004</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Eventos de massa</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2019/novembro/07/Boletim-epidemiologico-SVS-30.pdf">BE Vol. 50 Nº 30, 2019: &nbsp;Sarampo (SE 29 a 40 de 2019); violências contra mulheres (2011-2018)&nbsp; e CIOCS - Copa do Mundo FIFA Sub-17</a></p>
<p><a href="/images/pdf/2014/maio/29/BE-2014-45--8----Copa-do-Brasil.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 8 - 2014 - Preparação e resposta da vigilância em saúde para a Copa do Mundo da FIFA Brasil 2014</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Esquistossomose mansoni</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2018/novembro/19/2018-032.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 49, 2018 - Situação Epidemiológica e estratégias de prevenção, controle e eliminação das Doenças Tropicais Negligenciadas no Brasil, 1995 a 2016</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Febre Amarela</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/April/13/Boletim-epidemiologico-SVS-15.pdf" target="_blank">BE Vol. 51 Nº 15 - Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes Aegypti (dengue, chikungunya e zika), Semanas Epidemiológicas 1 a 14 de 2020, e da febre amarela (2019/2020)</a></p>
<p><a href="/images/pdf/2020/April/03/Boletim-epidemiologico-SVS-14.pdf">BE Vol. 51 Nº 14 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 13, 2020; febre amarela; diferenças regionais no consumo de alimentos entre adolescentes e adultos e leishmaniose visceral: Experiência do IEC</a></p>
<p><a href="/images/pdf/2020/marco/27/Boletim-epidemiologico-SVS-13.pdf" target="_blank">BE Vol.51 Nº 13 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 12, 2020; febre amarela; análise da anomalias congênitas no Brasil; abastecimento de água para consumo humano em 2019 e medicamentos antileishmania no âmbito do SUS</a></p>
<p><a href="/images/pdf/2020/marco/30/Boletim-epidemiologico-SVS-12.pdf">BE Vol. 51 Nº 12 - Monitoramento dos casos de arboviroses (dengue, chikungunya e zika), SE 1 a 11, 2020; febre amarela 2019/2020; sarampo: SE 1 a 10, 2020; ações de enfrentamento da tuberculose em imigrantes e Síndrome congênita associada à infecção pelo vírus Zika: SE 10, 2020</a></p>
<p><a href="/images/pdf/2020/marco/05/Boletim-epidemiologico-SVS-09--.pdf" target="_blank">BE Vol. 51 Nº 9 - Vigilância epidemiológica do sarampo no Brasil 2020: Semana Epidemiológica 01; situação epidemiológica da febre amarela 2019/2020; importância dos códigos garbage para a magnitude da mortalidade por causas externas no Brasil; tracoma e acidentes ofídicos no Brasil, 2018</a></p>
<p><a href="/images/pdf/2020/April/08/Boletim-epidemiologico-SVS-06-v2.pdf" target="_blank" rel="noopener">BE Vol. 51 Nº 6 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 04; sarampo no Brasil 2019: janeiro a dezembro e situação da febre amarela: monitoramento 2019/2020</a></p>
<p><a href="/images/pdf/2020/janeiro/03/Boletim-epidemiologico-SVS-40.pdf">BE Vol. 51 Nº 1 - </a><a href="/images/pdf/2020/janeiro/15/Boletim-epidemiologico-SVS-01.pdf" target="_blank">Situação epidemiológica, fluxograma classificação de risco e manejo clínico da febre amarela; Evento de monitoramento internacional: China – Pneumonia de etiologia desconhecida; Violência contra mulheres e meninas do campo, da floresta e das águas</a></p>
<p><a href="/images/pdf/2020/janeiro/15/Boletim-epidemiologico-SVS-01.pdf" target="_blank"></a><a href="/images/PDF/2019/dezembro/27/Boletim-epidemiologico-SVS-39-FINAL.PDF" target="_blank">BE Vol. 50 Nº 39 – Sarampo (SE 39 a 50 de 2019); manejo de capivaras em áreas com casos de febre maculosa brasileira; Verão seguro; Situação Epidemiológica da Febre Amarela no monitoramento 2018/2019 e perspectivas para o período sazonal 2019/2020 no Brasil</a></p>
<p><a href="/images/pdf/2019/dezembro/23/Boletim-epidemiologico-SVS-38-2-interativo.pdf" target="_blank">BE Vol. 50 Nº 38 – Influenza (SE 1 a 49 de 2019); ampliação do diagnóstico da toxoplasmose congênita, monitoramento de dengue, chikungunya e Zika, até a SE 49; atualizações das indicações da vacina Febre Amarela e Campanha de Vacinação contra o HPV</a></p>
<p><a href="/images/pdf/2017/setembro/06/2017_027.pdf">Vol. 48 - Nº 28 - 2017 - Emergência epidemiológica de febre amarela no Brasil, no período de dezembro de 2016 a julho de 2017</a></p>
<p><a href="/images/pdf/2015/outubro/19/2015-032---FA-ok.pdf">Vol. 46 - Nº 29 - 2015 - Reem</a><a href="/images/pdf/2015/outubro/19/2015-032---FA-ok.pdf" target="_blank" rel="noopener noreferrer">ergência da Febre Amarela Silvestre no Brasil, 2014/2015: situação epidemiológica e a importância da vacinação preventiva e da vigilância intensificada no período sazonal</a></p>
<p><a href="/images/pdf/2015/agosto/07/2015-008---FA.pdf">Vol. 46 - Nº 23 - 2015 - </a><a href="/images/pdf/2015/agosto/07/2015-008---FA.pdf" target="_blank" rel="noopener noreferrer">Vigilância Epidemiológica da Febre Amarela e a importância da preparação para o monitoramento do período sazonal, Brasil, 2014-2015</a></p>
<p><a href="/images/pdf/2014/maio/27/BE-V45-n---07-FebreAmarela.pdf">Vol. 45 - Nº 7 - 2014 - F</a><a href="/images/pdf/2014/maio/27/BE-V45-n---07-FebreAmarela.pdf" target="_blank" rel="noopener noreferrer">ebre Amarela: Aspectos epidemiológicos da febre amarela silvestre e a vigilância intensificada durante período de monitoramento, Brasil, 2012/2013</a></p>
<p><a href="/images/pdf/2014/julho/17/Ano03-n06-epidemia-fa-mg-completo.pdf">Vol. 3 - Nº 6 - 2003 - Epid</a><a href="/images/pdf/2014/julho/17/Ano03-n06-epidemia-fa-mg-completo.pdf" target="_blank" rel="noopener noreferrer">emia de febre amarela silvestre na região da Bacia do Rio Doce - Minas Gerais. Dezembro de 2002 a Março de 2003</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Febre de Chikungunya</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/April/16/Boletim-epidemiologico-SVS-16.pdf" target="_blank">BE Vol. 51 Nº 16 -&nbsp;Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes Aegypti (dengue, chikungunya e zika), Semanas Epidemiológicas 1 a 15, 2020</a></p>
<p><a href="/images/pdf/2020/April/13/Boletim-epidemiologico-SVS-15.pdf" target="_blank">BE Vol. 51 Nº 15 - Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes Aegypti (dengue, chikungunya e zika), Semanas Epidemiológicas 1 a 14 de 2020, e da febre amarela (2019/2020)</a></p>
<p><a href="/images/pdf/2020/April/03/Boletim-epidemiologico-SVS-14.pdf">BE Vol. 51 Nº 14 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 13, 2020; febre amarela; diferenças regionais no consumo de alimentos entre adolescentes e adultos e leishmaniose visceral: Experiência do IEC</a></p>
<p><a href="/images/pdf/2020/marco/27/Boletim-epidemiologico-SVS-13.pdf" target="_blank">BE Vol.51 Nº 13 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 12, 2020; febre amarela; análise da anomalias congênitas no Brasil; abastecimento de água para consumo humano em 2019 e medicamentos antileishmania no âmbito do SUS</a><a href="/images/pdf/2020/marco/30/Boletim-epidemiologico-SVS-12.pdf"></a></p>
<p><a href="/images/pdf/2020/marco/30/Boletim-epidemiologico-SVS-12.pdf">BE Vol. 51 Nº 12 - Monitoramento dos casos de arboviroses (dengue, chikungunya e zika), SE 1 a 11, 2020; febre amarela 2019/2020; sarampo: SE 1 a 10, 2020; ações de enfrentamento da tuberculose em imigrantes e Síndrome congênita associada à infecção pelo vírus Zika: SE 10, 2020</a></p>
<p><a href="/images/pdf/2020/marco/30/Boletim-epidemiologico-SVS11.pdf" target="_blank">BE Vol. 51 Nº 11 - Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e zika), SE 1 a 10, 2020; Vigilância da infecção latente pelo Mycobacterium tuberculosis; Doença de Chagas - Protocolos clínicos de seguimento pós-tratamento em grupos especiais</a><a href="/images/pdf/2020/marco/21/Boletim_epidemiologico_SVS_11.pdf" target="_blank"><br></a></p>
<p><a href="/images/pdf/2020/marco/06/Boletim-epidemiologico-SVS-10.pdf" target="_blank">BE Vol. 51 Nº10 Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika), Semanas Epidemiológicas 1 a 7, 2020.</a></p>
<p><a href="/images/pdf/2020/fevereiro/19/Boletim-epidemiologico-SVS-07.pdf" target="_blank">BE Vol. 51 Nº 7 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 05; síndrome congênita associada à infecção pelo vírus Zika até a SE 05 e mobilização nacional para enfrentamento da hanseníase</a></p>
<p><a href="/images/pdf/2020/April/08/Boletim-epidemiologico-SVS-06-v2.pdf" target="_blank" rel="noopener">BE Vol. 51 Nº 6 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 04; sarampo no Brasil 2019: janeiro a dezembro e situação da febre amarela: monitoramento 2019/2020</a></p>
<p><a href="/images/pdf/2020/fevereiro/05/Boletim-epidemiologico-SVS-05.pdf">BE Vol. 51 Nº 5 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 03; Síndrome Congênita associada à infecção pelo vírus Zika: monitoramento do ano de 2019 e a qualidade da água para consumo humano no Rio de Janeiro.</a></p>
<p><a href="/images/pdf/2020/janeiro/20/Boletim-epidemiologico-SVS-02-1-.pdf" target="_blank">BE Vol. 51 Nº 2 – Monitoramento de dengue, chikungunya e Zika, até a SE 52 de 2019; Campanha de vacinação contra o sarampo</a></p>
<p><a href="/images/pdf/2019/dezembro/23/Boletim-epidemiologico-SVS-38-2-interativo.pdf" target="_blank">BE Vol. 50 Nº 38 – Influenza (SE 1 a 49 de 2019); ampliação do diagnóstico da toxoplasmose congênita, monitoramento de dengue, chikungunya e Zika, até a SE 49; atualizações das indicações da vacina Febre Amarela e Campanha de Vacinação contra o HPV</a></p>
<p><a href="/images/pdf/2019/dezembro/16/Boletim-epidemiologico-SVS-37-interativo-final.pdf">BE Vol. 50 Nº 37 – Sarampo (SE 36 a 47) vigilância epidemiológica, apuração final da campanha, análise da mortalidade de pedestres no Brasil (2010 a 2017), monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (SE 1 a 47)</a></p>
<p><a href="https://portalarquivos2.saude.gov.br/images/pdf/2019/novembro/13/Boletim-epidemiologico-SVS-34-V1.pdf" target="_blank" rel="noopener">BE Vol. 50 Nº 34 - Monitoramento das manchas de óleo no Nordeste; sarampo (Orientações para 2ª etapa da vacinação) e arboviroses (SE 1 a 44)</a></p>
<p><a href="/images/pdf/2019/novembro/07/Boletim-epidemiologico-SVS-33-7nov19.pdf" target="_blank">BE Vol. 50 Nº 33, Sarampo (SE 32 a 43); vigilância laboratorial do sarampo; informações sobre vacinação contra o sarampo; situação dos imunobiológicos; Programa Vida no Trânsito e monitoramento das manchas de óleo no Nordeste</a></p>
<p><a href="/images/pdf/2019/outubro/31/Boletim-epidemiologico-SVS-32.pdf" target="_blank">BE Vol. 50 Nº 32, 2019: Vigilância em Saúde: monitoramento das manchas de óleo no litoral do Nordeste ; arboviroses (SE 1 a 41 de 2019) e consumo de bebidas alcoólicas entre escolares no Brasil: uma análise da Pesquisa Nacional de Saúde do Escolar, 2015</a></p>
<p><a href="http://saude.gov.br/images/pdf/2019/outubro/24/Boletim-epidemiologico-SVS-31.pdf">BE Vol. 50 Nº 31, 2019: Sarampo (SE 30 a 42 de 2019);&nbsp; arboviroses (SE 1 a 40 de 2019) e óbitos por lesões de trânsito (2000 a 2017)</a></p>
<p><a href="/images/pdf/2019/outubro/10/Boletim-epidemiologico-SVS-29.pdf" target="_blank">BE Vol. 50 Nº 29, 2019: Cobertura nacional da vacina tríplice viral: primeira dose para crianças com até 1 ano, de 2015 a 2019</a><a href="/images/pdf/2019/outubro/04/BE-multitematico-n28.pdf">&nbsp;arboviroses urbanas transmitidas pelo Aedes Aegypti (SE 1 a 38 de 2019)</a></p>
<p><a href="/images/pdf/2019/outubro/04/BE-multitematico-n28.pdf" target="_blank">BE Vol. 50 Nº 28, 2019: sarampo (SE 28 a 39 de 2019); febre do Mayaro e febre do Oropouche; acidentes escorpiônicos; arboviroses urbanas transmitidas pelo Aedes Aegypti (SE 1 a 36 de 2019)</a></p>
<p><a href="/images/pdf/2019/setembro/11/BE-arbovirose-22.pdf">BE Vol. 50&nbsp; Nº&nbsp; 22, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 34 de 2019</a></p>
<p><a href="/images/pdf/2019/abril/30/2019-013-Monitoramento-dos-casos-de-arboviroses-urbanas-transmitidas-pelo-Aedes-publicacao.pdf">BE Vol. 50&nbsp; Nº&nbsp; 13, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 12 de 2019</a><a href="http://portalarquivos2.saude.gov.br/images/pdf/2019/abril/01/2018-066.pdf" target="_blank" rel="noopener noreferrer"></a></p>
<p><a href="/images/pdf/2019/marco/25/2019-013-Monitoramento-dos-casos-de-arboviroses-publicacao-25-03-2019.pdf">BE Vol. 50&nbsp; Nº&nbsp; 10, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 11 de 2019</a></p>
<p><a href="/images/pdf/2019/marco/21/2019-006.pdf">BE Vol. 50&nbsp; Nº&nbsp; 07, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 7 de 2019</a></p>
<p><a href="/images/pdf/2019/marco/21/2019-005.pdf">BE Vol. 50&nbsp; Nº&nbsp; 06, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 6 de 2019</a></p>
<p><a href="/images/pdf/2019/fevereiro/26/2019-004-Dengue-SE-5-publica----o-18-02-2019.pdf">Vol. 50 Nº 05, 2019: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 5 de 2019</a></p>
<p><a href="/images/pdf/2019/janeiro/28/2019-002.pdf" target="_blank" rel="noopener noreferrer">Vol. 50&nbsp; Nº 04, 2019: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica SE 52&nbsp; 2018 </a></p>
<p><a href="/images/pdf/2019/janeiro/28/2018-068.pdf" target="_blank" rel="noopener noreferrer">Vol. 50&nbsp; Nº 01, 2019: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica SE 50 2018</a></p>
<p><a href="/images/pdf/2019/janeiro/02/2018-067.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 Nº 59, 2018: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 49 de 2018</a></p>
<p><a href="/images/pdf/2018/dezembro/18/2018-064.pdf">Vol. 49 Nº 57, 2018: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 48 de 2018</a></p>
<p><a href="/images/pdf/2019/janeiro/14/BE-Vol. 49 – Nº 56, 2018- dengue chikungunya zika_aed_10jan19.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 Nº 56, 2018: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 47 de 2018</a></p>
<p><a href="/images/pdf/2019/janeiro/14/BE-Vol. 49 - Nº 55, 2018- dengue chikungunya zika_aed_10jan19.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 Nº 55, 2018: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 46 de 2018</a></p>
<p><a href="/images/pdf/2019/janeiro/14/BE-Vol. 49 - Nº 53, 2018 - dengue chikungunya zika_aed_10jan19.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 53, 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 45&nbsp;de 2018</a></p>
<p><a href="/images/pdf/2018/novembro/23/Publicacao-2018-57-SE44.pdf">Vol. 49 - Nº 51, 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 44&nbsp;de 2018</a></p>
<p><a href="/images/pdf/2018/novembro/13/2018-056.pdf">Vol. 49 Nº 48, 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 43&nbsp;de 2018</a></p>
<p><a href="/images/pdf/2018/novembro/05/2018-051-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-doen--a-aguda-pelo-v--rus-Zika-at---a-Semana-Epidemiol--gica-42-de-2018-publica--ao.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 44 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 42 de 2018 </a></p>
<p><a href="/images/pdf/2018/outubro/31/BE-2018-050-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-doen--a-aguda-pelo-v--rus-Zika-at---a-Semana-Epidemiol--gica-41-de-2018-ajustado.pdf" target="_blank" rel="noopener noreferrer">Vol. 49&nbsp; Nº&nbsp; 43, 2018: Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 41 de 2018</a></p>
<p><a href="/images/pdf/2018/outubro/26/2018-049.pdf">Vol. 49&nbsp; Nº&nbsp; 42, 2018: Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 38&nbsp;de 2018</a></p>
<p><a href="/images/pdf/2018/outubro/31/Vol. 49 Nº 40.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 40 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 36 de 2018</a></p>
<p><a href="/images/pdf/2018/outubro/31/Vol. 49 Nº 39.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 39 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 34 - 2018</a></p>
<p><a href="/images/pdf/2018/agosto/31/2018-043.pdf">Vol. 49 - Nº 36- 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 32 -2018</a></p>
<p><a href="/images/pdf/2018/outubro/31/Vol. 49 Nº 35.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 35 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 32 -2018</a></p>
<p><a href="/images/pdf/2018/agosto/21/Publicacao-BE-2018-SE-30.pdf">Vol. 49 -&nbsp; Nº&nbsp; 34- 2018 - &nbsp;Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 30 - 2018</a></p>
<p><a href="http://portalarquivos2.saude.gov.br/images/pdf/2018/agosto/07/2018_040.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 33- 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 28 - 2018</a></p>
<p><a href="/images/pdf/2018/julho/12/Monitoramento-dos-casos-ate-a-Semana-Epidemiologica-25-de-2018.pdf">Vol. 49 -&nbsp; Nº&nbsp; 32 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 25 de 2018</a></p>
<p><a href="/images/pdf/2018/julho/06/Monitoramento-dos-casos-de-dengue-23-018.pdf">Vol. 49 -&nbsp; Nº&nbsp; 31 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 23 de 2018</a></p>
<p><a href="/images/pdf/2018/julho/06/Monitoramento-dos-casos-de-dengue-21-de-2018.pdf">Vol. 49 -&nbsp; Nº&nbsp; 30 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 21 de 2018</a></p>
<p><a href="/images/pdf/2018/junho/25/2018-023.pdf">Vol. 49 -&nbsp; Nº&nbsp; 28- 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 19_2018</a></p>
<p><a href="/images/pdf/2018/junho/11/2018-022.pdf">Vol. 49 -&nbsp; Nº&nbsp; 24- 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 17 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/07/2018-018.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 -&nbsp; Nº&nbsp; 20 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 15 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/02/2018-017-Publicacao.pdf">Vol. 49 -&nbsp; Nº&nbsp; 18 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 14 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/02/2018-015-Publicacao.pdf">Vol. 49 -&nbsp; Nº&nbsp; 17 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 13 de 2018</a></p>
<p><a href="/images/pdf/2018/abril/27/BE-2018-14-SE-12.pdf">Vol. 49 - Nº 16- 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 12 de 2018</a></p>
<p><a href="/images/pdf/2018/abril/19/BE-2018-13-SE-11-Publicacao.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 15 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 11, de 2018</a></p>
<p><a href="/images/pdf/2018/abril/06/2018-012.pdf">Vol. 49 - Nº 14 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 10, de 2018</a></p>
<p><a href="/images/pdf/2018/abril/06/2018-011.pdf">Vol. 49 - Nº 13 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 09, de 2018</a></p>
<p><a href="/images/pdf/2018/marco/29/2018-010.pdf">Vol. 49 -&nbsp; Nº&nbsp; 12- 2018 -&nbsp;Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 8 de 2018</a></p>
<p><a href="/images/pdf/2018/marco/06/2018-008-Publicacao.pdf">Vol. 49 - Nº 9 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 07 de 2018</a></p>
<p><a href="/images/pdf/2018/marco/06/2018-007-Publicacao.pdf">Vol. 49 - Nº 8 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 06 de 2018</a></p>
<p><a href="/images/pdf/2018/fevereiro/20/2018-007.pdf">Vol. 49 - Nº 7 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 5 de 2018</a></p>
<p><a href="/images/pdf/2018/janeiro/23/Boletim-2018-001-Dengue.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 2 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 52 de 2017</a></p>
<p><a href="/images/pdf/2018/janeiro/23/Boletim-2017-051-Dengue.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 1 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 51 de 2017</a></p>
<p><a href="/images/pdf/2018/janeiro/10/2017-046-Publicacao.pdf">Vol. 48 Nº 45 -2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 50, 2017</a></p>
<p><a href="/images/pdf/2017/dezembro/26/2017-044.pdf">Vol. 48 - Nº 44 - 2017 - “Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 49 de 2017</a></p>
<p><a href="/images/pdf/2017/dezembro/26/BE-2017-043-Boletim.pdf">Vol. 48 - Nº 43 - 2017 - “Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 48 de 2017</a></p>
<p><a href="/images/pdf/2017/dezembro/21/BE-2017-042-Boletim.pdf">Vol. 48 - Nº 42 - 2017 - “Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 47 de 2017</a></p>
<p><a href="/images/pdf/2017/dezembro/08/2017-040.pdf">Vol. 48&nbsp; Nº 39 -2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 46 </a></p>
<p><a href="/images/PDF/2017/novembro/28/2017_039.pdf">Vol. 48&nbsp;- Nº 38 -2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 45, 2017</a></p>
<p><a href="/images/pdf/2017/novembro/13/2017-037.pdf">Vol. 48 - Nº 35 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 43 de 2017</a><br> <a href="/images/pdf/2017/novembro/07/2017-035-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica-41.pdf"><br> Vol. 48 - Nº 34 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 41 de 2017</a></p>
<p><a href="/images/pdf/2017/outubro/18/boletim-n32.pdf">Vol. 48 - Nº 32 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 39, 2017</a><br> <a href="/images/pdf/2017/outubro/18/BE-Vol-48-n-31-2017-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-ate-a-Semana-Epidemiologica-37.pdf"><br> Vol. 48 - Nº 31 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 37, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/setembro/15/2017-028-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica-35.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 29 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 35, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/agosto/29/2017-026-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica-33-de-2017.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 27 - 2017 - “Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 33, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/agosto/23/2017_024-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 26 - 2017 - “Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 31, 2017”</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/julho/25/Boletim-2017_020-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-Zika.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 20 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 25, 2017</a></p>
<p><a href="/images/pdf/2017/maio/25/Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 16 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 19, 2017</a></p>
<p><a href="/images/pdf/2017/maio/05/Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 14 - 2017 -Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 15, 2017</a></p>
<p><a href="/images/pdf/2017/abril/06/2017-010-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiol--gica-12--2017.pdf">Vol. 48 - Nº 11 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 12, 2017</a></p>
<p><a href="/images/pdf/2017/marco/31/Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-10-%202017.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 9 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 10, 2017</a></p>
<p><a href="/images/pdf/2017/marco/28/2017_006-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica-7.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 7 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 7, 2017</a></p>
<p><a href="/images/pdf/2017/fevereiro/17/Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-febre-pelo-v--rus-Semana-Epidemiologica-4-2017.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 5 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 4, 2017</a></p>
<p><a href="/images/pdf/2017/fevereiro/05/2017_002-Dengue%20SE52_corrigido.pdf">Vol. 48 - Nº 3 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 52, 2016</a></p>
<p><a href="/images/pdf/2017/janeiro/12/2017_001 - Dengue SE51_publicacao.pdf">Vol. 48 - Nº 2 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 51, 2016</a></p>
<p><a href="/images/pdf/2016/dezembro/20/2016-033---Dengue-SE49-publicacao.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 38 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 49, 2016</a></p>
<p><a href="/images/pdf/2016/outubro/18/2016-029-Dengue-publicacao-n-34.pdf" target="_blank" rel="noopener noreferrer">Vol 47 - Nº 34 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 37, 2016</a></p>
<p><a href="/images/pdf/2016/agosto/10/2016-026--2-..pdf">Vol. 47 - Nº 31 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 27, 2016</a></p>
<p><a href="/images/pdf/2016/julho/15/2016-boletim-epi-n28-dengue-chik-zika-se23.pdf">Vol. 47 - Nº 28 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 23, 2016</a></p>
<p><a href="/images/pdf/2016/junho/30/2016-021.pdf">Vol. 47 - Nº 27 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 21, 2016</a></p>
<p><a href="/images/pdf/2016/junho/30/2016-020--1-.pdf">Vol. 47 - Nº 26 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 20, 2016</a></p>
<p><a href="/images/pdf/2016/junho/17/2016-019.pdf">Vol. 47 - Nº 25 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 19, 2016</a></p>
<p><a href="/images/pdf/2016/junho/15/2016-018---Dengue-SE18-substitui----o.pdf">Vol. 47 - Nº 24 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 18, 2016</a></p>
<p><a href="/images/pdf/2016/junho/14/boletim-epid-2016-n23-dcz-se17.pdf">Vol. 47 - Nº 23 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 17, 2016</a></p>
<p><a href="/images/pdf/2016/maio/17/2016-016---Dengue-SE16-publica----o.pdf">Vol. 47 - Nº 20 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 16, 2016</a></p>
<p><a href="/images/pdf/2016/junho/10/2016-014---Dengue-SE13-substitui----o.pdf">Vol. 47 - Nº 18 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 13, 2016</a></p>
<p><a href="/images/pdf/2016/abril/11/2015-013---Dengue-SE9.pdf">Vol. 47 - Nº 16 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 9, 2016</a></p>
<p><a href="/images/pdf/2016/abril/05/2016-012---Relatorio-reuniao-especialistas-Aedes-publica----o.pdf">Vol. 47 - Nº 15 - 2016 - Relatório da Reunião Internacional para Implementação de Alternativas para o Controle do Aedes aegypti no Brasil</a></p>
<p><a href="/images/pdf/2016/marco/29/2016-011-boletim-epi-dcz-se8.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 14 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 8, 2016 </a></p>
<p><a href="/images/pdf/2016/marco/23/2016-008---DengueSE8-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 11 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 7, 2016 </a></p>
<p><a href="/images/pdf/2016/marco/23/2016-007---Dengue-SE-6-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 10 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 6, 2016</a></p>
<p><a href="/images/pdf/2016/fevereiro/29/2016-006---Dengue-SE5-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 8 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 5, 2016</a></p>
<p><a href="/images/pdf/2016/fevereiro/29/2016-005---Dengue-SE4-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 7 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 4, 2016</a></p>
<p><a href="/images/pdf/2016/fevereiro/04/2016-004---Dengue-SE3.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 6 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 3, 2016</a></p>
<p><a href="/images/pdf/2016/fevereiro/04/2016-003-Dengue-SE2.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 5 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 2, 2016</a></p>
<p><a href="/images/pdf/2016/janeiro/15/svs2016-be003-dengue-se52.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 3 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 52, 2015</a></p>
<p><a href="/images/pdf/2016/janeiro/15/svs2016-be002-dengue-se51.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 2 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 51, 2015</a></p>
<p><a href="/images/pdf/2016/janeiro/07/2015-svs-be-pncd-se48.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 44 - 2015 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 48, 2015</a></p>
<p><a href="/images/pdf/2015/dezembro/11/svs-be-2015-047-dengue-se47-final.pdf">Vol. 46 - Nº 42 - 2015 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 47, 2015</a></p>
<p><a href="/images/pdf/2015/novembro/26/2015-dengue-SE45.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 36 - 2015 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 45, 2015</a></p>
<p><a href="/images/pdf/2015/novembro/24/2015-040.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 35 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 43, 2015 </a></p>
<p><a href="/images/pdf/2015/outubro/29/2015-035---Dengue-SE-40---29.10.15.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 33 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 40, 2015 </a></p>
<p><a href="/images/pdf/2015/outubro/22/2015-034---Dengue-SE-38.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 32 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 38, 2015 </a></p>
<p><a href="/images/pdf/2015/outubro/15/svsbe-denchikzik-v46-n31.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 31 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 36, 2015</a></p>
<p><a href="/images/pdf/2015/outubro/01/2015-030-bol--2-.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 28 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 34, 2015</a></p>
<p><a href="/images/pdf/2015/outubro/01/2015-030-bol--1-.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 27 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 32, 2015</a></p>
<p><a href="/images/pdf/2015/setembro/03/2015-029---SE-30.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 24 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 30, 2015</a></p>
<p><a href="/images/pdf/2015/agosto/07/2015-028---Dengue-SE-28.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 22 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 28, 2015</a></p>
<p><a href="/images/pdf/2015/julho/27/2015-027---Dengue-SE-26--.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 20 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 26, 2015</a></p>
<p><a href="/images/pdf/2015/julho/20/20150716-Boletim-dengue-SE24-2.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 19 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 24, 2015</a></p>
<p><a href="/images/pdf/2015/julho/06/2015-024---Dengue-SE-23.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 18 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 23, 2015</a></p>
<p><a href="/images/pdf/2015/julho/02/Monitoramento-dos-casos-de-dengue-e-febre-de-chikungunya-22-NUCOM.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 16 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 21, 2015</a></p>
<p><a href="/images/pdf/2015/junho/30/Monitoramento-dos-casos-de-dengue-e-febre-de-chikungunya-20.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 15 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 20, 2015</a></p>
<p><a href="/images/pdf/2015/maio/04/2015-016---Boletim-Dengue-SE15-2015.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 14 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 15, 2015</a></p>
<p><a href="/images/pdf/2015/maio/04/2015-015---Boletim-Dengue-SE14-2015.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 13 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 14, 2015</a></p>
<p><a href="/images/pdf/2015/abril/29/Boletim-Dengue-SE12-2015.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 12 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 12, 2015</a></p>
<p><a href="/images/pdf/2015/abril/29/2015-013---Boletim-Dengue-SE11-2015.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 11 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a semana epidemiológica 11, 2015</a></p>
<p><a href="/images/pdf/2015/marco/13/Boletim-Dengue-SE09-2015.final.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 8 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 9, 2015</a></p>
<p><a href="/images/pdf/2015/fevereiro/25/2015-006---Boletim-Dengue-SE06-2015.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 7 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 6, 2015</a></p>
<p><a href="/images/pdf/2015/fevereiro/25/2015-005---Boletim-Dengue-SE05-2015.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 6 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 5, 2015</a></p>
<p><a href="/images/pdf/2015/fevereiro/11/Boletim-Dengue-SE04-2015..pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 5 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica 4, 2015</a></p>
<p><a href="/images/pdf/2015/janeiro/19/2015-002---BE-at---SE-53.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 3 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 53 de 2014</a></p>
<p><a href="/images/pdf/2015/janeiro/22/BE-dengue-SE-52-20-01-15.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 2 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 52 de 2014</a></p>
<p><a href="/images/pdf/2015/janeiro/19/2014-042-ok-50.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 1 - 2015 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 50 de 2014</a></p>
<p><a href="/images/pdf/2015/janeiro/02/2014-039---Dengue-SE-47.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 30 - 2014 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 47 de 2014</a></p>
<p><a href="/images/pdf/2014/dezembro/11/2014-038---BE-Dengue-at---SE-46.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 29 - 2014 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 46 de 2014</a></p>
<p><a href="/images/pdf/2014/dezembro/11/2014-037---Dengue-SE-45.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 28 - 2014 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 45 de 2014</a></p>
<p><a href="/images/pdf/2014/dezembro/10/2014-036---BE-SE-44.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 27 - 2014 - Monitoramento dos casos de dengue e febre de chikungunya até a Semana Epidemiológica (SE) 44 de 2014</a></p>
<p><a href="/images/pdf/2014/outubro/27/BE-2014-45--26----Dengue-SE41-e-CHIKV-SE42.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 26 - 2014 - Monitoramento dos casos de dengue até a semana epidemiológica (SE) 41 e febre de chikungunya até a 42 de 2014</a></p>
<p><a href="/images/pdf/2014/outubro/22/BE-2014-45--25----Dengue--SE40--e-CHIKV--SE41-.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 25 - 2014 - Monitoramento dos casos de dengue até a semana epidemiológica (SE) 40 e febre de chikungunya até a SE 41 de 2014</a></p>
<p><a href="/images/pdf/2014/outubro/15/Boletim-Dengue-SE39-2.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 24 - 2014 - Monitoramento dos casos de dengue até a semana epidemiológica (SE) 39 e febre de chikungunya até a SE 40 de 2014</a></p>
<p><a href="/images/pdf/2014/outubro/03/BE-2014-45--23----Dengue--SE38--e-CHIKV--SE39-.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 23 - 2014 - Monitoramento dos casos de dengue até a Semana Epidemiológica (SE) 38 e febre de chikungunya até a SE 39 de 2014</a></p>
<p><a href="/images/pdf/2014/setembro/30/BE-2014-45--22----Dengue--SE37--e-CHIKV--SE38-.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 22 - 2014 - Monitoramento dos casos de dengue até a Semana Epidemiológica (SE) 37 e febre de chikungunya até a SE 38 de 2014</a></p>
<p><a href="/images/pdf/2014/setembro/30/BE-2014-45--21----Dengue--SE36--e-CHIKV--SE37-.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 21 - 2014 - Monitoramento dos casos de dengue até a Semana Epidemiológica (SE) 36 e febre de chikungunya até a SE 37 de 2014</a></p>
<p><a href="/images/pdf/2014/setembro/30/BE-2014-45--20----Dengue--SE35--e-CHIKV--SE36-.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 20 - 2014 - Monitoramento dos casos de dengue Semana Epidemiológica (SE) 35 e febre de chikungunya SE 36 de 2014</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Febre do Mayaro</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2019/outubro/04/BE-multitematico-n28.pdf" target="_blank">BE Vol. 50 Nº 28, 2019: sarampo (SE 28 a 39 de 2019); febre do Mayaro e febre do Oropouche; acidentes escorpiônicos; arboviroses urbanas transmitidas pelo Aedes Aegypti (SE 1 a 36 de 2019)</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Febre do Nilo Ocidental</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2014/julho/16/Ano04-n05-2---inq-sorol-aves-virus-fno-rs-completo.pdf">Vol. 4 - Nº 5 - 2004 - Segundo inquérito sorológico em aves migratórias e residentes do Parque Nacional Lagoa do Peixe/RS para detecção do vírus do Nilo Ocidental e outros vírus</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano04-n02-inq-sorol-aves-virus-fno-rn-completo.pdf">Vol. 4 - Nº 2 - 2004 - Inquérito sorológico em aves migratórias e residentes de Galinhos/RN para detecção do vírus da febre do Nilo ocidental e outros vírus</a></p>
<p><a href="/images/pdf/2014/julho/17/Ano03-n01-1---inq-sorol-aves-virus-fno-rs.pdf">Vol. 3 - Nº 1 - 2003 – Primeiro inquérito sorológico em aves migratórias e nativas do Parque Nacional Lagoa do Peixe/RS para detecção do vírus do Nilo Ocidental</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Febre do Oropouche</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2019/outubro/04/BE-multitematico-n28.pdf" target="_blank">BE Vol. 50 Nº 28, 2019: sarampo (SE 28 a 39 de 2019); febre do Mayaro e febre do Oropouche; acidentes escorpiônicos; arboviroses urbanas transmitidas pelo Aedes Aegypti (SE 1 a 36 de 2019)</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Febre maculosa brasileira e outras rickettsioses</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/PDF/2019/dezembro/27/Boletim-epidemiologico-SVS-39-FINAL.PDF" target="_blank">BE Vol. 50 Nº 39 – Sarampo (SE 39 a 50 de 2019); manejo de capivaras em áreas com casos de febre maculosa brasileira; Verão seguro; Situação Epidemiológica da Febre Amarela no monitoramento 2018/2019 e perspectivas para o período sazonal 2019/2020 no Brasil</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n04-surto-rickettsiose-rj-completo.pdf">Vol. 6 - Nº 4 - 2006 - Surto de rickettsiose em Petrópolis-RJ, 2005</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Filariose linfática</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2018/novembro/19/2018-032.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 49, 2018 - Situação Epidemiológica e estratégias de prevenção, controle e eliminação das Doenças Tropicais Negligenciadas no Brasil, 1995 a 2016</a></p>
<p><a href="/images/pdf/2016/marco/11/2015-039---Filariose-vers--ofinal.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 9 - 2016 - Situação epidemiológica da filariose linfática no Brasil</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Febre Hemorrágica Brasileira</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/janeiro/21/Boletim-epidemiologico-SVS-03.pdf" target="_blank">BE Vol. 51 Nº 3 -&nbsp;Identificação de um caso de febre&nbsp;hemorrágica brasileira no estado de&nbsp;São Paulo, janeiro de 2020</a><a href="/images/pdf/2018/novembro/19/2018-032.pdf" target="_blank" rel="noopener noreferrer"><br></a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Geo-helmintíases</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2018/novembro/19/2018-032.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 49, 2018 - Situação Epidemiológica e estratégias de prevenção, controle e eliminação das Doenças Tropicais Negligenciadas no Brasil, 1995 a 2016</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Hanseníase</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/fevereiro/19/Boletim-epidemiologico-SVS-07.pdf" target="_blank">BE Vol. 51 Nº 7 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 05; síndrome congênita associada à infecção pelo vírus Zika até a SE 05 e mobilização nacional para enfrentamento da hanseníase</a></p>
<p><a href="/images/pdf/2020/janeiro/31/Boletim-hanseniase-2020-web.pdf" target="_blank">BE Número Especial - Hanseníase</a></p>
<p><a href="/images/pdf/2018/novembro/19/2018-032.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 49, 2018 - Situação Epidemiológica e estratégias de prevenção, controle e eliminação das Doenças Tropicais Negligenciadas no Brasil, 1995 a 2016</a></p>
<p><a href="/images/pdf/2018/janeiro/31/2018-004-Hanseniase-publicacao.pdf">Vol. 49 - Nº 4 - 2018 - Caracterização da situação epidemiológica da hanseníase e diferenças por sexo, Brasil, 2012-2016</a></p>
<p><a href="/images/pdf/2016/maio/12/2015-038---Campanha-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 21 - 2016 - </a><a href="/images/pdf/2016/maio/12/2015-038---Campanha-publica----o.pdf" target="_blank" rel="noopener noreferrer">“Hanseníase, Verminoses e Tracoma tem cura: a experiência de uma campanha integrada”</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--11----Hanseniase.pdf" target="_blank" rel="noopener noreferrer">Vol. 44 - Nº 11 - 2013 - Situação epidemiológica da hanseníase no Brasil - análise de indicadores selecionados na última década e desafios para eliminação</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Hantavirose</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2014/julho/23/BE-2012-43--2--pag-6-a-9-Hantavirose.pdf" target="_blank">Vol. 43 - n. 2 - 2012 - Investigação de casos e aumento da letalidade por hantavirose, Distrito Federal, 2010</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano07-n06-scp-hantavirus-br-completo.pdf" target="_blank">Vol. 7 - Nº 6 - 2007 - Síndrome cardiopulmonar por hantavírus no Brasil em 2006: estudo clínico epidemiológico</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n05-epidemiol-hantavirose-br-completo.pdf" target="_blank">Vol. 6 – Nº 5 - 2006 - Epidemiologia da hantavírus e da síndrome cardiopulmonar por hantavírus no Brasil no ano de 2005</a></p>
<p><a href="/images/pdf/2014/julho/15/Ano05-n03-asp-epid-patog-hant-br-completo.pdf" target="_blank">Vol. 5 - Nº 3 - 2005 - Aspectos epidemiológicos da infecção e da patogenicidade por hantavírus no Brasil (2004)</a></p>
<p><a href="/images/pdf/2014/julho/15/Ano05-n01-surto-scp-hantavirus-df-completo.pdf" target="_blank">Vol. 5 - Nº 1 - 2005 - Surto de síndrome cardiopulmonar por hantavírus no Distrito Federal e Goiás - Maio a Setembro de 2004</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Hepatites virais</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/fevereiro/27/boletim-hepatites-2019-c.pdf" target="_blank">Boletim Epidemiológico Hepatites Virais 2019 - Secretaria de Vigilância em Saúde | Ministério da Saúde Volume 50 | jul. 2019</a></p>
<p><a href="/images/pdf/2015/fevereiro/23/boletim-epi-n8-dez2010-.pdf" target="_blank">Vol. 10 - Nº 8 - 2010 - Surto de hepatite viral tipo A no município de Águas Lindas de Goiás, em 2009</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n06-hep-a-caract-casos-sinan-br-completo.pdf" target="_blank">Vol. 8 - Nº 6 - 2008 - Hepatite viral tipo A - caracterização dos casos notificados pelo Sistema Nacional de Vigilância no Brasil, 2006</a></p>
<p><a href="/images/pdf/2014/julho/15/Ano05-n07-surto-hep-a-mg-completo.pdf" target="_blank">Vol. 5 - Nº 7 - 2005 - Surto de hepatite viral tipo A (HVA), Minas Gerais, 2004</a></p>
<p><a href="/images/pdf/2014/julho/17/Ano03-n02-surto-hep-a-indigenas-mt.pdf%20(1).pdf" target="_blank">Vol. 3 - Nº 2 - 2003 – Surto de hepatite viral do tipo A em população indígena. Aldeia Kukuruzinho. Mato Grosso, 2001</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Imunizações </a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/fevereiro/21/Boletim-epidemiologico-SVS-08.pdf" target="_blank">BE Vol. 51 Nº 8 - Vacinação para o viajante e população em geral; orientações para prevenção de doenças transmissíveis e acidentes por animais peçonhentos; vigilância das IST; uso de álcool: alerta para o período do carnaval</a></p>
<p><a href="/images/pdf/2020/April/08/Boletim-epidemiologico-SVS-06-v2.pdf" target="_blank" rel="noopener">BE Vol. 51 Nº 6 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 04; sarampo no Brasil 2019: janeiro a dezembro e situação da febre amarela: monitoramento 2019/2020</a></p>
<p><a href="/images/pdf/2020/janeiro/20/Boletim-epidemiologico-SVS-02-1-.pdf" target="_blank">BE Vol. 51 Nº 2 – Monitoramento de dengue, chikungunya e Zika, até a SE 52 de 2019; Campanha de vacinação contra o sarampo</a></p>
<p><a href="/images/pdf/2019/dezembro/23/Boletim-epidemiologico-SVS-38-2-interativo.pdf" target="_blank">BE Vol. 50 Nº 38 – Influenza (SE 1 a 49 de 2019); ampliação do diagnóstico da toxoplasmose congênita, monitoramento de dengue, chikungunya e Zika, até a SE 49; atualizações das indicações da vacina Febre Amarela e Campanha de Vacinação contra o HPV</a></p>
<p><a href="/images/pdf/2019/novembro/20/Boletim-epidemiologico-SVS-35.pdf" target="_blank">BE Vol. 50 Nº 35 - Sarampo (SE 34 a 45), vigilância laboratorial do sarampo, vacinação contra o sarampo, orientações para a 2ª etapa da vacinação contra o sarampo, raiva, recomendações para uso dos imunobiológicos, malária (2017 a 2019) e monitoramento das manchas de óleo</a></p>
<p><a href="https://portalarquivos2.saude.gov.br/images/pdf/2019/novembro/07/Boletim-epidemiologico-SVS-33-7nov19.pdf" target="_blank" rel="noopener">BE Vol. 50 Nº 33, Sarampo (SE 32 a 43); vigilância laboratorial do sarampo; informações sobre vacinação contra o sarampo; situação dos imunobiológicos; Programa Vida no Trânsito e monitoramento das manchas de óleo no Nordeste</a></p>
<p><a href="/images/pdf/2019/outubro/10/Boletim-epidemiologico-SVS-29.pdf" target="_blank">BE Vol. 50 Nº 29, 2019: Cobertura nacional da vacina tríplice viral: primeira dose para crianças com até 1 ano, de 2015 a 2019</a><a href="/images/pdf/2019/outubro/04/BE-multitematico-n28.pdf">&nbsp;arboviroses urbanas transmitidas pelo Aedes Aegypti (SE 1 a 38 de 2019)</a></p>
<p><a href="/images/pdf/2018/novembro/21/2018-028.pdf">Vol. 49 - Nº 52, 2018 - Perfil epidemiológico dos casos notificados dos erros de imunização, Brasil, 2016</a></p>
<p><a href="http://intranet.saude.gov/images/pdf/2016/outubro/22/boletim_2015_042.pdf" target="_blank" rel="noopener noreferrer">Volume 47 - Nº 36 - 2016 - Situação epidemiológica da vigilância de eventos adversos pós vacinação nas Secretarias Estaduais de Saúde, Brasil, 2013</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Influenza, Síndrome Gripal e Síndrome Respiratória Aguda Grave (SRAG)</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2019/dezembro/23/Boletim-epidemiologico-SVS-38-2-interativo.pdf" target="_blank">BE Vol. 50 Nº 38 – Influenza (SE 1 a 49 de 2019); ampliação do diagnóstico da toxoplasmose congênita, monitoramento de dengue, chikungunya e Zika, até a SE 49; atualizações das indicações da vacina Febre Amarela e Campanha de Vacinação contra o HPV</a></p>
<p><a href="/images/pdf/2019/setembro/06/BE-21-influenza-04set19.pdf" target="_blank">BE Vol. 50 Nº 21, 2019: Monitoramento dos casos de influenza no Brasil, Semanas Epidemiológicas 1 a 32 de 2019</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/agosto/21/2017_023-Situacao-Epidemiologica-da-Influenza-no-Brasil-ate-a-Semana-Epidemiologica-32-de-2017.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 25 - 2017 - Situação Epidemiológica da Influenza no Brasil, até a Semana Epidemiológica 32 de 2017</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--15----Influenza--SE29-.pdf" target="_blank">Vol. 44 - Nº 15 - 2013 - Influenza: monitoramento até a semana epidemiológica 29 de 2013</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--14----Influenza.pdf" target="_blank">Vol. 44 - Nº 14 - 2013 - Análise da situação epidemiológica da influenza. Brasil, 2012</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--7----Influenza--SE19-.pdf" target="_blank">Vol. 44 - Nº 7 - 2013 - Influenza: monitoramento da semana epidemiológica 19</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--6----Influenza--SE18-.pdf" target="_blank">Vol. 44 - Nº 6 - 2013 - Influenza: monitoramento da semana epidemiológica 18</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--5----Influenza--SE17--1.pdf" target="_blank">Vol. 44 - Nº 5 - 2013 - Influenza: monitoramento da semana epidemiológica 17</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--4----Influenza--SE16-.pdf" target="_blank">Vol. 44 - Nº 4 - 2013 - Influenza: monitoramento da semana epidemiológica 16</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--3----Influenza--SE15-.pdf" target="_blank">Vol. 44 - Nº 3 - 2013 - Influenza: monitoramento das primeiras 15 semanas epidemiológicas de 2013</a></p>
<p><a href="/images/pdf/2014/julho/17/Ano03-n02-surto-hep-a-indigenas-mt.pdf%20(1).pdf" target="_blank">Vol. 43 - Nº 3 - 2012 - Investigação de casos de Síndrome Respiratória Aguda Grave no Município de Coruripe-Alagoas, outubro de 2010</a></p>
<p><a href="/images/pdf/2014/julho/23/BE-2012-43--3--pag-11-a-16-Sindrome-gripal.pdf" target="_blank">Vol. 43 - Nº 3 - 2012 - Investigação de Surto de Síndrome Gripal em Populações Indígenas, Altamira – Pará, abril – setembro de 2010</a></p>
<p><a href="/images/pdf/2014/julho/23/BE-2012-43-Mar--o---Protocolo-Influenza.pdf" target="_blank">Vol. 43 - 2012 - Protocolo de tratamento de influenza - 2011</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano04-n01-desafio-influenza-br-completo.pdf" target="_blank">Vol. 4 - Nº 1 - 2004 - O desafio da influenza: epidemiologia e organização da vigilância no Brasil</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Intoxicação exógena</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2018/dezembro/26/2018-027.pdf">Vol. 49 Nº 58, 2018: Intoxicações exógenas relacionadas ao trabalho no Brasil de 2007 a 2016</a></p>
<p><a href="/images/pdf/2018/dezembro/18/2018-031.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 50, 2018 - Casos notificados de intoxicações exógenas relacionados ao glifosato no Brasil, no período de 2007 a 2016</a></p>
<p><a href="/images/pdf/2014/julho/23/BE-2012-43--4---pag-7-a-10-Agente-ambiental.pdf" target="_blank">Vol. 43 - Nº 4 - 2012 - Avaliação de Conhecimentos, Atitudes e Práticas dos Agentes de Saúde Ambiental e Controle de Endemias (Asace) do município do Recife em relação ao Diflubenzuron, Pernambuco, 2011</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n13-surto-intox-piretroides-go-completo.pdf" target="_blank">Vol. 8 - Nº 13 - 2008 - Surto de intoxicação por piretróides no Município de Porangatu-GO</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n07-susp-intox-pi-completo--1-.pdf" target="_blank">Vol. 6 - Nº 7 - 2006 - Investigação de surto suspeito de intoxicação exógena, Riveiro Gonçalves-PI, 2005</a></p>
<p><a href="/images/pdf/2014/julho/17/Ano03-n03-intoxicacao-radiol-bario-go.pdf.pdf" target="_blank">Vol. 3 - Nº 3 - 2003 - Investigação de surto de intoxicação pelo bário. Goiás, Junho de 2003</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Investigação epidemiológica de casos, surtos e epidemias</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/janeiro/03/Boletim-epidemiologico-SVS-40.pdf">BE Vol. 51 Nº 1 - </a><a href="/images/pdf/2020/janeiro/15/Boletim-epidemiologico-SVS-01.pdf" target="_blank">Situação epidemiológica, fluxograma classificação de risco e manejo clínico da febre amarela; Evento de monitoramento internacional: China – Pneumonia de etiologia desconhecida; Violência contra mulheres e meninas do campo, da floresta e das águas</a></p>
<p><a href="/images/pdf/2018/junho/11/2017-041-Tetano-publicacao.pdf">Vol. 49 -&nbsp; Nº&nbsp; 25- 2018 - Situação epidemiológica do tétano acidental, Brasil, 2007 a 2016</a></p>
<p><a href="/images/pdf/2014/junho/03/Boletim-Sarampo-02-12-13.pdf" target="_blank" rel="noopener noreferrer">Vol. 44 - Nº 16 - 2013 - Surto de sarampo em Pernambuco</a></p>
<p><a href="/images/pdf/2014/julho/23/BE-2012-43--3--pag-1-a-5-Coqueluche.pdf">Vol. 43 - Nº 3 - 2012 - Surto de Coqueluche em Santana do Mundaú, Alagoas, julho de 2010</a></p>
<p><a href="/images/pdf/2014/julho/23/BE-2012-43--3--pag-11-a-16-Sindrome-gripal.pdf">Vol. 43 - Nº 3 - 2012 - Investigação de Surto de Síndrome Gripal em Populações Indígenas, Altamira – Pará, abril – setembro de 2010</a></p>
<p><a href="/images/pdf/2014/julho/23/BE-2012-43--3--pag-17-a-20-Sarampo.pdf">Vol. 43 - Nº 3 - 2012 - Investigação de surto de sarampo no Rio Grande do Sul, agosto de 2010</a></p>
<p><a href="/images/pdf/2014/julho/23/BE-2012-43--3--pag-6-a-10-SRAG.pdf">Vol. 43 - Nº 3 - 2012 - Investigação de casos de Síndrome Respiratória Aguda Grave no Município de Coruripe-Alagoas, outubro de 2010</a></p>
<p><a href="/images/pdf/2014/julho/23/BE-2012-43--2---pag-10-a-11-Malaria.pdf">Vol. 43 - Nº 2 - 2012 - Surto de malária no assentamento Três Pontões, município de Nova Venécia, estado do Espírito Santo,2012</a></p>
<p><a href="/images/pdf/2015/fevereiro/23/boletim-epi-n9-dez2010-.pdf">Vol. 10 - Nº 9 - 2010 - Investigação do surto de leptospirose no município de Pacoti, Ceará, em 2009</a></p>
<p><a href="/images/pdf/2015/fevereiro/23/boletim-epi-n8-dez2010-.pdf">Vol. 10 - Nº 8 - 2010 - Surto de hepatite viral tipo A no município de Águas Lindas de Goiás, em 2009</a></p>
<p><a href="/images/pdf/2015/fevereiro/23/boletim-epi-n7-dez2010-.pdf">Vol. 10 - Nº 7 - 2010 - Investigação de surto de botulismo alimentar em Coruripe/AL, em abril de 2009</a></p>
<p><a href="/images/pdf/2015/fevereiro/23/boletim-epi-n6-dez2010-2-.pdf" target="_blank" rel="noopener noreferrer">Vol. 10 - Nº 6 - 2010 - Surto</a><a href="/images/pdf/2015/fevereiro/23/boletim-epi-n6-dez2010-2-.pdf" target="_blank" rel="noopener noreferrer"> de varicela em Várzea, na Paraíba, setembro, 2008</a></p>
<p><a href="/images/pdf/2015/fevereiro/23/boletim-epi-n5-17dez2010-.pdf" target="_blank" rel="noopener noreferrer">Vol. 10 - Nº 5 - 2010 - Surto de malá</a><a href="/images/pdf/2015/fevereiro/23/boletim-epi-n5-17dez2010-.pdf" target="_blank" rel="noopener noreferrer">ria no Distrito de Jaci-Paraná, municípios de Porto Velho - RO, em 2009</a></p>
<p><a href="/images/pdf/2015/fevereiro/23/boletim-epi-n4-18dez2010-.pdf">Vol. 10 - Nº 4 - 2010 - Investigação do surto de malária em Anajás– Pará</a></p>
<p><a href="/images/pdf/2015/fevereiro/23/boletim-epi-n3-dez2010-.pdf">Vol. 10 - Nº 3 - 2010 - Surto de doença meningocócica no município de Itaguaí-RJ, 2009</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n20-surto-leptospirose-ce-completo.pdf">Vol. 8 - Nº 20 - 2008 - Investigação de surto de leptospirose em Várzea Alegre-CE</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n19-surto-micob-cirurg-pr-completo.pdf">Vol. 8 - Nº 19 - 2008 - Surto de infecção hospitalar por microbactérias de crescimento rápido em procedimento de videocirurgia, Curitiba-PR</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n18-meningite-eosinofilica-es-completo.pdf">Vol. 8 - Nº 18 - 2008 - Investigação de casos de meningite eosinofilica causada pela infecção por Angiostrongylus cantonensis no Espírito Santo. Brasil</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n17-surto-dmeningococica-go-completo.pdf">Vol. 8 - Nº 17 - 2008 - Surto de doença meningocócica no Município de Rio Verde-GO. Brasil</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n17-surto-dmeningococica-go-completo.pdf">Vol. 8 - Nº 16 - 2008 - Surto de doença diarreica aguda em Anápolis, Goiás</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n15-surto-rubeola-pa-completo.pdf">Vol. 8 - Nº 15 - 2008 - Investigação de surto de rubéola em Ananindeua-PA, julho de 2008</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n14-surto-diarreia-aguda-msdf-completo.pdf">Vol. 8 - Nº 14 - 2008 - Investigação de surto de doença diarreica aguda no Edifício-Sede do Ministério da Saúde, Distrito Federal</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n13-surto-intox-piretroides-go-completo.pdf">Vol. 8 - Nº 13 - 2008 - Surto de intoxicação por piretróides no Município de Porangatu-GO</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n12-surto-brucelose-humana-to-completo.pdf">Vol. 8 - Nº 12 - 2008 - Investigação de casos de brucelose humana em Araguaína no Estado do Tocantins, Brasil, junho de 2008</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n11-surto-rotavirus-cob-vacinal-ms-completo.pdf">Vol. 8 - Nº 11 - 2008 - Surto de doença diarreica aguda por rotavírus e levantamento de cobertura de vacina oral de rotavírus humano, Corumbá-MS, 2007</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n10-infeccao-hosp-micob-pr-completo.pdf">Vol. 8 - n. 10 - 2008 - Surto de infecção hospitalar por microbactérias de crescimento rápido, Curitiba-PR, 2007</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n09-cob-vacinal-surto-rubeola-ce-completo.pdf">Vol. 8 - Nº 9 - 2008 - Levantamento rápido de cobertura vacinal para intervenção em um surto de rubéola em Horizonte-CE, agosto de 2007</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n11-surto-rotavirus-cob-vacinal-ms-completo.pdf">Vol. 8 - Nº 8 - 2008 - Investigação de surto de doença diarreica aguda em evento científico, Brasília-DF, 2007</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n07-surto-rubeola-ce-completo.pdf">Vol. 8 - Nº 7 - 2008 - Surto de rubéola em Fortaleza-CE, janeiro a junho de 2007</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n05-surto-febre-hem-dengue-pa-completo.pdf">Vol. 8 - Nº 5 - 2008 - Investigação de surto de febre hemorrágica da dengue, Redenção-PA, dezembro de 2007</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n04-surtos-dchagas-aguda-pa-completo.pdf">Vol. 8 - Nº 4 - 2008 - Investigação de dois surtos de doença de chagas aguda na região do arquipélago do Marajó, Estado do Pará, Brasil, em 2007</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n03-intox-alim-tapioca-ce-completo.pdf">Vol. 8 - Nº 3 - 2008 - Investigação de surto de intoxicação alimentar por ingestão de tapioca em Sobral-CE, 2007</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n02-surto-deteccao-rotavirus-to-completo.pdf">Vol. 8 - Nº 2 - 2008 - Investigação de surto de doença diarreica aguda com detecção de rotavírus do Tocatins-TO, Julho a agosto de 2006</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n01-obitos-criancas-aldeia-ro-completo.pdf">Vol. 8 - Nº 1 - 2008 - Investigação epidemiológica de óbitos de crianças indígenas da Aldeia Lage Velho, Etnia Pacaás Novos, Guajará-Mirim-RO, Fevereiro de 2007</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano07-n08-toxoplasmose-adquirida-go-completo.pdf">Vol. 7 - Nº 8 - 2007 - Surto de toxoplasmose adquirida, Anápolis-GO, Fevereiro de 2006</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano07-n05-endoftalmite-cirurg-oftam-rr-completo.pdf">Vol. 7 - Nº 5 - 2007 - Surto de endofiltamite aguda após cirurgias oftalmológicas em Boa Vista-RR</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano07-n04-surto-gastroenterite-viral-ba-completo.pdf">Vol. 7 - Nº 4 - 2007 - Investigação de surto de gastroenterite em Salvador-BA</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano07-n03-eventos-imunogl-humana-mg-completo.pdf">Vol. 7 - Nº 3 - 2007 - Investigação de eventos adversos à imunoglobulina humana intravenosa total no Estado de Minas Gerais, Brasil</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano07-n02-surto-rubeola-mg-completo.pdf">Vol. 7 - Nº 2 - 2007 - Investigação de surto de rubéola em Belo Horizonte, Estado de Minas Gerais</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano07-n01-surto-dengue-conh-atit-prat-to-completo.pdf">Vol. 7 - Nº 1 - 2007 - Investigação de surto de dengue em município do estado do Tocantins e avaliação de conhecimentos, atitudes e práticas da população em relação à dengue durante o surto em 2006</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n10-surto-susp-histoplasmose-df-completo.pdf">Vol. 6 - Nº 10 - 2006 - Surto de doença febril de etiologia desconhecida em adolescentes que visitaram a gruta do Taboril em Unaí/MG - Setembro de 2004</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n09-surto-malaria-df-completo.pdf">Vol. 6 - Nº 9 - 2006 - Surto de malária autóctone por Plasmodium vivax no Distrito Federal - Maio de 2005</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n08-surto-rotavirus-ac-completo.pdf">Vol. 6 - Nº 8 - 2006 - Surto de gastroenterite por rotavírus no Município de Rio Branco-AC</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n07-susp-intox-pi-completo--1-.pdf">Vol. 6 - Nº 7 - 2006 - Investigação de surto suspeito de intoxicação exógena, Ribeiro Gonçalves-PI, 2005</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n06-conj-aguda-adiaspiromicose-to-completo.pdf">Vol. 6 - Nº 6 - 2006 - Investigação epidemiológica do surto de doença ocular, Araguatins/TO, 2005</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n04-surto-rickettsiose-rj-completo.pdf">Vol. 6 - Nº 4 - 2006 - Surto de rickettsiose em Petrópolis-RJ, 2005</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n03-surto-familiar-toxoplasmose-rs-completo.pdf">Vol. 6 - Nº 3 - 2006 - Surto intrafamiliar de toxoplasmose, Santa Vitória do Palmar-RS, Julho de 2005</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n02-raiva-hum-morcegos-ma-completo.pdf">Vol. 6 - Nº 2 - 2006 - Raiva humana transmitida por morcegos no Alto Turi, MA, Julho-Setembro de 2005</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n01-surto-rotavirus-creche-df-completo.pdf">Vol. 6 - Nº 1 - 2006 - Investigação de surto de gastroenterite associada a rotavírus em creche de Brasília-DF, no ano de 2005</a></p>
<p><a href="/images/pdf/2014/julho/15/Ano05-n07-surto-hep-a-mg-completo.pdf">Vol. 5 - n. 7 - 2005 - Surto de hepatite viral tipo A (HVA), Minas Gerais, 2004</a></p>
<p><a href="/images/pdf/2014/julho/15/Ano05-n05-fasciola-hepatica-am-completo.pdf">Vol. 5 - Nº 5 - 2005 - Detecção de casos humanos de fascíola hepática no Estado do Amazonas</a></p>
<p><a href="/images/pdf/2014/julho/15/Ano05-n01-surto-scp-hantavirus-df-completo.pdf">Vol. 5 - Nº 1 - 2005 - Surto de síndrome cardiopulmonar por hantavírus no Distrito Federal e Goiás - Maio a Setembro de 2004</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano04-n08-sdiarreia-vcholerae-pe-completo.pdf">Vol. 4 - Nº 8 - 2004 - Investigação de surto de diarreia aguda com isolamento de <em>V. cholerae</em> o1 Ogawa toxigênico no município de São Bento do Una - PE</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano04-n06-surto-raiva-hum-morcegos-pa-completo.pdf">Vol. 4 - Nº 6 - 2004 - Surto de raiva humana transmitida por morcegos no Município de Portel-Pará, Março/Abril de 2004</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano04-n03-surto-gast-astro-noro-virus-rj.pdf.pdf">Vol. 4 - Nº 3 - 2004 - Investigação de surto de gastroenterite associado com astrovírus sorotipo 1, Itatiaia, RJ - Agosto de 2002</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano04-n03-endoftalmites-pos-cirug-rj.pdf.pdf">Vol. 4 - Nº 3 - 2004 - Investigação sobre a ocorrência de endofitalmites pós-cirúrgica de catarata, RJ, Junho de 2003</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano-04-n03-obitos-hemodialise-pe.pdf.pdf">Vol. 4 - Nº 3 - 2004 – Investigação de óbitos ocorridos em serviço de hemodiálise de Caruarú, Pernambuco - julho 2003</a></p>
<p><a href="/images/pdf/2014/julho/17/Ano03-n06-epidemia-fa-mg-completo.pdf">Vol. 3 - Nº 6 - 2003 - Epidemia de febre amarela silvestre na região da Bacia do Rio Doce - Minas Gerais. Dezembro de 2002 a Março de 2003<br> </a><a href="/images/pdf/2014/julho/17/Ano03-n03-reacoes-sacarato-hidroxido-ce.pdf.pdf"><br> Vol. 3 - Nº 3 - 2003 - Reações adversas depois da administração do sacarato de hidróxido de ferro III endovenoso em Unidades de Hemodiálise, Fortaleza-CE, 2002 - Resultados preliminares</a></p>
<p><a href="/images/pdf/2014/julho/17/Ano03-n03-intoxicacao-radiol-bario-go.pdf.pdf">Vol. 3 - Nº 3 - 2003 - Investigação de surto de intoxicação pelo bário. Goiás, Junho de 2003</a></p>
<p><a href="/images/pdf/2014/julho/17/Ano03-n02-surto-rotavirus-es.pdf">Vol. 3 - Nº 2 - 2003 - Surto de rotavírus em Ecoporanga-ES, agosto de 2002</a></p>
<p><a href="/images/pdf/2014/julho/17/Ano03-n02-surto-hep-a-indigenas-mt.pdf (1).pdf">Vol. 3 - Nº 1 - 2003 - Investigação de óbitos por meningite criptocócica com provável transmissão por fezes de quirópteros. Rondônia, 2002</a></p>
<p><a href="/images/pdf/2014/julho/17/Ano03-n02-surto-hep-a-indigenas-mt.pdf (1).pdf">Vol. 3 - Nº 1 - 2003 – Surto de hepatite viral do tipo A em população indígena. Aldeia Kukuruzinho. Mato Grosso, 2001</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Leishmanioses</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/April/03/Boletim-epidemiologico-SVS-14.pdf">BE Vol. 51 Nº 14 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 13, 2020; febre amarela; diferenças regionais no consumo de alimentos entre adolescentes e adultos e leishmaniose visceral: Experiência do IEC</a></p>
<p><a href="/images/pdf/2020/marco/27/Boletim-epidemiologico-SVS-13.pdf" target="_blank">BE Vol.51 Nº 13 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 12, 2020; febre amarela; análise da anomalias congênitas no Brasil; abastecimento de água para consumo humano em 2019 e medicamentos antileishmania no âmbito do SUS</a></p>
<p><a href="/images/pdf/2020/janeiro/16/Boletim-epidemiologico-SVS-40-v2.pdf">BE Vol. 50 Nº 40 - Panorama da vigilância de doenças crônicas não transmissíveis no Brasil e Fortalecimento das ações de prevenção, controle e eliminação da malária e ações de vigilância da leishmaniose visceral e doença de Chagas</a></p>
<p><a href="/images/pdf/2014/julho/17/Bol-Eletr-Ano03-05-2003-completo.pdf">Vol. 3 - Nº 5 - 2003- Reunião Nacional das Leishmanioses - Uberaba - MG, 22 a 24 de outubro de 2003</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Leptospirose</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2018/outubro/25/2018-033-Leptospirose-situa----o-epidemiol--gica-do-Brasil-no-per--odo-de-2007-a-2016-publica--ao.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº&nbsp; 41- 2018 - Leptospirose: Situação epidemiológica do Brasil no período de 2007 a 2016</a></p>
<p><a href="/images/pdf/2015/fevereiro/23/boletim-epi-n9-dez2010-.pdf" target="_blank" rel="noopener noreferrer">Vol. 10 - Nº 9 - 2010 - Investigação do surto de leptospirose no município de Pacoti, Ceará, em 2009</a>
 <a href="/images/pdf/2014/junho/25/Ano08-n20-surto-leptospirose-ce-completo.pdf" target="_blank" rel="noopener noreferrer">Vol. 8 - Nº 20 - 2008 - Investigação de surto de leptospirose em Várzea Alegre-CE</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Malária</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/janeiro/16/Boletim-epidemiologico-SVS-40-v2.pdf">BE Vol. 50 Nº 40 - Panorama da vigilância de doenças crônicas não transmissíveis no Brasil e Fortalecimento das ações de prevenção, controle e eliminação da malária e ações de vigilância da leishmaniose visceral e doença de Chagas</a></p>
<p><a href="/images/pdf/2019/novembro/20/Boletim-epidemiologico-SVS-35.pdf" target="_blank">BE Vol. 50 Nº 35 - Sarampo (SE 34 a 45), vigilância laboratorial do sarampo, vacinação contra o sarampo, orientações para a 2ª etapa da vacinação contra o sarampo, raiva, recomendações para uso dos imunobiológicos, malária (2017 a 2019) e monitoramento das manchas de óleo</a></p>
<p><a href="http://portalarquivos2.saude.gov.br/images/pdf/2019/abril/01/2018-066.pdf" target="_blank" rel="noopener noreferrer">BE Vol. 50&nbsp; Nº&nbsp; 12, 2019: Descrição do Processo de Monitoramento dos Testes de Diagnóstico Rápido de Malária, Brasil, 2014 a 2016</a></p>
<p><a href="/images/pdf/2015/dezembro/16/2015-003---Mal--ria.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 43 - 2015 - Situação epidemiológica da malária no Brasil, 2012 e 2013</a></p>
<p><a href="/images/pdf/2015/agosto/18/2015-009---Mal--ria-para-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 25 - 2015 - Malária: Monitoramento dos casos no Brasil em 2014</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--1----Mal--ria.pdf" target="_blank" rel="noopener noreferrer">Vol. 44 - Nº 01 - 2013 - Situação epidemiológica da malária no Brasil, 2000 a 2011</a></p>
<p><a href="/images/pdf/2014/julho/23/BE-2012-43--2---pag-10-a-11-Malaria.pdf">Vol. 43 - Nº 2 - 2012 - Surto de malária no assentamento Três Pontões, município de Nova Venécia, estado do Espírito Santo,2012</a></p>
<p><a href="/images/pdf/2015/fevereiro/23/boletim-epi-n5-17dez2010-.pdf">Vol. 10 - Nº 5 - 2010 - Surto de malária no Distrito de Jaci-Paraná, municípios de Porto Velho - RO, em 2009</a></p>
<p><a href="/images/pdf/2015/fevereiro/23/boletim-epi-n4-18dez2010-.pdf">Vol. 10 - Nº 4 - 2010 - Investigação do surto de malária em Anajás– Pará</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n09-surto-malaria-df-completo.pdf">Vol. 6 - Nº 9 - 2006 - Surto de malária autóctone por Plasmodium vivax no Distrito Federal - Maio de 2005</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Meningites</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2019/fevereiro/01/2018-038.pdf">Vol. 50&nbsp; Nº 03, 2019: Meningites bacterianas não especificadas, Brasil, 2007 a 2016</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n18-meningite-eosinofilica-es-completo.pdf">Vol. 8 - Nº 18 - 2008 - Investigação de casos de meningite eosinofílica causada pela infecção por <em>Angiostrongylus cantonensis</em> no Espírito Santo. Brasil</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n17-surto-dmeningococica-go-completo.pdf">Vol. 8 - Nº 17 - 2008 - Surto de doença meningocócica no Município de Rio Verde-GO. Brasil</a></p>
<p><a href="/images/pdf/2014/julho/17/Ano03-n01-obito-mening-criptocococica-ro.pdf">Vol. 3 - Nº 1 - 2003 - Investigação de óbitos por meningite criptocócica com provável transmissão por fezes de quirópteros. Rondônia, 2002</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Microcefalia e/ou alterações do Sistema Nervoso Central (SNC) associadas à infecção por vírus Zika</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/marco/30/Boletim-epidemiologico-SVS-12.pdf">BE Vol. 51 Nº 12 - Monitoramento dos casos de arboviroses (dengue, chikungunya e zika), SE 1 a 11, 2020; febre amarela 2019/2020; sarampo: SE 1 a 10, 2020; ações de enfrentamento da tuberculose em imigrantes e Síndrome congênita associada à infecção pelo vírus Zika: SE 10, 2020</a></p>
<p><a href="/images/pdf/2020/fevereiro/19/Boletim-epidemiologico-SVS-07.pdf" target="_blank">BE Vol. 51 Nº 7 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 05; síndrome congênita associada à infecção pelo vírus Zika até a SE 05 e mobilização nacional para enfrentamento da hanseníase</a></p>
<p><a href="/images/pdf/2020/fevereiro/05/Boletim-epidemiologico-SVS-05.pdf">BE Vol. 51 Nº 5 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 03; Síndrome Congênita associada à infecção pelo vírus Zika: monitoramento do ano de 2019 e a qualidade da água para consumo humano no Rio de Janeiro.</a></p>
<p><a href="/images/pdf/2019/dezembro/05/be-sindrome-congenita-vfinal.pdf">Número Especial -&nbsp;Síndrome congênita associada à infecção pelo vírus Zika - Situação Epidemiológica, ações desenvolvidas e desafios de 2015 a 2019</a></p>
<p><a href="/images/pdf/2019/marco/22/2019-001.pdf">BE Vol. 50&nbsp; Nº&nbsp; 08, 2019: Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 52 de 2018</a></p>
<p><a href="/images/pdf/2018/dezembro/14/2018-061.pdf">Vol. 49&nbsp; Nº&nbsp; 54, 2018: Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 45 de 2018</a></p>
<p><a href="/images/pdf/2018/novembro/07/2018-053-Monitoramento-integrado-Semana-Epidemiologica-40-publicacao.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 46 - 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 40 de 2018</a></p>
<p><a href="/images/pdf/2018/novembro/07/2018-052-Monitoramento-integrado-Semana-Epidemiologica-35-publicacao.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 45 - 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 35 de 2018</a></p>
<p><a href="/images/pdf/2018/setembro/11/2018-047.pdf">Vol. 49 -&nbsp; Nº&nbsp; 39- 2018, intitulado: “Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 30</a></p>
<p><a href="/images/pdf/2018/setembro/11/2018-046.pdf">Vol. 49 -&nbsp; Nº&nbsp; 38- 2018, intitulado: “Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 25</a></p>
<p><a href="http://portalarquivos2.saude.gov.br/images/pdf/2018/junho/29/Monitoramento-integrado-de-alteracoes-no-crescimento-e-desenvolvimento-relacionadas-a-infeccao-pelo-virus-Zika.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 -&nbsp; Nº&nbsp; 29 - 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 20 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/28/2018-023.pdf">Vol. 49 -&nbsp; Nº&nbsp; 23 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 16 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/28/2018-022.pdf">Vol. 49 -&nbsp; Nº&nbsp; 22- 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 15 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/28/BE-2018-021.pdf">Vol. 49 -&nbsp; Nº&nbsp; 21- 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 09 de 2018</a></p>
<p><a href="http://portalarquivos2.saude.gov.br/images/pdf/2018/maio/04/2018-016.pdf">Vol. 49 -&nbsp; Nº&nbsp; 19 - 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 05 de 2018</a></p>
<p><a href="/images/pdf/2018/fevereiro/20/2018-007.pdf">Vol. 49 - Nº 7 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 5 de 2018</a></p>
<p><a href="/images/pdf/2018/fevereiro/20/2018-003-Final.pdf">Vol. 49 - Nº 6 - 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 52 de 2017</a></p>
<p><a href="/images/pdf/2018/janeiro/30/2018-002.pdf">Vol. 49 -&nbsp; Nº&nbsp; 3- 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 48 de 2017</a></p>
<p><a href="/images/pdf/2017/dezembro/12/Boletim-2017-043-final.pdf">Vol. 48&nbsp;- Nº &nbsp;41 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 44</a></p>
<p><a href="/images/pdf/2017/novembro/16/2017-036.pdf">Vol.48 Nº 37 -2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 38/2017</a></p>
<p><a href="/images/pdf/2017/outubro/23/2017-034-Monitoramento-integrado-de-alteracoes-no-crescimento-e-desenvolvimento-relacionadas-a-infeccao-pelo-virus-Zika-e-outras-etiologias-infecciosas--ate-a-Semana-Epidemiologica-34.pdf">Vol.48&nbsp;&nbsp; Nº 33 -2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 34/2017</a><br> <a href="http://portalsaude.saude.gov.br/images/pdf/2017/agosto/16/BE-2017_024-Monitoramento-integrado-de-alteracoes-no-crescimento-e-desenvolvimento-relacionadas-a-infeccao-pelo-virus-Zika.pdf" target="_blank" rel="noopener noreferrer"><br> Vol. 48 - Nº 24 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 28/2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/agosto/04/BE_Integrado_2017_023_Final.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 23 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 26/2017</a>
 <a href="http://portalsaude.saude.gov.br/images/pdf/2017/agosto/04/BE_Integrado_2017_022_Final.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 22 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 24/2017</a>
 <a href="http://portalsaude.saude.gov.br/images/pdf/2017/agosto/04/BE_Integrado_2017_021_Final.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 21 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 22/2017</a>
 <a href="http://portalsaude.saude.gov.br/images/pdf/2017/julho/04/2017-017-Monitoramento-integrado-de-alteracoes-no-crescimento-e-desenvolvimento-relacionadas-a-infeccao-pelo-virus-Zika.pdf">Vol. 48 - Nº 19 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 20/2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/junho/01/BE-2017-015-Monitoramento-integrado-de-alteracoes-no-crescimento-e-desenvolvimento.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 17 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 18/2017</a></p>
<p><a href="/images/pdf/2017/maio/15/BE-2017-014-Monitoramento-integrado-de-alteracoes-no-crescimento-e-desenvolvimento-relacionadas-a-infeccao-pelo-virus-Zika-e-outras-etiologias-infecciosas--ate-a-Semana-Epidemiologica-16.pdf">Vol. 48 - Nº 15 - 2017 - “Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 16/2017"</a></p>
<p><a href="/images/pdf/2017/abril/27/Monitoramento-integrado-de-alteracoes-no-crescimento-e-desenvolvimento-relacionadas-a-infeccao-pelo-virus-Zika.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 13 - 2017 -Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 14/2017</a></p>
<p><a href="/images/pdf/2017/abril/10/2017_011%20Monitoramento%20integrado%20de%20alteracoes%20no%20crescimento%20e%20desenvolvimento%20relacionadas%20a%20infeccao%20pelo%20virus%20Zika%20e%20outras%20etiologias%20infecciosas%20ate%20a%20Semana%20Epidemiologica%2012_rev.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 12 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 12/2017</a></p>
<p class="default"><a href="/images/pdf/2017/abril/03/Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-10-2017.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 10 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 10, 2017</a></p>
<p><a href="/images/pdf/2017/fevereiro/27/2017_003.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 6 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, da Semana Epidemiológica 45/2015 até a Semana Epidemiológica 02/2017</a></p>
<p><a href="/images/pdf/2016/janeiro/11/2015-053-para-substituir-na-p--gina.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 01 - 2016 - Monitoramento dos casos de microcefalias no Brasil até a Semana Epidemiológica 51</a>
 <a href="/images/pdf/2015/dezembro/28/2015-boletim-microcefalia-se50-vol46-n46.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 46 - 2015 - Monitoramento dos casos de microcefalias no Brasil até a Semana Epidemiológica 50</a>
 <a href="/images/pdf/2015/dezembro/23/2015-050---microcefalia-SE-49---publica----o-23.12.15.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 45 - Monitoramento dos casos de microcefalias no Brasil até a Semana Epidemiológica 49</a>
 <a href="/images/pdf/2015/dezembro/11/svs-be-2015-048-microcefalia-se48-final2.pdf">Vol. 46 - Nº 41 - 2015 - Monitoramento dos casos de microcefalia no Brasil até a Semana Epidemiológica 48</a>
 <a href="/images/pdf/2015/dezembro/02/2015-046---Microcefalia----02.12.15.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 38 - 2015 - Monitoramento dos casos de microcefalias no Brasil, até a semana epidemiológica 47, 2015</a>
 <a href="/images/pdf/2015/novembro/30/Microcefalia-2-boletim.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 37 - 2015 - Monitoramento dos casos de microcefalias no Brasil, até a semana epidemiológica 46, 2015</a>
 <a href="/images/pdf/2015/novembro/19/Microcefalia-bol-final.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 34 - 2015 - Situação epidemiológica de ocorrência de microcefalias no Brasil, 2015</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Novo coronavírus (2019-nCoV)</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="https://portalarquivos2.saude.gov.br/images/pdf/2020/fevereiro/13/Boletim-epidemiologico-COEcorona-SVS-13fev20.pdf" target="_blank" rel="noopener">BE Coronavírus - 10/02/2020</a></p>
<p><a href="/images/pdf/2020/fevereiro/04/Boletim-epidemiologico-SVS-04fev20.pdf">BE Coronavírus - 03/02/2020</a></p>
<p><a style="text-align: left; color: #0066cc; text-transform: none; text-indent: 0px; letter-spacing: normal; font-size: 13px; font-style: normal; font-variant: normal; font-weight: 400; text-decoration: underline; word-spacing: 0px; white-space: normal; orphans: 2;" href="/images/pdf/2020/janeiro/23/Boletim_epidemiologico_SVS_04.pdf" target="_blank" rel="noopener">BE Vol. 51 Nº 4 - Novo coronavírus (2019-nCoV); e vigilância de violências e acidentes no Brasil: análise da cobertura da notificação compulsória de violência interpessoal/autoprovocada nos municípios brasileiros</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Oncocercose</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2018/novembro/19/2018-032.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 49, 2018 - Situação Epidemiológica e estratégias de prevenção, controle e eliminação das Doenças Tropicais Negligenciadas no Brasil, 1995 a 2016</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Raiva</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2019/novembro/20/Boletim-epidemiologico-SVS-35.pdf" target="_blank">BE Vol. 50 Nº 35 - Sarampo (SE 34 a 45), vigilância laboratorial do sarampo, vacinação contra o sarampo, orientações para a 2ª etapa da vacinação contra o sarampo, raiva, recomendações para uso dos imunobiológicos, malária (2017 a 2019) e monitoramento das manchas de óleo</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n02-raiva-hum-morcegos-ma-completo.pdf">Vol. 6 - Nº 2 - 2006 - Raiva humana transmitida por morcegos no Alto Turi, MA, Julho-Setembro de 2005<br> </a><br> <a href="/images/pdf/2014/julho/16/Ano04-n06-surto-raiva-hum-morcegos-pa-completo.pdf">Vol. 4 - Nº 6 - 2004 - Surto de raiva humana transmitida por morcegos no Município de Portel-Pará, Março/Abril de 2004</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Regulamento Sanitário Internacional</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2014/julho/15/Ano05-n02-novo-rsi-completo.pdf">Vol. 5 - Nº 2 - 2005 - O novo Regulamento Sanitário Internacional</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Rotavírus</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2014/julho/18/Ano06-n08-surto-rotavirus-ac-completo.pdf">Vol. 6 - Nº 8 - 2006 - Surto de gastroenterite por rotavírus no Município de Rio Branco-AC</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n01-surto-rotavirus-creche-df-completo.pdf">Vol. 6 - Nº 1 - 2006 - Investigação de surto de gastroenterite associada a rotavírus em creche de Brasília-DF, no ano de 2005</a></p>
<p><a href="/images/pdf/2014/julho/17/Ano03-n02-surto-rotavirus-es.pdf">Vol. 3 - Nº 2 - 2003 - Surto de rotavírus em Ecoporanga-ES, agosto de 2002</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Rubéola</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2014/junho/25/Ano08-n15-surto-rubeola-pa-completo.pdf">Vol. 8 – Nº 15 - 2008 - Investigação de surto de rubéola em Ananindeua-PA, julho de 2008</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n09-cob-vacinal-surto-rubeola-ce-completo.pdf">Vol. 8 - Nº 9 - 2008 - Levantamento rápido de cobertura vacinal para intervenção em um surto de rubéola em Horizonte-CE, agosto de 2007</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n07-surto-rubeola-ce-completo.pdf">Vol. 8 - Nº 7 - 2008 - Surto de rubéola em Fortaleza-CE, janeiro a junho de 2007</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano07-n02-surto-rubeola-mg-completo.pdf">Vol. 7 - Nº 2 - 2007 - Investigação de surto de rubéola em Belo Horizonte, Estado de Minas Gerais</a></p>
<p><a href="/images/pdf/2014/julho/15/Ano05-n04-evento-adv-vac-tv-pr-completo.pdf">Vol. 5 – Nº 4 - 2005 - Investigação de eventos adversos após a administração de vacina tríplice viral durante a campanha de seguimento contra sarampo em 2004</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Sarampo</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<!-- START: Articles Anywhere -->
<ul>
<li><a href="/images/pdf/2020/marco/23/Boletim-epidemiologico-SVS-12.pdf">BE Vol. 51 Nº 12 - Monitoramento dos casos de arboviroses (dengue, chikungunya e zika), SE 1 a 11, 2020; febre amarela 2019/2020; sarampo: SE 1 a 10, 2020; ações de enfrentamento da tuberculose em imigrantes e Síndrome congênita associada à infecção pelo vírus Zika: SE 10, 2020</a></li>
<li><a href="/images/pdf/2020/marco/04/Boletim-epidemiologico-SVS-09.pdf" target="_blank">BE Vol. 51 Nº 9 - Vigilância epidemiológica do sarampo no Brasil 2020: Semana Epidemiológica 01; situação epidemiológica da febre amarela 2019/2020; importância dos códigos garbage para a magnitude da mortalidade por causas externas no Brasil; tracoma e acidentes ofídicos no Brasil, 2018</a></li>
<li><a href="/images/pdf/2020/fevereiro/10/Boletim-epidemiologico-SVS-06.pdf">BE Vol. 51 Nº 6 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 04; sarampo no Brasil 2019: janeiro a dezembro e situação da febre amarela: monitoramento 2019/2020</a></li>
<li><a href="/images/pdf/2020/janeiro/20/Boletim-epidemiologico-SVS-02-1-.pdf" target="_blank">BE Vol. 51 Nº 2 – Monitoramento de dengue, chikungunya e Zika, até a SE 52 de 2019; Campanha de vacinação contra o sarampo</a></li>
<li><a href="http://portalarquivos2.saude.gov.br/images/PDF/2019/dezembro/27/Boletim-epidemiologico-SVS-39-FINAL.PDF" target="_blank" rel="noopener">BE Vol. 50 Nº 39 – Sarampo (SE 39 a 50 de 2019); manejo de capivaras em áreas com casos de febre maculosa brasileira; Verão seguro; Situação Epidemiológica da Febre Amarela no monitoramento 2018/2019 e perspectivas para o período sazonal 2019/2020 no Brasil</a></li>
<li><a href="/images/pdf/2019/dezembro/16/Boletim-epidemiologico-SVS-37-interativo-final.pdf">BE Vol. 50 Nº 37 – Sarampo (SE 36 a 47) vigilância epidemiológica, apuração final da campanha, análise da mortalidade de pedestres no Brasil (2010 a 2017), monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (SE 1 a 47)</a></li>
<li><a href="/images/pdf/2019/novembro/20/Boletim-epidemiologico-SVS-35.pdf" target="_blank">BE Vol. 50 Nº 35 - Sarampo (SE 34 a 45), vigilância laboratorial do sarampo, vacinação contra o sarampo, orientações para a 2ª etapa da vacinação contra o sarampo, raiva, recomendações para uso dos imunobiológicos, malária (2017 a 2019) e monitoramento das manchas de óleo</a></li>
<li><a href="https://portalarquivos2.saude.gov.br/images/pdf/2019/novembro/13/Boletim-epidemiologico-SVS-34-V1.pdf" target="_blank" rel="noopener">BE Vol. 50 Nº 34 - Monitoramento das manchas de óleo no Nordeste; sarampo (Orientações para 2ª etapa da vacinação) e arboviroses (SE 1 a 44)</a></li>
<li><a href="https://portalarquivos2.saude.gov.br/images/pdf/2019/novembro/07/Boletim-epidemiologico-SVS-33-7nov19.pdf" target="_blank" rel="noopener">BE Vol. 50 Nº 33, Sarampo (SE 32 a 43); vigilância laboratorial do sarampo; informações sobre vacinação contra o sarampo; situação dos imunobiológicos; Programa Vida no Trânsito e monitoramento das manchas de óleo no Nordeste</a></li>
<li><a href="https://portalarquivos2.saude.gov.br/images/pdf/2019/novembro/07/Boletim-epidemiologico-SVS-33-7nov19.pdf" target="_blank" rel="noopener"></a><a href="http://saude.gov.br/images/pdf/2019/outubro/24/Boletim-epidemiologico-SVS-31.pdf">BE Vol. 50 Nº 31, 2019: Sarampo (SE 30 a 42 de 2019);&nbsp; arboviroses (SE 1 a 40 de 2019) e óbitos por lesões de trânsito (2000 a 2017)</a></li>
<li><a href="/images/pdf/2019/novembro/07/Boletim-epidemiologico-SVS-30.pdf">BE Vol. 50 Nº 30, 2019: &nbsp;Sarampo (SE 29 a 40 de 2019); violências contra mulheres (2011-2018)&nbsp; e CIOCS - Copa do Mundo FIFA Sub-17</a></li>
<li><a href="/images/pdf/2019/outubro/10/Boletim-epidemiologico-SVS-29.pdf" target="_blank">BE Vol. 50 Nº 29, 2019: Cobertura nacional da vacina tríplice viral: primeira dose para crianças com até 1 ano, de 2015 a 2019</a></li>
<li><a href="/images/pdf/2019/outubro/04/BE-multitematico-n28.pdf" target="_blank">BE Vol. 50 Nº 28, 2019: sarampo (SE 28 a 39 de 2019); febre do Mayaro e febre do Oropouche; acidentes escorpiônicos; arboviroses urbanas transmitidas pelo Aedes Aegypti (SE 1 a 36 de 2019)</a></li>
<li><a href="/images/pdf/2019/setembro/25/BE-sarampo-27-25set19.pdf">BE Vol. 50 Nº 27, 2019: &nbsp;Vigilância Epidemiológica do Sarampo no Brasil 2019: Semanas Epidemiológicas 27 a 38 de 2019</a></li>
<li><a href="/images/pdf/2019/setembro/19/BE-sarampo-25-18set19.pdf">BE Vol. 50 Nº 25, 2019: Vigilância Epidemiológica do Sarampo no Brasil 2019: Semanas Epidemiológicas 26 a 37 de 2019</a></li>
<li><a href="/images/pdf/2019/setembro/13/BE-sarampo-23-final.pdf" target="_blank" rel="noopener">BE Vol. 50 Nº 23, 2019: Vigilância Epidemiológica do Sarampo&nbsp;no Brasil, 2019, Semanas Epidemiológicas 25 a 36 de 2019</a></li>
<li><a href="/images/pdf/2019/setembro/06/BE-sarampo-20-.pdf">BE Vol. 50&nbsp; Nº&nbsp; 20, 2019: Vigilância Epidemiológica do sarampo no Brasil, Semanas Epidemiológicas 24 a 35 de 2019</a></li>
<li><a href="/images/pdf/2019/setembro/09/BE-sarampo-19.pdf" target="_blank">BE Vol. 50&nbsp; Nº&nbsp; 19, 2019: Vigilância Epidemiológica do sarampo no Brasil, Semanas Epidemiológicas 23 a 34 de 2019</a></li>
<li><a href="/images/pdf/2019/setembro/09/BE-sarampo-18.pdf">BE Vol. 50&nbsp; Nº&nbsp; 18, 2019: Vigilância Epidemiológica do sarampo no Brasil, Semanas Epidemiológicas 21 a 33 de 2019&nbsp;</a></li>
<li><a href="/images/pdf/2019/setembro/09/BE-sarampo-17.pdf">BE Vol. 50&nbsp; Nº&nbsp; 17, 2019: Monitoramento dos casos de sarampo no Brasil, Semanas Epidemiológicas 20 a 31 de 2019. Lista de municípios com surto de sarampo até 13/08/2019.</a></li>
<li><a href="/images/pdf/2014/junho/03/Boletim-Sarampo-02-12-13.pdf" target="_blank" rel="noopener noreferrer">Vol. 44 - Nº 16 - 2013 - Surto de sarampo em Pernambuco</a></li>
<li><a href="/images/pdf/2014/julho/23/BE-2012-43--3--pag-17-a-20-Sarampo.pdf" target="_blank" rel="noopener noreferrer">Vol. 43 - Nº 3 - 2012 - Investigação de surto de sarampo no Rio Grande do Sul, agosto de 2010</a></li>
<li><a href="/images/pdf/2014/julho/15/Ano05-n04-evento-adv-vac-tv-pr-completo.pdf" target="_blank" rel="noopener noreferrer">Vol. 5 - Nº 4 - 2005 - Investigação de eventos adversos após a administração de vacina tríplice viral durante a campanha de seguimento contra sarampo em 2004</a></li>
</ul>
<!-- END: Articles Anywhere -->
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Sífilis</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/fevereiro/21/Boletim-epidemiologico-SVS-08.pdf" target="_blank">BE Vol. 51 Nº 8 - Vacinação para o viajante e população em geral; orientações para prevenção de doenças transmissíveis e acidentes por animais peçonhentos; vigilância das IST; uso de álcool: alerta para o período do carnaval</a>Feito.</p>
<p><a href="/images/pdf/2019/outubro/30/Boletim-S--filis-2019-internet.pdf" target="_blank">BE Número Especial: Sífilis | 2019</a></p>
<p><a href="http://portalarquivos.saude.gov.br/images/pdf/2017/novembro/13/BE-2017-038-Boletim-Sifilis-11-2017-publicacao-.pdf">Vol. 48 - Nº 36 - 2017, Sífilis 2017</a></p>
<p><a href="/images/pdf/2016/outubro/31/2016_030_Sifilis-publicao2.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 35 - 2016 - Sífilis V, 2016</a></p>
<p><a href="/images/pdf/2014/julho/16/Ano07-n07-aval-notif-sifilis-cong-br-completo--1-.pdf">Vol. 7 - Nº 7 - 2007 - Avaliação da notificação da sífilis congênita no Brasil, 2000 a 2003</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Síndrome febril hemorrágica aguda</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2014/julho/16/Ano04-n04-sve-sindr-feb-ict-hem-br-completo.pdf" target="_blank" rel="noopener noreferrer">Vol. 4 - Nº 4 - 2004 - Sistema de Vigilância da Síndrome Febril Ictérica Aguda e/ou Síndrome Febril Hemorrágica Aguda. Brasil, 2004</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Toxoplasmose</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2019/dezembro/23/Boletim-epidemiologico-SVS-38-2-interativo.pdf" target="_blank">BE Vol. 50 Nº 38 – Influenza (SE 1 a 49 de 2019); ampliação do diagnóstico da toxoplasmose congênita, monitoramento de dengue, chikungunya e Zika, até a SE 49; atualizações das indicações da vacina Febre Amarela e Campanha de Vacinação contra o HPV</a></p>
<p><a href="/images/pdf/2014/julho/23/BE-2012-43--2---pag-12-a-16-Toxoplasmose.pdf" target="_blank" rel="noopener noreferrer">Vol. 43 - Nº 2 - 2012 - Investigação de casos de toxoplasmose aguda em Paraty e Angra dos Reis – Rio de Janeiro, Agosto de 2010</a></p>
<p><a href="/images/pdf/2014/julho/18/Ano06-n03-surto-familiar-toxoplasmose-rs-completo.pdf" target="_blank" rel="noopener noreferrer">Vol. 6 - Nº 3 - 2006 - Surto intrafamiliar de toxoplasmose, Santa Vitória do Palmar-RS, Julho de 2005</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Tuberculose</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/marco/24/Boletim-tuberculose-2020-marcas--1-.pdf" target="_blank">BE Número Especial – Tuberculose 2020</a></p>
<p><a href="/images/pdf/2020/marco/30/Boletim-epidemiologico-SVS-12.pdf">BE Vol. 51 Nº 12 - Monitoramento dos casos de arboviroses (dengue, chikungunya e zika), SE 1 a 11, 2020; febre amarela 2019/2020; sarampo: SE 1 a 10, 2020; ações de enfrentamento da tuberculose em imigrantes e Síndrome congênita associada à infecção pelo vírus Zika: SE 10, 2020</a></p>
<p><a href="/images/pdf/2020/marco/30/Boletim-epidemiologico-SVS11.pdf" target="_blank">BE Vol. 51 Nº 11 - Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e zika), SE 1 a 10, 2020; Vigilância da infecção latente pelo Mycobacterium tuberculosis; Doença de Chagas - Protocolos clínicos de seguimento pós-tratamento em grupos especiais</a><a href="/images/pdf/2020/marco/21/Boletim_epidemiologico_SVS_11.pdf" target="_blank"><br></a></p>
<p><a href="/images/pdf/2019/outubro/01/Boletim-tuberculose-2019.pdf" target="_blank">BE Vol. 50 Nº 26, 2019: Panorama epidemiológico da coinfecção TB-HIV no Brasil 2019: Setembro 2019</a></p>
<p><a href="/images/pdf/2019/marco/22/2019-009.pdf">BE Vol. 50&nbsp; Nº&nbsp; 09, 2019: Brasil Livre da Tuberculose: evolução dos cenários epidemiológicos e operacionais da doença</a></p>
<p><a href="/images/pdf/2018/setembro/05/2018-041.pdf">Vol. 49 -&nbsp; Nº&nbsp; 37- 2018 - Experiências de Programas de Controle da Tuberculose: Porque juntos iremos detectar, tratar e acabar com a tuberculose como problema de saúde pública no Brasil</a></p>
<p><a href="/images/pdf/2018/marco/26/2018-009.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 -&nbsp; Nº&nbsp; 11- 2018 - Implantação do Plano Nacional pelo Fim da Tuberculose como Problema de Saúde Pública no Brasil: primeiros passos rumo ao alcance das metas.</a></p>
<p><a href="/images/pdf/2017/dezembro/05/2017-030.pdf">Vol. 48 - Nº 40 - 2017 - Coinfecção TB-HIV no Brasil: panorama epidemiológico e atividades colaborativas</a></p>
<p><a href="/images/pdf/2017/marco/23/2017-V-48-N-8-Indicadores-priorit--rios-para-o-monitoramento-do-Plano-Nacional-pelo-Fim-da-Tuberculose-como-Problema-de-Sa--de-P--blica-no-Brasil.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 8 - 2017 - Indicadores prioritários para o monitoramento do Plano Nacional pelo Fim da Tuberculose como Problema de Saúde Pública no Brasil</a></p>
<p><a href="/images/pdf/2016/marco/24/2016-009-Tuberculose-001.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 13 - 2016 - Perspectivas brasileiras para o fim da tuberculose como problema de saúde pública&nbsp;</a></p>
<p><a href="/images/pdf/2015/marco/27/2015-007---BE-Tuberculose---para-substitui----o-no-site.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 9 - 2015 - Tuberculose - 2015: Detectar, tratar e curar: desafios e estratégias brasileiras frente à tuberculose</a></p>
<p><a href="/images/pdf/2014/agosto/13/BE-2014-45--18----Tuberculose.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 18 - 2014 - Tuberculose, população indígena e determinantes sociais</a></p>
<p><a href="/images/pdf/2014/maio/29/BE-2014-45--2--tb.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 2 - 2014 - O controle da tuberculose no Brasil: avanços, inovações e desafios</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--13----TB.pdf" target="_blank" rel="noopener noreferrer">Vol. 44 - Nº 13 - 2013 - O controle da tuberculose na população indígena</a></p>
<p><a href="/images/pdf/2014/maio/06/boletim2-2013-tb-web.pdf" target="_blank" rel="noopener noreferrer">Vol. 44 - Nº 2 - 2013 - Tuberculose: alinhada com o social, afinada com a tecnologia</a></p>
<p><a href="/images/pdf/2014/julho/23/BE-2012-43--1--pag-15-19-Tuberculose-no-Brasil.pdf" target="_blank" rel="noopener noreferrer">Vol. 43 - Nº 1 - 2012 - Tuberculose no Brasil: realidade e perspectivas</a></p>
<p><a href="/images/pdf/2014/julho/23/BE-2012-43-Mar--o---Especial-Tuberculose.pdf" target="_blank" rel="noopener noreferrer">Vol. 43 - Março - 2012 - Especial Tuberculose</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Tracoma</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/marco/05/Boletim-epidemiologico-SVS-09--.pdf" target="_blank">BE Vol. 51 Nº 9 - Vigilância epidemiológica do sarampo no Brasil 2020: Semana Epidemiológica 01; situação epidemiológica da febre amarela 2019/2020; importância dos códigos garbage para a magnitude da mortalidade por causas externas no Brasil; tracoma e acidentes ofídicos no Brasil, 2018</a></p>
<p><a href="/images/pdf/2018/novembro/19/2018-032.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 49, 2018 - Situação Epidemiológica e estratégias de prevenção, controle e eliminação das Doenças Tropicais Negligenciadas no Brasil, 1995 a 2016</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Varicela</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2015/fevereiro/23/boletim-epi-n6-dez2010-2-.pdf" target="_blank" rel="noopener noreferrer">Vol. 10 - Nº 6 - 2010 - Surto de varicela em Várzea, na Paraíba, setembro, 2008</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Vigilância da qualidade da água</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/marco/27/Boletim-epidemiologico-SVS-13.pdf" target="_blank">BE Vol.51 Nº 13 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 12, 2020; febre amarela; análise da anomalias congênitas no Brasil; abastecimento de água para consumo humano em 2019 e medicamentos antileishmania no âmbito do SUS</a></p>
<p><a href="/images/pdf/2020/fevereiro/05/Boletim-epidemiologico-SVS-05.pdf">BE Vol. 51 Nº 5 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 03; Síndrome Congênita associada à infecção pelo vírus Zika: monitoramento do ano de 2019 e a qualidade da água para consumo humano no Rio de Janeiro.</a></p>
<p><a href="/images/pdf/2016/marco/23/2015-052---Agrot--xicos-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 12 - 2016 - Monitoramento de agrotóxicos em água para consumo humano no Brasil, 2014 </a></p>
<p><a href="/images/pdf/2015/dezembro/08/2014-015---Fl--or---prelo---01.12.15.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 40 - 2015 - Monitoramento do parâmetro fluoreto na água para consumo humano e a situação da fluorose e cárie nas capitais brasileiras no ano de 2010</a></p>
<p><a href="/images/pdf/2015/agosto/20/2014-028---Boletim-Agrotoxico-.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 4 - 2015 - Monitoramento de agrotóxicos na Água para Consumo Humano no Brasil, 2013</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2014-45--1--Cianobact--rias.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 1 - 2014 - Monitoramento de cianobactérias nos mananciais de abastecimento de água para consumo humano no Brasil, 2012</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--17----Agrot--xicos.pdf" target="_blank" rel="noopener noreferrer">Vol. 44 - Nº 17 - 2013 - Monitoramento de agrotóxicos na água para consumo humano no Brasil, 2012</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--10----Agua--agrotoxicos-.pdf" target="_blank" rel="noopener noreferrer">Vol. 44 - Nº 10 - 2013 - Monitoramento de agrotóxicos na água para consumo humano no Brasil, 2011</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Vigilância em saúde ambiental</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2018/dezembro/26/2018-027.pdf">Vol. 49 Nº 58, 2018: Intoxicações exógenas relacionadas ao trabalho no Brasil de 2007 a 2016</a></p>
<p><a href="/images/pdf/2018/marco/22/2017-032-Publicacao.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 10 - 2018 - Desastres naturais e saúde: análise do cenário de eventos hidrológicos no Brasil e seus potenciais impactos sobre o Sistema Único de Saúde</a></p>
<p><a href="/images/pdf/2016/marco/23/2015-052---Agrot--xicos-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 12 - 2016 - Monitoramento de agrotóxicos em água para consumo humano no Brasil, 2014&nbsp;</a></p>
<p><a href="/images/pdf/2015/dezembro/08/2014-015---Fl--or---prelo---01.12.15.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 40 - 2015 - Monitoramento do parâmetro fluoreto na água para consumo humano e a situação da fluorose e cárie nas capitais brasileiras no ano de 2010</a></p>
<p><a href="/images/pdf/2015/agosto/20/2014-028---Boletim-Agrotoxico-.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 4 - 2015 - Monitoramento de Agrotóxicos na Água para Consumo Humano no Brasil, 2013</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2014-45--1--Cianobact--rias.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 1 - 2014 - Monitoramento de cianobactérias nos mananciais de abastecimento de água para consumo humano no Brasil, 2012</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--17----Agrot--xicos.pdf" target="_blank" rel="noopener noreferrer">Vol. 44 - Nº 17 - 2013 - Monitoramento de agrotóxicos na água para consumo humano no Brasil, 2012</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--10----Agua--agrotoxicos-.pdf" target="_blank" rel="noopener noreferrer">Vol. 44 - Nº 10 - 2013 - Monitoramento de agrotóxicos na água para consumo humano no Brasil, 2011</a></p>
<p><a href="/images/pdf/2014/julho/23/BE-2012-43--4---pag-7-a-10-Agente-ambiental.pdf" target="_blank" rel="noopener noreferrer">Vol. 43 - Nº 4 - 2012 - Avaliação de Conhecimentos, Atitudes e Práticas dos Agentes de Saúde Ambiental e Controle de Endemias (Asace) do município do Recife em relação ao Diflubenzuron, Pernambuco, 2011</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n13-surto-intox-piretroides-go-completo.pdf" target="_blank" rel="noopener noreferrer">Vol. 8 - Nº 13 - 2008 - Surto de intoxicação por piretróides no Município de Porangatu-GO</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Vigilância em saúde do trabalhador </a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="http://portalarquivos2.saude.gov.br/images/pdf/2019/marco/29/2018-059.pdf" target="_blank" rel="noopener noreferrer">Vol. 50&nbsp; Nº&nbsp; 11, 2019: Acidentes de Trabalho por animais peçonhentos entre trabalhadores do campo, floresta e águas, Brasil 2007 a 2017</a></p>
<p><a href="/images/pdf/2018/dezembro/26/2018-027.pdf">Vol. 49 Nº 58, 2018: Intoxicações exógenas relacionadas ao trabalho no Brasil de 2007 a 2016</a></p>
<p><a href="/images/pdf/2018/junho/20/2017-033-publicacao.pdf">Vol. 49 - Nº 26- 2018 - Acidentes de transporte relacionados ao trabalho no Brasil, 2007-2016</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/junho/23/2017-005-Vigilancia-em-Saude-do-Trabalhador.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 18 - 2017 - Vigilância em Saúde do Trabalhador: um breve panorama</a></p>
<p><a href="/images/pdf/2016/fevereiro/02/2015-011---Asbestose.pdf" target="_blank" rel="noopener noreferrer"> Vol. 47 - Nº 4 - 2016 - Asbestose na saúde do trabalhador notificados no SINAN, Brasil, 2007-2013</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Vigilância em saúde indígena</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2015/abril/22/Boletim-raca-cor-09-04-15-v2.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 10 - 2015 - Indicadores de Vigilância em Saúde, analisados segundo a variável raça/cor</a></p>
<p><a href="/images/pdf/2014/agosto/13/BE-2014-45--18----Tuberculose.pdf" target="_blank" rel="noopener noreferrer">Vol. 45 - Nº 18 - 2014 - Tuberculose, população indígena e determinantes sociais</a></p>
<p><a href="/images/pdf/2014/junho/11/BE-2013-44--13----TB.pdf" target="_blank" rel="noopener noreferrer">Vol. 44 - Nº 13 - 2013 - O controle da tuberculose na população indígena</a></p>
<p><a href="/images/pdf/2014/julho/23/BE-2012-43--3--pag-11-a-16-Sindrome-gripal.pdf" target="_blank" rel="noopener noreferrer">Vol. 43 - Nº 3 - 2012 - Investigação de Surto de Síndrome Gripal em Populações Indígenas, Altamira – Pará, abril – setembro de 2010</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n01-obitos-criancas-aldeia-ro-completo.pdf" target="_blank" rel="noopener noreferrer">Vol. 8 - Nº 1 - 2008 - Investigação epidemiológica de óbitos de crianças indígenas da Aldeia Lage Velho, Etnia Pacaás Novos, Guajará-Mirim-RO, Fevereiro de 2007</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Vigilância epidemiológica hospitalar</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2014/julho/23/BE-2012-43--4---pag-11-a-15-micobacteria.pdf" target="_blank" rel="noopener noreferrer">Vol. 43 - Nº 4 - 2012 - Investigação de surto de infecção por micobactéria de crescimento rápido em pacientes submetidos a procedimentos de videocirurgia no município de Manaus, agosto de 2010</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n19-surto-micob-cirurg-pr-completo.pdf" target="_blank" rel="noopener noreferrer">Vol. 8 - Nº 19 - 2008 - Surto de infecção hospitalar por microbactérias de crescimento rápido em procedimento de videocirurgia, Curitiba-PR</a></p>
<p><a href="/images/pdf/2014/junho/25/Ano08-n10-infeccao-hosp-micob-pr-completo.pdf" target="_blank" rel="noopener noreferrer">Vol. 8 - Nº 10 - 2008 - Surto de infecção hospitalar por micobactérias de crescimento rápido, Curitiba-PR, 2007</a></p>
</div>
	</li>
<li class="jp-accordion-item">
<h3 class="ui-accordion-header ui-helper-reset ui-state-default ui-corner-all"><a href="#">Zika</a></h3>
<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom">
<p><a href="/images/pdf/2020/April/16/Boletim-epidemiologico-SVS-16.pdf" target="_blank">BE Vol. 51 Nº 16 -&nbsp;Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes Aegypti (dengue, chikungunya e zika), Semanas Epidemiológicas 1 a 15, 2020</a></p>
<p><a href="/images/pdf/2020/April/13/Boletim-epidemiologico-SVS-15.pdf" target="_blank">BE Vol. 51 Nº 15 - Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes Aegypti (dengue, chikungunya e zika), Semanas Epidemiológicas 1 a 14 de 2020, e da febre amarela (2019/2020)</a></p>
<p><a href="/images/pdf/2020/April/03/Boletim-epidemiologico-SVS-14.pdf">BE Vol. 51 Nº 14 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 13, 2020; febre amarela; diferenças regionais no consumo de alimentos entre adolescentes e adultos e leishmaniose visceral: Experiência do IEC</a><a href="/images/pdf/2020/marco/27/Boletim-epidemiologico-SVS-13.pdf" target="_blank"></a></p>
<p><a href="/images/pdf/2020/marco/27/Boletim-epidemiologico-SVS-13.pdf" target="_blank">BE Vol.51 Nº 13 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 12, 2020; febre amarela; análise da anomalias congênitas no Brasil; abastecimento de água para consumo humano em 2019 e medicamentos antileishmania no âmbito do SUS</a><a href="/images/pdf/2020/marco/30/Boletim-epidemiologico-SVS-12.pdf"></a></p>
<p><a href="/images/pdf/2020/marco/30/Boletim-epidemiologico-SVS-12.pdf">BE Vol. 51 Nº 12 - Monitoramento dos casos de arboviroses (dengue, chikungunya e zika), SE 1 a 11, 2020; febre amarela 2019/2020; sarampo: SE 1 a 10, 2020; ações de enfrentamento da tuberculose em imigrantes e Síndrome congênita associada à infecção pelo vírus Zika: SE 10, 2020</a></p>
<p><a href="/images/pdf/2020/marco/30/Boletim-epidemiologico-SVS11.pdf" target="_blank">BE Vol. 51 Nº 11 - Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e zika), SE 1 a 10, 2020; Vigilância da infecção latente pelo Mycobacterium tuberculosis; Doença de Chagas - Protocolos clínicos de seguimento pós-tratamento em grupos especiais</a><a href="/images/pdf/2020/marco/21/Boletim_epidemiologico_SVS_11.pdf" target="_blank"><br></a></p>
<p><a href="/images/pdf/2020/marco/06/Boletim-epidemiologico-SVS-10.pdf" target="_blank">BE Vol. 51 Nº10 Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika), Semanas Epidemiológicas 1 a 7, 2020.</a></p>
<p><a href="/images/pdf/2020/fevereiro/19/Boletim-epidemiologico-SVS-07.pdf" target="_blank">BE Vol. 51 Nº 7 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 05; síndrome congênita associada à infecção pelo vírus Zika até a SE 05 e mobilização nacional para enfrentamento da hanseníase</a></p>
<p><a href="/images/pdf/2020/April/08/Boletim-epidemiologico-SVS-06-v2.pdf" target="_blank" rel="noopener">BE Vol. 51 Nº 6 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 04; sarampo no Brasil 2019: janeiro a dezembro e situação da febre amarela: monitoramento 2019/2020</a></p>
<p><a href="/images/pdf/2020/fevereiro/05/Boletim-epidemiologico-SVS-05.pdf">BE Vol. 51 Nº 5 - Monitoramento da dengue, chikungunya e Zika, SE 01 a 03; Síndrome Congênita associada à infecção pelo vírus Zika: monitoramento do ano de 2019 e a qualidade da água para consumo humano no Rio de Janeiro.</a></p>
<p><a href="/images/pdf/2020/janeiro/20/Boletim-epidemiologico-SVS-02-1-.pdf" target="_blank">BE Vol. 51 Nº 2 – Monitoramento de dengue, chikungunya e Zika, até a SE 52 de 2019; Campanha de vacinação contra o sarampo</a></p>
<p><a href="/images/pdf/2019/dezembro/23/Boletim-epidemiologico-SVS-38-2-interativo.pdf" target="_blank">BE Vol. 50 Nº 38 – Influenza (SE 1 a 49 de 2019); ampliação do diagnóstico da toxoplasmose congênita, monitoramento de dengue, chikungunya e Zika, até a SE 49; atualizações das indicações da vacina Febre Amarela e Campanha de Vacinação contra o HPV</a></p>
<p><a href="/images/pdf/2019/dezembro/16/Boletim-epidemiologico-SVS-37-interativo-final.pdf">BE Vol. 50 Nº 37 – Sarampo (SE 36 a 47) vigilância epidemiológica, apuração final da campanha, análise da mortalidade de pedestres no Brasil (2010 a 2017), monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (SE 1 a 47)</a></p>
<p><a href="https://portalarquivos2.saude.gov.br/images/pdf/2019/novembro/13/Boletim-epidemiologico-SVS-34-V1.pdf" target="_blank" rel="noopener">BE Vol. 50 Nº 34 - Monitoramento das manchas de óleo no Nordeste; sarampo (Orientações para 2ª etapa da vacinação) e arboviroses (SE 1 a 44)</a></p>
<p><a href="/images/pdf/2019/novembro/07/Boletim-epidemiologico-SVS-33-7nov19.pdf" target="_blank">BE Vol. 50 Nº 33, Sarampo (SE 32 a 43); vigilância laboratorial do sarampo; informações sobre vacinação contra o sarampo; situação dos imunobiológicos; Programa Vida no Trânsito e monitoramento das manchas de óleo no Nordeste</a></p>
<p><a href="/images/pdf/2019/outubro/31/Boletim-epidemiologico-SVS-32.pdf" target="_blank">BE Vol. 50 Nº 32, 2019: Vigilância em Saúde: monitoramento das manchas de óleo no litoral do Nordeste ; arboviroses (SE 1 a 41 de 2019) e consumo de bebidas alcoólicas entre escolares no Brasil: uma análise da Pesquisa Nacional de Saúde do Escolar, 2015</a></p>
<p><a href="http://saude.gov.br/images/pdf/2019/outubro/24/Boletim-epidemiologico-SVS-31.pdf">BE Vol. 50 Nº 31, 2019: Sarampo (SE 30 a 42 de 2019);&nbsp; arboviroses (SE 1 a 40 de 2019) e óbitos por lesões de trânsito (2000 a 2017)</a></p>
<p><a href="/images/pdf/2019/outubro/10/Boletim-epidemiologico-SVS-29.pdf" target="_blank">BE Vol. 50 Nº 29, 2019: Cobertura nacional da vacina tríplice viral: primeira dose para crianças com até 1 ano, de 2015 a 2019</a><a href="/images/pdf/2019/outubro/04/BE-multitematico-n28.pdf">&nbsp;arboviroses urbanas transmitidas pelo Aedes Aegypti (SE 1 a 38 de 2019)</a></p>
<p><a href="/images/pdf/2019/outubro/04/BE-multitematico-n28.pdf" target="_blank">BE Vol. 50 Nº 28, 2019: sarampo (SE 28 a 39 de 2019); febre do Mayaro e febre do Oropouche; acidentes escorpiônicos; arboviroses urbanas transmitidas pelo Aedes Aegypti (SE 1 a 36 de 2019)</a></p>
<p><a href="/images/pdf/2019/setembro/11/BE-arbovirose-22.pdf">BE Vol. 50&nbsp; Nº&nbsp; 22, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 34 de 2019</a></p>
<p><a href="/images/pdf/2019/abril/30/2019-013-Monitoramento-dos-casos-de-arboviroses-urbanas-transmitidas-pelo-Aedes-publicacao.pdf">BE Vol. 50&nbsp; Nº&nbsp; 13, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 12 de 2019</a><a href="http://portalarquivos2.saude.gov.br/images/pdf/2019/abril/01/2018-066.pdf" target="_blank" rel="noopener noreferrer"></a></p>
<p><a href="/images/pdf/2019/marco/25/2019-013-Monitoramento-dos-casos-de-arboviroses-publicacao-25-03-2019.pdf">BE Vol. 50&nbsp; Nº&nbsp; 10, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 11 de 2019</a></p>
<p><a href="/images/pdf/2019/marco/21/2019-006.pdf">BE Vol. 50&nbsp; Nº&nbsp; 07, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 7 de 2019</a></p>
<p><a href="/images/pdf/2019/marco/21/2019-005.pdf">BE Vol. 50&nbsp; Nº&nbsp; 06, 2019: Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes (dengue, chikungunya e Zika) até a Semana Epidemiológica 6 de 2019</a></p>
<p><a href="/images/pdf/2019/fevereiro/26/2019-004-Dengue-SE-5-publica----o-18-02-2019.pdf">Vol. 50 Nº 05, 2019: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 5 de 2019</a></p>
<p><a href="/images/pdf/2019/janeiro/28/2019-002.pdf" target="_blank" rel="noopener noreferrer">Vol. 50&nbsp; Nº 04, 2019: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica SE 52&nbsp; 2018 </a></p>
<p><a href="/images/pdf/2019/janeiro/28/2018-068.pdf" target="_blank" rel="noopener noreferrer">Vol. 50&nbsp; Nº 01, 2019: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica SE 50 2018</a></p>
<p><a href="/images/pdf/2019/janeiro/02/2018-067.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 Nº 59, 2018: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 49 de 2018</a></p>
<p><a href="/images/pdf/2018/dezembro/18/2018-064.pdf">Vol. 49 Nº 57, 2018: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 48 de 2018</a></p>
<p><a href="/images/pdf/2019/janeiro/14/BE-Vol. 49 – Nº 56, 2018- dengue chikungunya zika_aed_10jan19.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 Nº 56, 2018: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 47 de 2018</a></p>
<p><a href="/images/pdf/2019/janeiro/14/BE-Vol. 49 - Nº 55, 2018- dengue chikungunya zika_aed_10jan19.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 Nº 55, 2018: Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 46 de 2018</a></p>
<p><a href="/images/pdf/2019/janeiro/14/BE-Vol. 49 - Nº 53, 2018 - dengue chikungunya zika_aed_10jan19.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 53, 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 45&nbsp;de 2018</a></p>
<p><a href="/images/pdf/2018/novembro/23/Publicacao-2018-57-SE44.pdf">Vol. 49 - Nº 51, 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 44&nbsp;de 2018</a></p>
<p><a href="/images/pdf/2018/novembro/13/2018-056.pdf">Vol. 49 Nº 48, 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 43&nbsp;de 2018</a></p>
<p><a href="/images/pdf/2018/novembro/12/2018-034.pdf">Vol. 49&nbsp; Nº&nbsp; 47, 2018&nbsp;-&nbsp;Situação epidemiológica da infecção pelo vírus Zika no Brasil, de 2015 a 2017</a></p>
<p><a href="/images/pdf/2018/novembro/05/2018-051-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-doen--a-aguda-pelo-v--rus-Zika-at---a-Semana-Epidemiol--gica-42-de-2018-publica--ao.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 44 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 42 de 2018 </a></p>
<p><a href="/images/pdf/2018/outubro/31/BE-2018-050-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-doen--a-aguda-pelo-v--rus-Zika-at---a-Semana-Epidemiol--gica-41-de-2018-ajustado.pdf" target="_blank" rel="noopener noreferrer">Vol. 49&nbsp; Nº&nbsp; 43, 2018: Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 41 de 2018</a></p>
<p><a href="/images/pdf/2018/outubro/26/2018-049.pdf">Vol. 49&nbsp; Nº&nbsp; 42, 2018: Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 38&nbsp;de 2018</a></p>
<p><a href="/images/pdf/2018/outubro/31/Vol. 49 Nº 40.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 40 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 36 de 2018</a></p>
<p><a href="/images/pdf/2018/outubro/31/Vol. 49 Nº 39.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 39 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 34 - 2018</a></p>
<p><a href="/images/pdf/2018/agosto/31/2018-043.pdf">Vol. 49 - Nº 36 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 32 -2018</a></p>
<p><a href="/images/pdf/2018/outubro/31/Vol. 49 Nº 35.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 35 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica - 2018</a></p>
<p><a href="/images/pdf/2018/agosto/21/Publicacao-BE-2018-SE-30.pdf">Vol. 49 - Nº&nbsp; 34 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 30 - 2018</a></p>
<p><a href="http://portalarquivos2.saude.gov.br/images/pdf/2018/agosto/07/2018_040.pdf">Vol. 49 - Nº 33 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 28 - 2018</a></p>
<p><a href="/images/pdf/2018/julho/12/Monitoramento-dos-casos-ate-a-Semana-Epidemiologica-25-de-2018.pdf">Vol. 49 - Nº&nbsp; 32 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 25 de 2018</a></p>
<p><a href="/images/pdf/2018/julho/06/Monitoramento-dos-casos-de-dengue-23-018.pdf">Vol. 49 - Nº&nbsp; 31 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 23 de 2018</a></p>
<p><a href="/images/pdf/2018/julho/06/Monitoramento-dos-casos-de-dengue-21-de-2018.pdf">Vol. 49 - Nº&nbsp; 30 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 21 de 2018</a></p>
<p><a href="/images/pdf/2018/junho/25/2018-023.pdf">Vol. 49 - Nº&nbsp; 28- 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 19_2018</a></p>
<p><a href="/images/pdf/2018/junho/11/2018-022.pdf">Vol. 49 - Nº&nbsp; 24- 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 17 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/28/2018-023.pdf">Vol. 49 - Nº&nbsp; 23 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e doença aguda pelo vírus Zika até a Semana Epidemiológica 16 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/28/2018-022.pdf">Vol. 49 - Nº&nbsp; 22- 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 15 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/28/BE-2018-021.pdf">Vol. 49 - Nº&nbsp; 21 - 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 09 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/07/2018-018.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº&nbsp; 20 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 15 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/02/2018-017-Publicacao.pdf">Vol. 49 - Nº&nbsp; 18 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 14 de 2018</a></p>
<p><a href="/images/pdf/2018/maio/02/2018-015-Publicacao.pdf">Vol. 49 - Nº&nbsp; 17 - 2018 -&nbsp; Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 13 de 2018</a></p>
<p><a href="/images/pdf/2018/abril/27/BE-2018-14-SE-12.pdf">Vol. 49 - Nº 16 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 12 de 2018</a></p>
<p><a href="/images/pdf/2018/abril/19/BE-2018-13-SE-11-Publicacao.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 15 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 11, de 2018</a></p>
<p><a href="/images/pdf/2018/abril/06/2018-012.pdf">Vol. 49 - Nº 14 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 10, de 2018</a></p>
<p><a href="/images/pdf/2018/abril/06/2018-011.pdf">Vol. 49 - Nº 13 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 09, de 2018</a></p>
<p><a href="/images/pdf/2018/marco/29/2018-010.pdf">Vol. 49 - Nº&nbsp; 12- 2018 -&nbsp;Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 8 de 2018</a></p>
<p><a href="/images/pdf/2018/marco/06/2018-008-Publicacao.pdf">Vol. 49 - Nº 9 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 07 de 2018</a></p>
<p><a href="/images/pdf/2018/marco/06/2018-007-Publicacao.pdf">Vol. 49 - Nº 8 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 06 de 2018</a></p>
<p><a href="/images/pdf/2018/fevereiro/20/2018-007.pdf">Vol. 49 - Nº 7 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 5 de 2018</a></p>
<p><a href="/images/pdf/2018/fevereiro/20/2018-003-Final.pdf">Vol. 49 - Nº 6 - 2018 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 52 de 2017</a></p>
<p><a href="/images/pdf/2018/fevereiro/19/2018-005.pdf">Vol. 49 - Nº 5 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 3 de 2018</a></p>
<p><a href="/images/pdf/2018/janeiro/23/Boletim-2018-001-Dengue.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 2 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 52 de 2017</a></p>
<p><a href="/images/pdf/2018/janeiro/23/Boletim-2017-051-Dengue.pdf" target="_blank" rel="noopener noreferrer">Vol. 49 - Nº 1 - 2018 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 51 de 2017</a></p>
<p><a href="/images/pdf/2018/janeiro/10/2017-046-Publicacao.pdf">Vol. 48 - Nº 45 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 50, 2017</a></p>
<p><a href="/images/pdf/2017/dezembro/26/2017-044.pdf">Vol. 48 - Nº 44 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 49 de 2017</a></p>
<p><a href="/images/pdf/2017/dezembro/26/BE-2017-043-Boletim.pdf">Vol. 48 - Nº 43 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 48 de 2017</a></p>
<p><a href="/images/pdf/2017/dezembro/21/BE-2017-042-Boletim.pdf">Vol. 48 - Nº 42 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 47 de 2017</a></p>
<p><a href="/images/pdf/2017/dezembro/12/Boletim-2017-043-final.pdf">Vol. 48&nbsp;- Nº &nbsp;41 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 44</a></p>
<p><a href="/images/pdf/2017/dezembro/08/2017-040.pdf">Vol. 48 - Nº 39 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 46 </a></p>
<p><a href="/images/PDF/2017/novembro/28/2017_039.pdf">Vol. 48&nbsp;- Nº 38 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 45, 2017</a></p>
<p><a href="/images/pdf/2017/novembro/13/2017-037.pdf">Vol. 48 - Nº 35 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 43 de 2017</a><a href="/images/pdf/2017/novembro/07/2017-035-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica-41.pdf">
 Vol. 48 - Nº 34 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 41 de 2017</a></p>
<p><a href="/images/pdf/2017/outubro/18/boletim-n32.pdf">Vol. 48 - Nº 32 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 39, 2017</a><a href="/images/pdf/2017/outubro/18/BE-Vol-48-n-31-2017-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-ate-a-Semana-Epidemiologica-37.pdf">
 Vol. 48 - Nº 31 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 37, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/setembro/15/2017-028-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica-35.pdf">Vol. 48 - Nº 29 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 35, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/agosto/29/2017-026-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica-33-de-2017.pdf">Vol. 48 - Nº 27 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 33, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/agosto/23/2017_024-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 26 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 31, 2017</a></p>
<p><a href="http://portalsaude.saude.gov.br/images/pdf/2017/julho/25/Boletim-2017_020-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-Zika.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 20 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 25, 2017</a></p>
<p><a href="/images/pdf/2017/maio/25/Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 16 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 19, 2017<br> </a><a href="/images/pdf/2017/maio/05/Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica.pdf" target="_blank" rel="noopener noreferrer"><br> Vol. 48 - Nº 14 - 2017 -Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 15, 2017</a><a href="/images/pdf/2017/abril/27/Monitoramento-integrado-de-alteracoes-no-crescimento-e-desenvolvimento-relacionadas-a-infeccao-pelo-virus-Zika.pdf" target="_blank" rel="noopener noreferrer">
 Vol. 48 - Nº 13 - 2017 -Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 14/2017</a><a href="/images/pdf/2017/marco/28/2017_006-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica-7.pdf" target="_blank" rel="noopener noreferrer"><br> </a><a href="/images/pdf/2017/abril/10/2017_011%20Monitoramento%20integrado%20de%20alteracoes%20no%20crescimento%20e%20desenvolvimento%20relacionadas%20a%20infeccao%20pelo%20virus%20Zika%20e%20outras%20etiologias%20infecciosas%20ate%20a%20Semana%20Epidemiologica%2012_rev.pdf" target="_blank" rel="noopener noreferrer"><br> Vol. 48 - Nº 12 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 12/2017</a></p>
<p><a href="/images/pdf/2017/abril/06/2017-010-Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiol--gica-12--2017.pdf">Vol. 48 - Nº 11 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 12, 2017</a></p>
<p><a href="/images/pdf/2017/abril/03/Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-10-2017.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 10 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, até a Semana Epidemiológica 10, 2017</a></p>
<p><a href="/images/pdf/2017/marco/31/Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-10-%202017.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 9 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 10, 2017</a></p>
<p><a href="/images/pdf/2017/marco/28/2017_006-Monitoramento-dos-casos-de-dengue-febre-de-chikungunya-e-febre-pelo-virus-Zika-ate-a-Semana-Epidemiologica-7.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 7 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 7, 2017</a></p>
<p><a href="/images/pdf/2017/fevereiro/27/2017_003.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 6 - 2017 - Monitoramento integrado de alterações no crescimento e desenvolvimento relacionadas à infecção pelo vírus Zika e outras etiologias infecciosas, da Semana Epidemiológica 45/2015 até a Semana Epidemiológica 02/2017</a></p>
<p><a href="/images/pdf/2017/fevereiro/17/Monitoramento-dos-casos-de-dengue--febre-de-chikungunya-e-febre-pelo-v--rus-Semana-Epidemiologica-4-2017.pdf" target="_blank" rel="noopener noreferrer">Vol. 48 - Nº 5 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 4, 2017”</a></p>
<p><a href="/images/pdf/2017/fevereiro/05/2017_002-Dengue%20SE52_corrigido.pdf">Vol. 48 - Nº 3 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 52, 2016</a></p>
<p><a href="/images/pdf/2017/janeiro/12/2017_001%20-%20Dengue%20SE51_publicacao.pdf">Vol. 48 - Nº 2 - 2017 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 51, 2016</a></p>
<p><a href="/images/pdf/2016/dezembro/20/2016-033---Dengue-SE49-publicacao.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 38 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 49, 2016</a></p>
<p><a href="/images/pdf/2016/novembro/15/2016_031-Mulheres_publicacao.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 37 - 2016 - Zika Vírus: perfil epidemiológico em mulheres 47, 2016</a></p>
<p><a href="/images/pdf/2016/setembro/16/2016-028---Dengue-SE32.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 33 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 32, 2016</a></p>
<p><a href="/images/pdf/2016/agosto/10/2016-026--2-..pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 31 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 27, 2016</a></p>
<p><a href="/images/pdf/2016/julho/15/2016-boletim-epi-n28-dengue-chik-zika-se23.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 28 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 23, 2016</a></p>
<p><a href="/images/pdf/2016/junho/30/2016-021.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 27 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 21, 2016</a></p>
<p><a href="/images/pdf/2016/junho/30/2016-020--1-.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 26 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 20, 2016</a></p>
<p><a href="/images/pdf/2016/junho/17/2016-019.pdf" target="_blank" rel="noopener noreferrer"> Vol. 47 - Nº 25 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 19, 2016</a></p>
<p><a href="/images/pdf/2016/junho/15/2016-018---Dengue-SE18-substitui----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 24 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 18, 2016</a></p>
<p><a href="/images/pdf/2016/junho/14/boletim-epid-2016-n23-dcz-se17.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 23 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 17, 2016</a></p>
<p><a href="/images/pdf/2016/maio/17/2016-016---Dengue-SE16-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 20 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 16, 2016</a></p>
<p><a href="/images/pdf/2016/junho/10/2016-014---Dengue-SE13-substitui----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 18 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 13, 2016</a></p>
<p><a href="/images/pdf/2016/abril/11/2015-013---Dengue-SE9.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 16 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 9, 2016 </a></p>
<p><a href="/images/pdf/2016/abril/05/2016-012---Relatorio-reuniao-especialistas-Aedes-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 15 - 2016 - Relatório da Reunião Internacional para Implementação de Alternativas para o Controle do Aedes aegypti no Brasil</a></p>
<p><a href="/images/pdf/2016/marco/29/2016-011-boletim-epi-dcz-se8.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 14 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 8, 2016 </a></p>
<p><a href="/images/pdf/2016/marco/23/2016-008---DengueSE8-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 11 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 7, 2016 </a></p>
<p><a href="/images/pdf/2016/marco/23/2016-007---Dengue-SE-6-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 10 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 6, 2016 </a></p>
<p><a href="/images/pdf/2016/fevereiro/29/2016-006---Dengue-SE5-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 8 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 5, 2016</a></p>
<p><a href="/images/pdf/2016/fevereiro/29/2016-005---Dengue-SE4-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 7 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 4, 2016</a></p>
<p><a href="/images/pdf/2016/fevereiro/04/2016-004---Dengue-SE3.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 6 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 3, 2016</a></p>
<p><a href="/images/pdf/2016/fevereiro/04/2016-003-Dengue-SE2.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 5 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 2, 2016</a></p>
<p><a href="/images/pdf/2016/janeiro/15/svs2016-be003-dengue-se52.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 3 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 52, 2015</a></p>
<p><a href="/images/pdf/2016/janeiro/15/svs2016-be002-dengue-se51.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 2 - 2016 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 51, 2015</a></p>
<p><a href="/images/pdf/2016/janeiro/11/2015-053-para-substituir-na-p--gina.pdf" target="_blank" rel="noopener noreferrer">Vol. 47 - Nº 1 - Monitoramento dos casos de microcefalias no Brasil até a Semana Epidemiológica 51</a></p>
<p><a href="/images/pdf/2015/dezembro/28/2015-boletim-microcefalia-se50-vol46-n46.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 46 - 2015 - Monitoramento dos casos de microcefalias no Brasil até a Semana Epidemiológica 50</a></p>
<p><a href="/images/pdf/2015/dezembro/23/2015-050---microcefalia-SE-49---publica----o-23.12.15.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 45 - Monitoramento dos casos de microcefalias no Brasil até a Semana Epidemiológica 49</a></p>
<p><a href="/images/pdf/2016/janeiro/07/2015-svs-be-pncd-se48.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 44 - 2015 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 48, 2015</a></p>
<p><a href="/images/pdf/2015/dezembro/11/svs-be-2015-047-dengue-se47-final.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 42 - 2015 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 47, 2015</a></p>
<p><a href="/images/pdf/2015/dezembro/11/svs-be-2015-048-microcefalia-se48-final2.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 41 - 2015 - Monitoramento dos casos de microcefalia no Brasil até a Semana Epidemiológica 48</a></p>
<p><a href="/images/pdf/2015/dezembro/02/2015-046---Microcefalia----02.12.15.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 38 - 2015 - Monitoramento dos casos de microcefalias no Brasil, até a semana epidemiológica 47, 2015</a></p>
<p><a href="/images/pdf/2015/novembro/30/Microcefalia-2-boletim.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 37 - 2015 - Monitoramento dos casos de microcefalias no Brasil, até a semana epidemiológica 46, 2015</a></p>
<p><a href="/images/pdf/2015/novembro/26/2015-dengue-SE45.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 36 - 2015 - Monitoramento dos casos de dengue, febre de chikungunya e febre pelo vírus Zika até a Semana Epidemiológica 45, 2015</a></p>
<p><a href="/images/pdf/2015/novembro/19/Microcefalia-bol-final.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 34 - 2015 - Situação epidemiológica de ocorrência de microcefalias no Brasil, 2015</a></p>
<p><a href="/images/pdf/2015/agosto/26/2015-020-publica----o.pdf" target="_blank" rel="noopener noreferrer">Vol. 46 - Nº 26 - 2015 - Febre pelo vírus Zika: uma revisão narrativa sobre a doença</a></p>
</div>
	</li>
	</ul>
</div></div>
'''

In [66]:
sopa = BeautifulSoup(data2)

/home/tbnsilveira/anaconda3/envs/analytics3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/tbnsilveira/anaconda3/envs/analytics3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [ ]:
#soup = BeautifulSoup(''.join(page), 'lxml')
for link in soup.find_all('a', href=True):
    if link['href'].lower().endswith(".pdf"):
        if link['href'].lower().startswith("http"):
            print(link['href'])

----------------------

#### Testing the "selector" from the real page:

In [149]:
section = soup.find('h3', string='Coronavírus/COVID-19')

In [144]:
section = soup.find('h3', string='Dengue')

In [150]:
section_parent = section.find_parent()

In [151]:
section_parent.find_all('a')

[<a href="#">Coronavírus/COVID-19</a>,
 <a href="https://portalarquivos.saude.gov.br/images/pdf/2020/April/19/BE12-Boletim-do-COE.pdf">BE 12 - Boletim COE Coronavírus</a>,
 <a href="/images/pdf/2020/April/18/2020-04-17---BE11---Boletim-do-COE-21h.pdf" target="_blank">BE 11 - Boletim COE Coronavírus</a>,
 <a href="https://portalarquivos.saude.gov.br/images/pdf/2020/April/17/2020-04-16---BE10---Boletim-do-COE-21h.pdf" rel="noopener" target="_blank">BE 10 - Boletim COE Coronavirus</a>,
 <a href="https://portalarquivos.saude.gov.br/images/pdf/2020/April/12/2020-04-11-BE9-Boletim-do-COE.pdf" rel="noopener" target="_blank">BE9 - Boletim Especial do COE Coronavírus Avaliação de Risco</a>,
 <a href="/images/pdf/2020/April/09/be-covid-08-final-2.pdf" target="_blank">BE8 - Boletim Especial do COE Coronavírus Avaliação de Risco</a>,
 <a href="/images/pdf/2020/April/09/be-covid-08-final.pdf" target="_blank"></a>,
 <a href="/images/pdf/2020/April/06/2020-04-06-BE7-Boletim-Especial-do-COE-Atualizaca

In [147]:
for report in section_parent.find_all('a'):
    print(report['href'])
    print(report.contents,'\n')

#
['Dengue'] 

/images/pdf/2020/April/16/Boletim-epidemiologico-SVS-16.pdf
['BE Vol. 51 Nº 16 -\xa0Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes Aegypti (dengue, chikungunya e zika), Semanas Epidemiológicas 1 a 15, 2020'] 

/images/pdf/2020/April/13/Boletim-epidemiologico-SVS-15.pdf
['BE Vol. 51 Nº 15 - Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes Aegypti (dengue, chikungunya e zika), Semanas Epidemiológicas 1 a 14 de 2020, e da febre amarela (2019/2020)'] 

/images/pdf/2020/April/03/Boletim-epidemiologico-SVS-14.pdf
['BE Vol. 51 Nº 14 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 13, 2020; febre amarela; diferenças regionais no consumo de alimentos entre adolescentes e adultos e leishmaniose visceral: Experiência do IEC'] 

/images/pdf/2020/marco/27/Boletim-epidemiologico-SVS-13.pdf
['BE Vol.51 Nº 13 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 12, 2020; febre amarela; análise da anomalia

In [152]:
for link in section_parent.find_all('a', href=True):
    if link['href'].lower().endswith(".pdf"):
        if link['href'].lower().startswith("http"):
            print(link['href'])

https://portalarquivos.saude.gov.br/images/pdf/2020/April/19/BE12-Boletim-do-COE.pdf
https://portalarquivos.saude.gov.br/images/pdf/2020/April/17/2020-04-16---BE10---Boletim-do-COE-21h.pdf
https://portalarquivos.saude.gov.br/images/pdf/2020/April/12/2020-04-11-BE9-Boletim-do-COE.pdf
https://portalarquivos.saude.gov.br/images/pdf/2020/April/03/BE6-Boletim-Especial-do-COE.pdf
https://portalarquivos2.saude.gov.br/images/pdf/2020/fevereiro/13/Boletim-epidemiologico-COEcorona-SVS-13fev20.pdf


In [154]:
for link in section_parent.find_all('a', href=True):
    if link['href'].lower().endswith(".pdf"):
        if link['href'].lower().startswith("http"):
            print(link['href'])
        elif link['href'].lower().startswith("/images/"):
            print('https://portalarquivos.saude.gov.br/'+link['href'])

https://portalarquivos.saude.gov.br/images/pdf/2020/April/19/BE12-Boletim-do-COE.pdf
https://portalarquivos.saude.gov.br//images/pdf/2020/April/18/2020-04-17---BE11---Boletim-do-COE-21h.pdf
https://portalarquivos.saude.gov.br/images/pdf/2020/April/17/2020-04-16---BE10---Boletim-do-COE-21h.pdf
https://portalarquivos.saude.gov.br/images/pdf/2020/April/12/2020-04-11-BE9-Boletim-do-COE.pdf
https://portalarquivos.saude.gov.br//images/pdf/2020/April/09/be-covid-08-final-2.pdf
https://portalarquivos.saude.gov.br//images/pdf/2020/April/09/be-covid-08-final.pdf
https://portalarquivos.saude.gov.br//images/pdf/2020/April/06/2020-04-06-BE7-Boletim-Especial-do-COE-Atualizacao-da-Avaliacao-de-Risco.pdf
https://portalarquivos.saude.gov.br/images/pdf/2020/April/03/BE6-Boletim-Especial-do-COE.pdf
https://portalarquivos.saude.gov.br//images/pdf/2020/marco/21/2020-03-13-Boletim-Epidemiologico-05.pdf
https://portalarquivos.saude.gov.br//images/pdf/2020/marco/04/2020-03-02-Boletim-Epidemiol--gico-04-corrig

--------------------------_

In [103]:
ul = sopa.find("h3", text="Asbestose")

In [104]:
teste = ul.find_parent()

In [105]:
teste.find_all('a')

[<a href="#">Asbestose</a>,
 <a href="/images/pdf/2016/fevereiro/02/2015-011---Asbestose.pdf" rel="noopener noreferrer" target="_blank">Vol. 47 - Nº 4 - 2016 - Asbestose na saúde do trabalhador notificados no Sinan, Brasil, 2007-2013</a>]

In [80]:
ul.find_next_siblings()

[<div class="ui-accordion-content ui-helper-reset ui-widget-content ui-corner-bottom ui-accordion-content-active" style="display: block;">
 <p><a href="/images/pdf/2020/April/16/Boletim-epidemiologico-SVS-16.pdf" target="_blank">BE Vol. 51 Nº 16 - Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes Aegypti (dengue, chikungunya e zika), Semanas Epidemiológicas 1 a 15, 2020</a></p>
 <p><a href="/images/pdf/2020/April/13/Boletim-epidemiologico-SVS-15.pdf" target="_blank">BE Vol. 51 Nº 15 - Monitoramento dos casos de arboviroses urbanas transmitidas pelo Aedes Aegypti (dengue, chikungunya e zika), Semanas Epidemiológicas 1 a 14 de 2020, e da febre amarela (2019/2020)</a></p>
 <p><a href="/images/pdf/2020/April/03/Boletim-epidemiologico-SVS-14.pdf">BE Vol. 51 Nº 14 - Monitoramento das arboviroses (dengue, chikungunya e zika), SE 1 a 13, 2020; febre amarela; diferenças regionais no consumo de alimentos entre adolescentes e adultos e leishmaniose visceral: Experiência do IE

In [69]:
for li in ul.find_all("li"):
    print(li.span.get_text(), li.a.get_text())

AttributeError: 'NoneType' object has no attribute 'find_all'

In [61]:
for div in sopa.findAll('div', attrs={'class':'image'}):
    print(div.find('a')['href'],'\n')
    print(div.find('a').contents[0])
    print(div.find('img')['src'])

http://www.example.com/eg1 

Content1
http://image.example.com/img1.jpg
http://www.example.com/eg2 

Content2
http://image.example.com/img2.jpg


### Downloading the files
At first, someone could use the os.system library ('wget http ...') to download each file. However, to allow the portability of the code in different operating systems, it is recommended to use the Python libraries themselves, whenever possible. 
[Ref.: http://stackoverflow.com/questions/2467609/using-wget-via-python]

In [49]:
import urllib

In [43]:
# Teste da linha de comando para incluir no loop:
urllib.urlretrieve(url + tags[2].get('href'), filename='apostila/'+tags[2].get('href'))

('apostila/nlpintro_co3_01.03_DR_Edit.pdf',
 <httplib.HTTPMessage instance at 0x7f9fe198f200>)

In [44]:
# Loop para o download de todos os links:
for tag in tags:
    nome_arquivo = tag.get('href')
    urllib.urlretrieve(url + nome_arquivo, filename='apostila/'+nome_arquivo)

## Conclusão
81 clicks were 'saved' ;)